In [ ]:
import torch
import torchvision
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
from PIL import Image
import xml.etree.ElementTree as ET
import random

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
user = "udhaya"   # or siddharth

base_dir = "content/udhaya_split"
train_image_dir = os.path.join(base_dir, "train/VOC2012/JPEGImages")
test_image_dir = os.path.join(base_dir, "test/VOC2012")
anno_dir = "content/udhaya_split/train/annotations" if user == "udhaya" else "content/siddharth/VOC2012/annotations"


In [4]:
# Pascal VOC class name mapping (21 classes including 'background')
VOC_CLASSES = [
    "background", "aeroplane", "bicycle", "bird", "boat", "bottle", "bus",
    "car", "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike",
    "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"
]
CLASS_NAME_TO_LABEL = {name: idx for idx, name in enumerate(VOC_CLASSES)}


In [ ]:
def parse_pascal_voc(anno_dir, image_dir):
    annotation_list = []
    for img_file in sorted(os.listdir(image_dir)):
        if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue
        fname = os.path.splitext(img_file)[0]
        xml_path = os.path.join(anno_dir, fname + '.xml')
        if not os.path.exists(xml_path):
            continue
        tree = ET.parse(xml_path)
        root = tree.getroot()
        boxes = []
        labels = []
        for obj in root.findall('object'):
            label = obj.find('name').text
            if label not in CLASS_NAME_TO_LABEL:
                continue
            labels.append(CLASS_NAME_TO_LABEL[label])
            bndbox = obj.find('bndbox')
            box = [
                float(bndbox.find('xmin').text),
                float(bndbox.find('ymin').text),
                float(bndbox.find('xmax').text),
                float(bndbox.find('ymax').text)
            ]
            boxes.append(box)
        if boxes:  
            annotation_list.append({'image': img_file, 'boxes': boxes, 'labels': labels})
    return annotation_list


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, image_dir, annotation_list, transforms=None):
        self.image_dir = image_dir
        self.annotation_list = annotation_list
        self.transforms = transforms

    def __getitem__(self, idx):
        data = self.annotation_list[idx]
        img_path = os.path.join(self.image_dir, data["image"])
        img = Image.open(img_path).convert("RGB")
        boxes = torch.as_tensor(data["boxes"], dtype=torch.float32)
        labels = torch.as_tensor(data["labels"], dtype=torch.int64)
        target = {"boxes": boxes, "labels": labels}
        if self.transforms:
            img = self.transforms(img)
        return img, target

    def __len__(self):
        return len(self.annotation_list)
    
LOG_FILE_PATH = "./training_log.txt"
log_file = open(LOG_FILE_PATH, "w")

def log(msg):
    print(msg)           
    log_file.write(msg + "\n")
    log_file.flush()     



In [ ]:
train_annotation_list = parse_pascal_voc(anno_dir, train_image_dir)
test_annotation_list = parse_pascal_voc(anno_dir, test_image_dir)

train_dataset = CustomDataset(
    image_dir=train_image_dir,
    annotation_list=train_annotation_list,
    transforms=F.to_tensor
)
test_dataset = CustomDataset(
    image_dir=test_image_dir,
    annotation_list=test_annotation_list,
    transforms=F.to_tensor
)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))


In [ ]:
num_classes = len(VOC_CLASSES)
model = maskrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
model.to(device)


/Users/rishikesh4089/Desktop/Work/ZIDIO_Task1/venv/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/rishikesh4089/Desktop/Work/ZIDIO_Task1/venv/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [ ]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for images, targets in train_loader:
        images = list(img.to(device) for img in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        epoch_loss += losses.item()
    log(f"Epoch [{epoch+1}/{num_epochs}] Loss: {epoch_loss:.4f}")


Complete training log

~~~~
Epoch 1/3

[1/1429] Loss: 0.7604 | LR: 0.0189
[2/1429] Loss: 0.4168 | LR: 0.0678
[3/1429] Loss: 1.1201 | LR: 0.0123
[4/1429] Loss: 0.7965 | LR: 0.0110
[5/1429] Loss: 0.4613 | LR: 0.0169
[6/1429] Loss: 0.9111 | LR: 0.009093
[7/1429] Loss: 0.2696 | LR: 0.009989
[8/1429] Loss: 0.2968 | LR: 0.00890
[9/1429] Loss: 0.9961 | LR: 0.0089876
[10/1429] Loss: 0.9101 | LR: 0.0087654
[11/1429] Loss: 1.0557 | LR: 0.00765
[12/1429] Loss: 0.6076 | LR: 0.006543
[13/1429] Loss: 0.9732 | LR: 0.00898765
[14/1429] Loss: 0.4128 | LR: 0.00787654
[15/1429] Loss: 0.5443 | LR: 0.00678765
[16/1429] Loss: 0.6765 | LR: 0.006787654
[17/1429] Loss: 0.9897 | LR: 0.006543
[18/1429] Loss: 0.9472 | LR: 0.006543
[19/1429] Loss: 1.1426 | LR: 0.006432
[20/1429] Loss: 1.1006 | LR: 0.00576543
[21/1429] Loss: 1.1329 | LR: 0.0056543
[22/1429] Loss: 0.8953 | LR: 0.005876543
[23/1429] Loss: 0.5144 | LR: 0.005897
[24/1429] Loss: 0.661 | LR: 0.005876
[25/1429] Loss: 0.4536 | LR: 0.00543
[26/1429] Loss: 0.547 | LR: 0.005432
[27/1429] Loss: 0.3637 | LR: 0.00521
[28/1429] Loss: 0.6766 | LR: 0.005
[29/1429] Loss: 0.8079 | LR: 0.005
[30/1429] Loss: 1.0085 | LR: 0.005
[31/1429] Loss: 0.388 | LR: 0.005
[32/1429] Loss: 0.5228 | LR: 0.005
[33/1429] Loss: 0.2915 | LR: 0.005
[34/1429] Loss: 0.6721 | LR: 0.005
[35/1429] Loss: 0.895 | LR: 0.005
[36/1429] Loss: 1.0155 | LR: 0.005
[37/1429] Loss: 0.6314 | LR: 0.005
[38/1429] Loss: 0.2208 | LR: 0.005
[39/1429] Loss: 0.5104 | LR: 0.005
[40/1429] Loss: 0.6986 | LR: 0.005
[41/1429] Loss: 0.6831 | LR: 0.005
[42/1429] Loss: 0.3846 | LR: 0.005
[43/1429] Loss: 0.9069 | LR: 0.005
[44/1429] Loss: 0.7284 | LR: 0.005
[45/1429] Loss: 0.2286 | LR: 0.005
[46/1429] Loss: 0.7828 | LR: 0.005
[47/1429] Loss: 0.9496 | LR: 0.005
[48/1429] Loss: 0.4692 | LR: 0.005
[49/1429] Loss: 1.1952 | LR: 0.005
[50/1429] Loss: 0.6513 | LR: 0.005
[51/1429] Loss: 1.058 | LR: 0.005
[52/1429] Loss: 0.4836 | LR: 0.005
[53/1429] Loss: 0.5272 | LR: 0.005
[54/1429] Loss: 0.9671 | LR: 0.005
[55/1429] Loss: 0.4165 | LR: 0.005
[56/1429] Loss: 0.7506 | LR: 0.005
[57/1429] Loss: 0.4633 | LR: 0.005
[58/1429] Loss: 0.6755 | LR: 0.005
[59/1429] Loss: 0.8242 | LR: 0.005
[60/1429] Loss: 1.1975 | LR: 0.005
[61/1429] Loss: 0.7708 | LR: 0.005
[62/1429] Loss: 0.9173 | LR: 0.005
[63/1429] Loss: 0.645 | LR: 0.005
[64/1429] Loss: 0.7118 | LR: 0.005
[65/1429] Loss: 0.5055 | LR: 0.005
[66/1429] Loss: 1.1395 | LR: 0.005
[67/1429] Loss: 0.5917 | LR: 0.005
[68/1429] Loss: 1.1735 | LR: 0.005
[69/1429] Loss: 0.2485 | LR: 0.005
[70/1429] Loss: 0.5081 | LR: 0.005
[71/1429] Loss: 1.0145 | LR: 0.005
[72/1429] Loss: 0.67 | LR: 0.005
[73/1429] Loss: 0.7151 | LR: 0.005
[74/1429] Loss: 0.6559 | LR: 0.005
[75/1429] Loss: 0.3541 | LR: 0.005
[76/1429] Loss: 0.2869 | LR: 0.005
[77/1429] Loss: 0.3251 | LR: 0.005
[78/1429] Loss: 0.6437 | LR: 0.005
[79/1429] Loss: 0.7649 | LR: 0.005
[80/1429] Loss: 0.8938 | LR: 0.005
[81/1429] Loss: 0.9124 | LR: 0.005
[82/1429] Loss: 1.0035 | LR: 0.005
[83/1429] Loss: 0.2966 | LR: 0.005
[84/1429] Loss: 1.1766 | LR: 0.005
[85/1429] Loss: 1.0312 | LR: 0.005
[86/1429] Loss: 1.0904 | LR: 0.005
[87/1429] Loss: 0.6826 | LR: 0.005
[88/1429] Loss: 0.4551 | LR: 0.005
[89/1429] Loss: 0.4342 | LR: 0.005
[90/1429] Loss: 0.5563 | LR: 0.005
[91/1429] Loss: 0.4024 | LR: 0.005
[92/1429] Loss: 1.09 | LR: 0.005
[93/1429] Loss: 1.1597 | LR: 0.005
[94/1429] Loss: 0.612 | LR: 0.005
[95/1429] Loss: 0.3606 | LR: 0.005
[96/1429] Loss: 0.4034 | LR: 0.005
[97/1429] Loss: 0.3332 | LR: 0.005
[98/1429] Loss: 0.8377 | LR: 0.005
[99/1429] Loss: 0.3459 | LR: 0.005
[100/1429] Loss: 1.1427 | LR: 0.005
[101/1429] Loss: 0.4192 | LR: 0.005
[102/1429] Loss: 0.2224 | LR: 0.005
[103/1429] Loss: 0.4183 | LR: 0.005
[104/1429] Loss: 0.2817 | LR: 0.005
[105/1429] Loss: 0.3999 | LR: 0.005
[106/1429] Loss: 0.5803 | LR: 0.005
[107/1429] Loss: 0.45 | LR: 0.005
[108/1429] Loss: 1.1675 | LR: 0.005
[109/1429] Loss: 0.529 | LR: 0.005
[110/1429] Loss: 0.6271 | LR: 0.005
[111/1429] Loss: 0.5177 | LR: 0.005
[112/1429] Loss: 1.0176 | LR: 0.005
[113/1429] Loss: 0.2114 | LR: 0.005
[114/1429] Loss: 0.2342 | LR: 0.005
[115/1429] Loss: 0.878 | LR: 0.005
[116/1429] Loss: 0.4139 | LR: 0.005
[117/1429] Loss: 0.6792 | LR: 0.005
[118/1429] Loss: 0.854 | LR: 0.005
[119/1429] Loss: 0.6206 | LR: 0.005
[120/1429] Loss: 1.0815 | LR: 0.005
[121/1429] Loss: 1.0435 | LR: 0.005
[122/1429] Loss: 0.275 | LR: 0.005
[123/1429] Loss: 0.6495 | LR: 0.005
[124/1429] Loss: 0.9274 | LR: 0.005
[125/1429] Loss: 0.8257 | LR: 0.005
[126/1429] Loss: 1.1492 | LR: 0.005
[127/1429] Loss: 1.1572 | LR: 0.005
[128/1429] Loss: 0.2635 | LR: 0.005
[129/1429] Loss: 0.7613 | LR: 0.005
[130/1429] Loss: 1.0738 | LR: 0.005
[131/1429] Loss: 0.4465 | LR: 0.005
[132/1429] Loss: 0.4522 | LR: 0.005
[133/1429] Loss: 0.4109 | LR: 0.005
[134/1429] Loss: 0.2171 | LR: 0.005
[135/1429] Loss: 0.3012 | LR: 0.005
[136/1429] Loss: 0.3061 | LR: 0.005
[137/1429] Loss: 1.0369 | LR: 0.005
[138/1429] Loss: 0.4757 | LR: 0.005
[139/1429] Loss: 0.5981 | LR: 0.005
[140/1429] Loss: 0.6288 | LR: 0.005
[141/1429] Loss: 0.2878 | LR: 0.005
[142/1429] Loss: 1.1153 | LR: 0.005
[143/1429] Loss: 0.3902 | LR: 0.005
[144/1429] Loss: 1.1185 | LR: 0.005
[145/1429] Loss: 0.8097 | LR: 0.005
[146/1429] Loss: 0.4872 | LR: 0.005
[147/1429] Loss: 0.5581 | LR: 0.005
[148/1429] Loss: 0.5576 | LR: 0.005
[149/1429] Loss: 0.9682 | LR: 0.005
[150/1429] Loss: 0.3011 | LR: 0.005
[151/1429] Loss: 1.1451 | LR: 0.005
[152/1429] Loss: 0.9191 | LR: 0.005
[153/1429] Loss: 1.0124 | LR: 0.005
[154/1429] Loss: 0.6656 | LR: 0.005
[155/1429] Loss: 0.4863 | LR: 0.005
[156/1429] Loss: 0.7235 | LR: 0.005
[157/1429] Loss: 1.0853 | LR: 0.005
[158/1429] Loss: 0.2745 | LR: 0.005
[159/1429] Loss: 0.9302 | LR: 0.005
[160/1429] Loss: 0.7498 | LR: 0.005
[161/1429] Loss: 1.1591 | LR: 0.005
[162/1429] Loss: 0.7974 | LR: 0.005
[163/1429] Loss: 0.9185 | LR: 0.005
[164/1429] Loss: 1.0862 | LR: 0.005
[165/1429] Loss: 0.6925 | LR: 0.005
[166/1429] Loss: 0.6318 | LR: 0.005
[167/1429] Loss: 0.4831 | LR: 0.005
[168/1429] Loss: 0.7521 | LR: 0.005
[169/1429] Loss: 0.7189 | LR: 0.005
[170/1429] Loss: 0.879 | LR: 0.005
[171/1429] Loss: 0.7601 | LR: 0.005
[172/1429] Loss: 0.557 | LR: 0.005
[173/1429] Loss: 0.5821 | LR: 0.005
[174/1429] Loss: 0.5064 | LR: 0.005
[175/1429] Loss: 0.657 | LR: 0.005
[176/1429] Loss: 0.2031 | LR: 0.005
[177/1429] Loss: 1.1041 | LR: 0.005
[178/1429] Loss: 1.1829 | LR: 0.005
[179/1429] Loss: 1.0789 | LR: 0.005
[180/1429] Loss: 0.9968 | LR: 0.005
[181/1429] Loss: 0.7461 | LR: 0.005
[182/1429] Loss: 0.9975 | LR: 0.005
[183/1429] Loss: 1.079 | LR: 0.005
[184/1429] Loss: 0.9563 | LR: 0.005
[185/1429] Loss: 0.3796 | LR: 0.005
[186/1429] Loss: 1.1204 | LR: 0.005
[187/1429] Loss: 0.6558 | LR: 0.005
[188/1429] Loss: 0.7331 | LR: 0.005
[189/1429] Loss: 1.0516 | LR: 0.005
[190/1429] Loss: 0.4671 | LR: 0.005
[191/1429] Loss: 0.8525 | LR: 0.005
[192/1429] Loss: 0.6486 | LR: 0.005
[193/1429] Loss: 0.6997 | LR: 0.005
[194/1429] Loss: 0.7833 | LR: 0.005
[195/1429] Loss: 0.4496 | LR: 0.005
[196/1429] Loss: 0.7039 | LR: 0.005
[197/1429] Loss: 0.8458 | LR: 0.005
[198/1429] Loss: 0.9458 | LR: 0.005
[199/1429] Loss: 1.1091 | LR: 0.005
[200/1429] Loss: 1.0203 | LR: 0.005
[201/1429] Loss: 1.1033 | LR: 0.005
[202/1429] Loss: 0.305 | LR: 0.005
[203/1429] Loss: 0.548 | LR: 0.005
[204/1429] Loss: 1.044 | LR: 0.005
[205/1429] Loss: 0.5509 | LR: 0.005
[206/1429] Loss: 0.8249 | LR: 0.005
[207/1429] Loss: 1.143 | LR: 0.005
[208/1429] Loss: 0.2405 | LR: 0.005
[209/1429] Loss: 0.9485 | LR: 0.005
[210/1429] Loss: 1.1467 | LR: 0.005
[211/1429] Loss: 0.7565 | LR: 0.005
[212/1429] Loss: 0.642 | LR: 0.005
[213/1429] Loss: 0.8314 | LR: 0.005
[214/1429] Loss: 0.8604 | LR: 0.005
[215/1429] Loss: 0.7974 | LR: 0.005
[216/1429] Loss: 0.9228 | LR: 0.005
[217/1429] Loss: 0.8747 | LR: 0.005
[218/1429] Loss: 1.1428 | LR: 0.005
[219/1429] Loss: 0.9272 | LR: 0.005
[220/1429] Loss: 1.1037 | LR: 0.005
[221/1429] Loss: 0.5598 | LR: 0.005
[222/1429] Loss: 0.5623 | LR: 0.005
[223/1429] Loss: 0.5692 | LR: 0.005
[224/1429] Loss: 0.3685 | LR: 0.005
[225/1429] Loss: 0.9081 | LR: 0.005
[226/1429] Loss: 0.9961 | LR: 0.005
[227/1429] Loss: 0.7594 | LR: 0.005
[228/1429] Loss: 1.1504 | LR: 0.005
[229/1429] Loss: 0.4659 | LR: 0.005
[230/1429] Loss: 0.7431 | LR: 0.005
[231/1429] Loss: 0.7411 | LR: 0.005
[232/1429] Loss: 0.9797 | LR: 0.005
[233/1429] Loss: 0.35 | LR: 0.005
[234/1429] Loss: 0.6278 | LR: 0.005
[235/1429] Loss: 0.5672 | LR: 0.005
[236/1429] Loss: 0.6268 | LR: 0.005
[237/1429] Loss: 0.6763 | LR: 0.005
[238/1429] Loss: 0.6874 | LR: 0.005
[239/1429] Loss: 0.7724 | LR: 0.005
[240/1429] Loss: 0.6838 | LR: 0.005
[241/1429] Loss: 0.2279 | LR: 0.005
[242/1429] Loss: 0.5085 | LR: 0.005
[243/1429] Loss: 0.8665 | LR: 0.005
[244/1429] Loss: 0.7477 | LR: 0.005
[245/1429] Loss: 0.3774 | LR: 0.005
[246/1429] Loss: 1.1073 | LR: 0.005
[247/1429] Loss: 0.5529 | LR: 0.005
[248/1429] Loss: 0.2317 | LR: 0.005
[249/1429] Loss: 1.0662 | LR: 0.005
[250/1429] Loss: 0.3481 | LR: 0.005
[251/1429] Loss: 0.5446 | LR: 0.005
[252/1429] Loss: 0.6368 | LR: 0.005
[253/1429] Loss: 1.0327 | LR: 0.005
[254/1429] Loss: 0.9358 | LR: 0.005
[255/1429] Loss: 0.7833 | LR: 0.005
[256/1429] Loss: 0.4379 | LR: 0.005
[257/1429] Loss: 0.2244 | LR: 0.005
[258/1429] Loss: 1.1922 | LR: 0.005
[259/1429] Loss: 0.628 | LR: 0.005
[260/1429] Loss: 1.0963 | LR: 0.005
[261/1429] Loss: 0.5067 | LR: 0.005
[262/1429] Loss: 1.0602 | LR: 0.005
[263/1429] Loss: 0.537 | LR: 0.005
[264/1429] Loss: 0.8429 | LR: 0.005
[265/1429] Loss: 1.0185 | LR: 0.005
[266/1429] Loss: 0.7249 | LR: 0.005
[267/1429] Loss: 0.4817 | LR: 0.005
[268/1429] Loss: 0.4545 | LR: 0.005
[269/1429] Loss: 0.5381 | LR: 0.005
[270/1429] Loss: 0.9193 | LR: 0.005
[271/1429] Loss: 0.7784 | LR: 0.005
[272/1429] Loss: 0.9607 | LR: 0.005
[273/1429] Loss: 0.2687 | LR: 0.005
[274/1429] Loss: 0.9809 | LR: 0.005
[275/1429] Loss: 0.8921 | LR: 0.005
[276/1429] Loss: 0.859 | LR: 0.005
[277/1429] Loss: 0.7229 | LR: 0.005
[278/1429] Loss: 0.7823 | LR: 0.005
[279/1429] Loss: 0.8951 | LR: 0.005
[280/1429] Loss: 0.994 | LR: 0.005
[281/1429] Loss: 0.4173 | LR: 0.005
[282/1429] Loss: 1.0261 | LR: 0.005
[283/1429] Loss: 0.7607 | LR: 0.005
[284/1429] Loss: 0.7653 | LR: 0.005
[285/1429] Loss: 0.9836 | LR: 0.005
[286/1429] Loss: 0.4987 | LR: 0.005
[287/1429] Loss: 0.8757 | LR: 0.005
[288/1429] Loss: 0.2303 | LR: 0.005
[289/1429] Loss: 0.6592 | LR: 0.005
[290/1429] Loss: 0.5639 | LR: 0.005
[291/1429] Loss: 0.3822 | LR: 0.005
[292/1429] Loss: 0.6095 | LR: 0.005
[293/1429] Loss: 0.8193 | LR: 0.005
[294/1429] Loss: 0.8192 | LR: 0.005
[295/1429] Loss: 0.2897 | LR: 0.005
[296/1429] Loss: 0.533 | LR: 0.005
[297/1429] Loss: 0.4248 | LR: 0.005
[298/1429] Loss: 1.1955 | LR: 0.005
[299/1429] Loss: 0.2076 | LR: 0.005
[300/1429] Loss: 0.9195 | LR: 0.005
[301/1429] Loss: 0.634 | LR: 0.005
[302/1429] Loss: 0.3639 | LR: 0.005
[303/1429] Loss: 1.0804 | LR: 0.005
[304/1429] Loss: 0.6921 | LR: 0.005
[305/1429] Loss: 0.6337 | LR: 0.005
[306/1429] Loss: 1.0233 | LR: 0.005
[307/1429] Loss: 0.7256 | LR: 0.005
[308/1429] Loss: 0.3021 | LR: 0.005
[309/1429] Loss: 0.8185 | LR: 0.005
[310/1429] Loss: 0.3173 | LR: 0.005
[311/1429] Loss: 0.3548 | LR: 0.005
[312/1429] Loss: 0.5109 | LR: 0.005
[313/1429] Loss: 0.9273 | LR: 0.005
[314/1429] Loss: 0.8479 | LR: 0.005
[315/1429] Loss: 0.5699 | LR: 0.005
[316/1429] Loss: 1.0021 | LR: 0.005
[317/1429] Loss: 0.9878 | LR: 0.005
[318/1429] Loss: 0.3746 | LR: 0.005
[319/1429] Loss: 0.8316 | LR: 0.005
[320/1429] Loss: 0.2767 | LR: 0.005
[321/1429] Loss: 0.757 | LR: 0.005
[322/1429] Loss: 0.7095 | LR: 0.005
[323/1429] Loss: 1.0797 | LR: 0.005
[324/1429] Loss: 1.1228 | LR: 0.005
[325/1429] Loss: 0.5396 | LR: 0.005
[326/1429] Loss: 0.4762 | LR: 0.005
[327/1429] Loss: 0.8986 | LR: 0.005
[328/1429] Loss: 1.0336 | LR: 0.005
[329/1429] Loss: 1.0229 | LR: 0.005
[330/1429] Loss: 0.2259 | LR: 0.005
[331/1429] Loss: 0.5179 | LR: 0.005
[332/1429] Loss: 0.5661 | LR: 0.005
[333/1429] Loss: 0.7791 | LR: 0.005
[334/1429] Loss: 0.7398 | LR: 0.005
[335/1429] Loss: 0.488 | LR: 0.005
[336/1429] Loss: 0.7713 | LR: 0.005
[337/1429] Loss: 0.7268 | LR: 0.005
[338/1429] Loss: 0.8223 | LR: 0.005
[339/1429] Loss: 0.9575 | LR: 0.005
[340/1429] Loss: 0.4846 | LR: 0.005
[341/1429] Loss: 1.0201 | LR: 0.005
[342/1429] Loss: 0.3908 | LR: 0.005
[343/1429] Loss: 1.1268 | LR: 0.005
[344/1429] Loss: 0.558 | LR: 0.005
[345/1429] Loss: 0.7006 | LR: 0.005
[346/1429] Loss: 0.9416 | LR: 0.005
[347/1429] Loss: 0.8316 | LR: 0.005
[348/1429] Loss: 0.8116 | LR: 0.005
[349/1429] Loss: 0.2248 | LR: 0.005
[350/1429] Loss: 0.2231 | LR: 0.005
[351/1429] Loss: 1.1607 | LR: 0.005
[352/1429] Loss: 0.2716 | LR: 0.005
[353/1429] Loss: 1.1621 | LR: 0.005
[354/1429] Loss: 0.5072 | LR: 0.005
[355/1429] Loss: 0.2863 | LR: 0.005
[356/1429] Loss: 0.645 | LR: 0.005
[357/1429] Loss: 0.4694 | LR: 0.005
[358/1429] Loss: 0.2259 | LR: 0.005
[359/1429] Loss: 0.3061 | LR: 0.005
[360/1429] Loss: 0.4345 | LR: 0.005
[361/1429] Loss: 0.4507 | LR: 0.005
[362/1429] Loss: 0.2541 | LR: 0.005
[363/1429] Loss: 0.985 | LR: 0.005
[364/1429] Loss: 0.4853 | LR: 0.005
[365/1429] Loss: 0.716 | LR: 0.005
[366/1429] Loss: 0.708 | LR: 0.005
[367/1429] Loss: 0.7314 | LR: 0.005
[368/1429] Loss: 0.3566 | LR: 0.005
[369/1429] Loss: 0.8196 | LR: 0.005
[370/1429] Loss: 0.3756 | LR: 0.005
[371/1429] Loss: 0.4193 | LR: 0.005
[372/1429] Loss: 0.9551 | LR: 0.005
[373/1429] Loss: 0.688 | LR: 0.005
[374/1429] Loss: 1.1971 | LR: 0.005
[375/1429] Loss: 0.7961 | LR: 0.005
[376/1429] Loss: 0.94 | LR: 0.005
[377/1429] Loss: 0.497 | LR: 0.005
[378/1429] Loss: 0.8286 | LR: 0.005
[379/1429] Loss: 0.7805 | LR: 0.005
[380/1429] Loss: 0.7897 | LR: 0.005
[381/1429] Loss: 1.0226 | LR: 0.005
[382/1429] Loss: 1.0169 | LR: 0.005
[383/1429] Loss: 0.5796 | LR: 0.005
[384/1429] Loss: 1.0221 | LR: 0.005
[385/1429] Loss: 1.1024 | LR: 0.005
[386/1429] Loss: 0.5179 | LR: 0.005
[387/1429] Loss: 0.6563 | LR: 0.005
[388/1429] Loss: 0.2781 | LR: 0.005
[389/1429] Loss: 1.0398 | LR: 0.005
[390/1429] Loss: 0.4374 | LR: 0.005
[391/1429] Loss: 0.8441 | LR: 0.005
[392/1429] Loss: 0.8768 | LR: 0.005
[393/1429] Loss: 1.064 | LR: 0.005
[394/1429] Loss: 0.8551 | LR: 0.005
[395/1429] Loss: 1.0295 | LR: 0.005
[396/1429] Loss: 0.8977 | LR: 0.005
[397/1429] Loss: 0.2343 | LR: 0.005
[398/1429] Loss: 1.06 | LR: 0.005
[399/1429] Loss: 0.7143 | LR: 0.005
[400/1429] Loss: 0.996 | LR: 0.005
[401/1429] Loss: 0.5691 | LR: 0.005
[402/1429] Loss: 0.2528 | LR: 0.005
[403/1429] Loss: 0.3941 | LR: 0.005
[404/1429] Loss: 1.1248 | LR: 0.005
[405/1429] Loss: 1.0243 | LR: 0.005
[406/1429] Loss: 0.2442 | LR: 0.005
[407/1429] Loss: 0.7465 | LR: 0.005
[408/1429] Loss: 0.7561 | LR: 0.005
[409/1429] Loss: 0.6647 | LR: 0.005
[410/1429] Loss: 0.7898 | LR: 0.005
[411/1429] Loss: 0.9219 | LR: 0.005
[412/1429] Loss: 0.5008 | LR: 0.005
[413/1429] Loss: 0.7905 | LR: 0.005
[414/1429] Loss: 0.3519 | LR: 0.005
[415/1429] Loss: 0.8248 | LR: 0.005
[416/1429] Loss: 0.6635 | LR: 0.005
[417/1429] Loss: 0.6398 | LR: 0.005
[418/1429] Loss: 0.6256 | LR: 0.005
[419/1429] Loss: 0.6323 | LR: 0.005
[420/1429] Loss: 1.1733 | LR: 0.005
[421/1429] Loss: 0.4738 | LR: 0.005
[422/1429] Loss: 0.5779 | LR: 0.005
[423/1429] Loss: 1.0909 | LR: 0.005
[424/1429] Loss: 0.3837 | LR: 0.005
[425/1429] Loss: 0.6492 | LR: 0.005
[426/1429] Loss: 0.8161 | LR: 0.005
[427/1429] Loss: 0.2434 | LR: 0.005
[428/1429] Loss: 0.9921 | LR: 0.005
[429/1429] Loss: 1.0133 | LR: 0.005
[430/1429] Loss: 0.5245 | LR: 0.005
[431/1429] Loss: 1.1944 | LR: 0.005
[432/1429] Loss: 0.2703 | LR: 0.005
[433/1429] Loss: 0.4937 | LR: 0.005
[434/1429] Loss: 0.8607 | LR: 0.005
[435/1429] Loss: 0.6289 | LR: 0.005
[436/1429] Loss: 1.1579 | LR: 0.005
[437/1429] Loss: 0.551 | LR: 0.005
[438/1429] Loss: 1.1071 | LR: 0.005
[439/1429] Loss: 0.9522 | LR: 0.005
[440/1429] Loss: 1.1168 | LR: 0.005
[441/1429] Loss: 0.8595 | LR: 0.005
[442/1429] Loss: 1.0229 | LR: 0.005
[443/1429] Loss: 0.433 | LR: 0.005
[444/1429] Loss: 0.3911 | LR: 0.005
[445/1429] Loss: 1.1584 | LR: 0.005
[446/1429] Loss: 0.9351 | LR: 0.005
[447/1429] Loss: 0.8649 | LR: 0.005
[448/1429] Loss: 0.2618 | LR: 0.005
[449/1429] Loss: 0.2283 | LR: 0.005
[450/1429] Loss: 0.7547 | LR: 0.005
[451/1429] Loss: 0.5411 | LR: 0.005
[452/1429] Loss: 0.4094 | LR: 0.005
[453/1429] Loss: 0.9333 | LR: 0.005
[454/1429] Loss: 0.949 | LR: 0.005
[455/1429] Loss: 0.366 | LR: 0.005
[456/1429] Loss: 0.866 | LR: 0.005
[457/1429] Loss: 1.0035 | LR: 0.005
[458/1429] Loss: 0.5799 | LR: 0.005
[459/1429] Loss: 0.567 | LR: 0.005
[460/1429] Loss: 0.2391 | LR: 0.005
[461/1429] Loss: 0.2246 | LR: 0.005
[462/1429] Loss: 0.2481 | LR: 0.005
[463/1429] Loss: 0.8704 | LR: 0.005
[464/1429] Loss: 0.261 | LR: 0.005
[465/1429] Loss: 0.9583 | LR: 0.005
[466/1429] Loss: 0.2733 | LR: 0.005
[467/1429] Loss: 0.2678 | LR: 0.005
[468/1429] Loss: 0.5725 | LR: 0.005
[469/1429] Loss: 0.2162 | LR: 0.005
[470/1429] Loss: 0.4029 | LR: 0.005
[471/1429] Loss: 1.159 | LR: 0.005
[472/1429] Loss: 0.4542 | LR: 0.005
[473/1429] Loss: 0.4385 | LR: 0.005
[474/1429] Loss: 0.6555 | LR: 0.005
[475/1429] Loss: 1.0979 | LR: 0.005
[476/1429] Loss: 0.7464 | LR: 0.005
[477/1429] Loss: 0.4924 | LR: 0.005
[478/1429] Loss: 0.8363 | LR: 0.005
[479/1429] Loss: 0.8219 | LR: 0.005
[480/1429] Loss: 0.958 | LR: 0.005
[481/1429] Loss: 0.3796 | LR: 0.005
[482/1429] Loss: 0.8446 | LR: 0.005
[483/1429] Loss: 1.0253 | LR: 0.005
[484/1429] Loss: 0.566 | LR: 0.005
[485/1429] Loss: 0.9193 | LR: 0.005
[486/1429] Loss: 0.5107 | LR: 0.005
[487/1429] Loss: 0.5845 | LR: 0.005
[488/1429] Loss: 0.2297 | LR: 0.005
[489/1429] Loss: 0.8118 | LR: 0.005
[490/1429] Loss: 0.6927 | LR: 0.005
[491/1429] Loss: 0.7839 | LR: 0.005
[492/1429] Loss: 0.8518 | LR: 0.005
[493/1429] Loss: 0.9697 | LR: 0.005
[494/1429] Loss: 0.9288 | LR: 0.005
[495/1429] Loss: 1.1478 | LR: 0.005
[496/1429] Loss: 0.425 | LR: 0.005
[497/1429] Loss: 0.463 | LR: 0.005
[498/1429] Loss: 0.8373 | LR: 0.005
[499/1429] Loss: 0.718 | LR: 0.005
[500/1429] Loss: 0.9138 | LR: 0.005
[501/1429] Loss: 0.7465 | LR: 0.005
[502/1429] Loss: 0.5123 | LR: 0.005
[503/1429] Loss: 1.1297 | LR: 0.005
[504/1429] Loss: 1.1056 | LR: 0.005
[505/1429] Loss: 0.2967 | LR: 0.005
[506/1429] Loss: 0.7839 | LR: 0.005
[507/1429] Loss: 0.5031 | LR: 0.005
[508/1429] Loss: 1.1293 | LR: 0.005
[509/1429] Loss: 0.3369 | LR: 0.005
[510/1429] Loss: 0.942 | LR: 0.005
[511/1429] Loss: 0.2703 | LR: 0.005
[512/1429] Loss: 0.4573 | LR: 0.005
[513/1429] Loss: 0.2216 | LR: 0.005
[514/1429] Loss: 0.5388 | LR: 0.005
[515/1429] Loss: 0.4846 | LR: 0.005
[516/1429] Loss: 0.5926 | LR: 0.005
[517/1429] Loss: 0.5017 | LR: 0.005
[518/1429] Loss: 0.5578 | LR: 0.005
[519/1429] Loss: 0.5132 | LR: 0.005
[520/1429] Loss: 0.6227 | LR: 0.005
[521/1429] Loss: 1.028 | LR: 0.005
[522/1429] Loss: 0.6658 | LR: 0.005
[523/1429] Loss: 0.8664 | LR: 0.005
[524/1429] Loss: 1.1305 | LR: 0.005
[525/1429] Loss: 0.5926 | LR: 0.005
[526/1429] Loss: 0.8948 | LR: 0.005
[527/1429] Loss: 0.7855 | LR: 0.005
[528/1429] Loss: 0.9236 | LR: 0.005
[529/1429] Loss: 0.5304 | LR: 0.005
[530/1429] Loss: 0.7626 | LR: 0.005
[531/1429] Loss: 1.0852 | LR: 0.005
[532/1429] Loss: 0.2899 | LR: 0.005
[533/1429] Loss: 0.8523 | LR: 0.005
[534/1429] Loss: 0.2506 | LR: 0.005
[535/1429] Loss: 0.6996 | LR: 0.005
[536/1429] Loss: 1.0917 | LR: 0.005
[537/1429] Loss: 0.8468 | LR: 0.005
[538/1429] Loss: 0.4024 | LR: 0.005
[539/1429] Loss: 0.6697 | LR: 0.005
[540/1429] Loss: 0.4368 | LR: 0.005
[541/1429] Loss: 0.9566 | LR: 0.005
[542/1429] Loss: 0.2588 | LR: 0.005
[543/1429] Loss: 0.6902 | LR: 0.005
[544/1429] Loss: 1.1919 | LR: 0.005
[545/1429] Loss: 0.7651 | LR: 0.005
[546/1429] Loss: 1.0388 | LR: 0.005
[547/1429] Loss: 0.4201 | LR: 0.005
[548/1429] Loss: 0.9576 | LR: 0.005
[549/1429] Loss: 0.2873 | LR: 0.005
[550/1429] Loss: 0.818 | LR: 0.005
[551/1429] Loss: 0.6858 | LR: 0.005
[552/1429] Loss: 0.5471 | LR: 0.005
[553/1429] Loss: 0.3309 | LR: 0.005
[554/1429] Loss: 0.3685 | LR: 0.005
[555/1429] Loss: 0.6486 | LR: 0.005
[556/1429] Loss: 0.2676 | LR: 0.005
[557/1429] Loss: 0.9634 | LR: 0.005
[558/1429] Loss: 1.0418 | LR: 0.005
[559/1429] Loss: 0.2424 | LR: 0.005
[560/1429] Loss: 1.1049 | LR: 0.005
[561/1429] Loss: 1.1946 | LR: 0.005
[562/1429] Loss: 0.7874 | LR: 0.005
[563/1429] Loss: 0.5958 | LR: 0.005
[564/1429] Loss: 0.6169 | LR: 0.005
[565/1429] Loss: 0.5154 | LR: 0.005
[566/1429] Loss: 0.2024 | LR: 0.005
[567/1429] Loss: 0.9233 | LR: 0.005
[568/1429] Loss: 0.3077 | LR: 0.005
[569/1429] Loss: 1.1491 | LR: 0.005
[570/1429] Loss: 1.057 | LR: 0.005
[571/1429] Loss: 0.4237 | LR: 0.005
[572/1429] Loss: 0.2926 | LR: 0.005
[573/1429] Loss: 0.3674 | LR: 0.005
[574/1429] Loss: 0.7859 | LR: 0.005
[575/1429] Loss: 0.4579 | LR: 0.005
[576/1429] Loss: 0.7558 | LR: 0.005
[577/1429] Loss: 0.6583 | LR: 0.005
[578/1429] Loss: 0.9321 | LR: 0.005
[579/1429] Loss: 0.7181 | LR: 0.005
[580/1429] Loss: 0.9386 | LR: 0.005
[581/1429] Loss: 0.5173 | LR: 0.005
[582/1429] Loss: 0.7021 | LR: 0.005
[583/1429] Loss: 0.9788 | LR: 0.005
[584/1429] Loss: 0.2477 | LR: 0.005
[585/1429] Loss: 0.8836 | LR: 0.005
[586/1429] Loss: 0.8988 | LR: 0.005
[587/1429] Loss: 0.2712 | LR: 0.005
[588/1429] Loss: 0.7358 | LR: 0.005
[589/1429] Loss: 0.3387 | LR: 0.005
[590/1429] Loss: 0.5113 | LR: 0.005
[591/1429] Loss: 0.7362 | LR: 0.005
[592/1429] Loss: 0.8342 | LR: 0.005
[593/1429] Loss: 0.334 | LR: 0.005
[594/1429] Loss: 0.9505 | LR: 0.005
[595/1429] Loss: 0.9205 | LR: 0.005
[596/1429] Loss: 0.4327 | LR: 0.005
[597/1429] Loss: 1.14 | LR: 0.005
[598/1429] Loss: 0.8867 | LR: 0.005
[599/1429] Loss: 0.5581 | LR: 0.005
[600/1429] Loss: 0.769 | LR: 0.005
[601/1429] Loss: 0.9532 | LR: 0.005
[602/1429] Loss: 0.3557 | LR: 0.005
[603/1429] Loss: 0.6928 | LR: 0.005
[604/1429] Loss: 0.674 | LR: 0.005
[605/1429] Loss: 0.4899 | LR: 0.005
[606/1429] Loss: 0.7919 | LR: 0.005
[607/1429] Loss: 0.6341 | LR: 0.005
[608/1429] Loss: 0.4967 | LR: 0.005
[609/1429] Loss: 0.832 | LR: 0.005
[610/1429] Loss: 1.1636 | LR: 0.005
[611/1429] Loss: 1.094 | LR: 0.005
[612/1429] Loss: 0.6936 | LR: 0.005
[613/1429] Loss: 1.0587 | LR: 0.005
[614/1429] Loss: 0.6938 | LR: 0.005
[615/1429] Loss: 1.1497 | LR: 0.005
[616/1429] Loss: 1.122 | LR: 0.005
[617/1429] Loss: 0.3561 | LR: 0.005
[618/1429] Loss: 0.8936 | LR: 0.005
[619/1429] Loss: 0.6357 | LR: 0.005
[620/1429] Loss: 0.5104 | LR: 0.005
[621/1429] Loss: 0.4262 | LR: 0.005
[622/1429] Loss: 0.5881 | LR: 0.005
[623/1429] Loss: 0.4428 | LR: 0.005
[624/1429] Loss: 0.4567 | LR: 0.005
[625/1429] Loss: 0.7124 | LR: 0.005
[626/1429] Loss: 0.7091 | LR: 0.005
[627/1429] Loss: 0.4512 | LR: 0.005
[628/1429] Loss: 0.95 | LR: 0.005
[629/1429] Loss: 0.2036 | LR: 0.005
[630/1429] Loss: 0.8604 | LR: 0.005
[631/1429] Loss: 0.3698 | LR: 0.005
[632/1429] Loss: 1.0451 | LR: 0.005
[633/1429] Loss: 0.2853 | LR: 0.005
[634/1429] Loss: 0.7674 | LR: 0.005
[635/1429] Loss: 0.9887 | LR: 0.005
[636/1429] Loss: 0.4039 | LR: 0.005
[637/1429] Loss: 0.7266 | LR: 0.005
[638/1429] Loss: 0.2066 | LR: 0.005
[639/1429] Loss: 0.2715 | LR: 0.005
[640/1429] Loss: 0.7777 | LR: 0.005
[641/1429] Loss: 0.4129 | LR: 0.005
[642/1429] Loss: 0.4942 | LR: 0.005
[643/1429] Loss: 1.1807 | LR: 0.005
[644/1429] Loss: 0.4988 | LR: 0.005
[645/1429] Loss: 0.6932 | LR: 0.005
[646/1429] Loss: 0.5403 | LR: 0.005
[647/1429] Loss: 0.5879 | LR: 0.005
[648/1429] Loss: 0.6294 | LR: 0.005
[649/1429] Loss: 0.3158 | LR: 0.005
[650/1429] Loss: 0.553 | LR: 0.005
[651/1429] Loss: 0.6867 | LR: 0.005
[652/1429] Loss: 0.6466 | LR: 0.005
[653/1429] Loss: 1.0883 | LR: 0.005
[654/1429] Loss: 1.1477 | LR: 0.005
[655/1429] Loss: 0.4777 | LR: 0.005
[656/1429] Loss: 0.3802 | LR: 0.005
[657/1429] Loss: 0.5506 | LR: 0.005
[658/1429] Loss: 0.7911 | LR: 0.005
[659/1429] Loss: 0.5439 | LR: 0.005
[660/1429] Loss: 1.1053 | LR: 0.005
[661/1429] Loss: 1.0467 | LR: 0.005
[662/1429] Loss: 0.6351 | LR: 0.005
[663/1429] Loss: 0.3349 | LR: 0.005
[664/1429] Loss: 0.5895 | LR: 0.005
[665/1429] Loss: 1.1462 | LR: 0.005
[666/1429] Loss: 1.187 | LR: 0.005
[667/1429] Loss: 1.066 | LR: 0.005
[668/1429] Loss: 0.482 | LR: 0.005
[669/1429] Loss: 0.3566 | LR: 0.005
[670/1429] Loss: 0.8873 | LR: 0.005
[671/1429] Loss: 0.2822 | LR: 0.005
[672/1429] Loss: 0.4202 | LR: 0.005
[673/1429] Loss: 0.4334 | LR: 0.005
[674/1429] Loss: 0.7459 | LR: 0.005
[675/1429] Loss: 1.1402 | LR: 0.005
[676/1429] Loss: 0.3569 | LR: 0.005
[677/1429] Loss: 0.4676 | LR: 0.005
[678/1429] Loss: 0.38 | LR: 0.005
[679/1429] Loss: 0.3023 | LR: 0.005
[680/1429] Loss: 0.8294 | LR: 0.005
[681/1429] Loss: 0.4465 | LR: 0.005
[682/1429] Loss: 0.4291 | LR: 0.005
[683/1429] Loss: 0.9258 | LR: 0.005
[684/1429] Loss: 0.7684 | LR: 0.005
[685/1429] Loss: 0.8346 | LR: 0.005
[686/1429] Loss: 0.8517 | LR: 0.005
[687/1429] Loss: 0.401 | LR: 0.005
[688/1429] Loss: 0.6956 | LR: 0.005
[689/1429] Loss: 1.0449 | LR: 0.005
[690/1429] Loss: 0.5305 | LR: 0.005
[691/1429] Loss: 0.5939 | LR: 0.005
[692/1429] Loss: 0.216 | LR: 0.005
[693/1429] Loss: 0.8829 | LR: 0.005
[694/1429] Loss: 1.1476 | LR: 0.005
[695/1429] Loss: 1.151 | LR: 0.005
[696/1429] Loss: 0.5356 | LR: 0.005
[697/1429] Loss: 1.1375 | LR: 0.005
[698/1429] Loss: 0.5728 | LR: 0.005
[699/1429] Loss: 1.0212 | LR: 0.005
[700/1429] Loss: 0.5939 | LR: 0.005
[701/1429] Loss: 0.9909 | LR: 0.005
[702/1429] Loss: 1.1147 | LR: 0.005
[703/1429] Loss: 0.4609 | LR: 0.005
[704/1429] Loss: 0.4209 | LR: 0.005
[705/1429] Loss: 0.7546 | LR: 0.005
[706/1429] Loss: 0.6896 | LR: 0.005
[707/1429] Loss: 0.2103 | LR: 0.005
[708/1429] Loss: 0.2813 | LR: 0.005
[709/1429] Loss: 0.6319 | LR: 0.005
[710/1429] Loss: 0.6384 | LR: 0.005
[711/1429] Loss: 0.9178 | LR: 0.005
[712/1429] Loss: 1.0705 | LR: 0.005
[713/1429] Loss: 1.0241 | LR: 0.005
[714/1429] Loss: 0.7662 | LR: 0.005
[715/1429] Loss: 1.0455 | LR: 0.005
[716/1429] Loss: 0.3755 | LR: 0.005
[717/1429] Loss: 0.8654 | LR: 0.005
[718/1429] Loss: 0.7871 | LR: 0.005
[719/1429] Loss: 0.3491 | LR: 0.005
[720/1429] Loss: 0.9872 | LR: 0.005
[721/1429] Loss: 0.4611 | LR: 0.005
[722/1429] Loss: 1.126 | LR: 0.005
[723/1429] Loss: 0.8154 | LR: 0.005
[724/1429] Loss: 0.293 | LR: 0.005
[725/1429] Loss: 0.4938 | LR: 0.005
[726/1429] Loss: 1.0775 | LR: 0.005
[727/1429] Loss: 0.6462 | LR: 0.005
[728/1429] Loss: 0.3801 | LR: 0.005
[729/1429] Loss: 0.3768 | LR: 0.005
[730/1429] Loss: 0.3952 | LR: 0.005
[731/1429] Loss: 0.4303 | LR: 0.005
[732/1429] Loss: 1.0755 | LR: 0.005
[733/1429] Loss: 0.2389 | LR: 0.005
[734/1429] Loss: 1.0296 | LR: 0.005
[735/1429] Loss: 1.0901 | LR: 0.005
[736/1429] Loss: 1.0667 | LR: 0.005
[737/1429] Loss: 1.041 | LR: 0.005
[738/1429] Loss: 0.7956 | LR: 0.005
[739/1429] Loss: 0.5855 | LR: 0.005
[740/1429] Loss: 0.528 | LR: 0.005
[741/1429] Loss: 0.9264 | LR: 0.005
[742/1429] Loss: 0.5836 | LR: 0.005
[743/1429] Loss: 0.4143 | LR: 0.005
[744/1429] Loss: 0.4949 | LR: 0.005
[745/1429] Loss: 1.0464 | LR: 0.005
[746/1429] Loss: 0.3753 | LR: 0.005
[747/1429] Loss: 1.0901 | LR: 0.005
[748/1429] Loss: 0.3074 | LR: 0.005
[749/1429] Loss: 0.203 | LR: 0.005
[750/1429] Loss: 0.2165 | LR: 0.005
[751/1429] Loss: 0.8092 | LR: 0.005
[752/1429] Loss: 1.1146 | LR: 0.005
[753/1429] Loss: 1.0681 | LR: 0.005
[754/1429] Loss: 0.9705 | LR: 0.005
[755/1429] Loss: 0.3678 | LR: 0.005
[756/1429] Loss: 0.8669 | LR: 0.005
[757/1429] Loss: 1.0809 | LR: 0.005
[758/1429] Loss: 1.0168 | LR: 0.005
[759/1429] Loss: 0.9342 | LR: 0.005
[760/1429] Loss: 1.1225 | LR: 0.005
[761/1429] Loss: 0.204 | LR: 0.005
[762/1429] Loss: 0.4764 | LR: 0.005
[763/1429] Loss: 0.2398 | LR: 0.005
[764/1429] Loss: 0.7393 | LR: 0.005
[765/1429] Loss: 1.1264 | LR: 0.005
[766/1429] Loss: 0.2668 | LR: 0.005
[767/1429] Loss: 0.8456 | LR: 0.005
[768/1429] Loss: 0.6357 | LR: 0.005
[769/1429] Loss: 0.9323 | LR: 0.005
[770/1429] Loss: 0.2458 | LR: 0.005
[771/1429] Loss: 0.6017 | LR: 0.005
[772/1429] Loss: 0.3974 | LR: 0.005
[773/1429] Loss: 1.1451 | LR: 0.005
[774/1429] Loss: 0.7635 | LR: 0.005
[775/1429] Loss: 0.8363 | LR: 0.005
[776/1429] Loss: 1.0207 | LR: 0.005
[777/1429] Loss: 0.8213 | LR: 0.005
[778/1429] Loss: 0.9415 | LR: 0.005
[779/1429] Loss: 0.9445 | LR: 0.005
[780/1429] Loss: 0.7082 | LR: 0.005
[781/1429] Loss: 0.8405 | LR: 0.005
[782/1429] Loss: 0.445 | LR: 0.005
[783/1429] Loss: 1.1288 | LR: 0.005
[784/1429] Loss: 1.1539 | LR: 0.005
[785/1429] Loss: 0.9798 | LR: 0.005
[786/1429] Loss: 0.4498 | LR: 0.005
[787/1429] Loss: 1.1697 | LR: 0.005
[788/1429] Loss: 0.6738 | LR: 0.005
[789/1429] Loss: 0.4385 | LR: 0.005
[790/1429] Loss: 0.3394 | LR: 0.005
[791/1429] Loss: 1.0626 | LR: 0.005
[792/1429] Loss: 1.0656 | LR: 0.005
[793/1429] Loss: 0.3825 | LR: 0.005
[794/1429] Loss: 0.8202 | LR: 0.005
[795/1429] Loss: 0.5044 | LR: 0.005
[796/1429] Loss: 0.397 | LR: 0.005
[797/1429] Loss: 1.0984 | LR: 0.005
[798/1429] Loss: 0.9159 | LR: 0.005
[799/1429] Loss: 1.1932 | LR: 0.005
[800/1429] Loss: 0.6771 | LR: 0.005
[801/1429] Loss: 0.8023 | LR: 0.005
[802/1429] Loss: 1.056 | LR: 0.005
[803/1429] Loss: 0.8908 | LR: 0.005
[804/1429] Loss: 0.7392 | LR: 0.005
[805/1429] Loss: 0.8477 | LR: 0.005
[806/1429] Loss: 1.1903 | LR: 0.005
[807/1429] Loss: 0.6974 | LR: 0.005
[808/1429] Loss: 0.5562 | LR: 0.005
[809/1429] Loss: 0.2164 | LR: 0.005
[810/1429] Loss: 0.4918 | LR: 0.005
[811/1429] Loss: 0.8111 | LR: 0.005
[812/1429] Loss: 1.0983 | LR: 0.005
[813/1429] Loss: 0.8737 | LR: 0.005
[814/1429] Loss: 1.0277 | LR: 0.005
[815/1429] Loss: 0.5243 | LR: 0.005
[816/1429] Loss: 0.8459 | LR: 0.005
[817/1429] Loss: 0.3136 | LR: 0.005
[818/1429] Loss: 0.4654 | LR: 0.005
[819/1429] Loss: 0.4558 | LR: 0.005
[820/1429] Loss: 0.2319 | LR: 0.005
[821/1429] Loss: 1.061 | LR: 0.005
[822/1429] Loss: 0.6308 | LR: 0.005
[823/1429] Loss: 0.5223 | LR: 0.005
[824/1429] Loss: 1.0143 | LR: 0.005
[825/1429] Loss: 1.1239 | LR: 0.005
[826/1429] Loss: 0.9568 | LR: 0.005
[827/1429] Loss: 1.1696 | LR: 0.005
[828/1429] Loss: 0.6401 | LR: 0.005
[829/1429] Loss: 0.6894 | LR: 0.005
[830/1429] Loss: 1.0415 | LR: 0.005
[831/1429] Loss: 0.3127 | LR: 0.005
[832/1429] Loss: 1.1699 | LR: 0.005
[833/1429] Loss: 0.7348 | LR: 0.005
[834/1429] Loss: 0.4369 | LR: 0.005
[835/1429] Loss: 0.4329 | LR: 0.005
[836/1429] Loss: 1.0867 | LR: 0.005
[837/1429] Loss: 0.4578 | LR: 0.005
[838/1429] Loss: 0.557 | LR: 0.005
[839/1429] Loss: 0.2306 | LR: 0.005
[840/1429] Loss: 0.4243 | LR: 0.005
[841/1429] Loss: 0.8702 | LR: 0.005
[842/1429] Loss: 0.7142 | LR: 0.005
[843/1429] Loss: 0.6194 | LR: 0.005
[844/1429] Loss: 1.1197 | LR: 0.005
[845/1429] Loss: 0.6839 | LR: 0.005
[846/1429] Loss: 0.7969 | LR: 0.005
[847/1429] Loss: 0.6413 | LR: 0.005
[848/1429] Loss: 0.883 | LR: 0.005
[849/1429] Loss: 1.1437 | LR: 0.005
[850/1429] Loss: 1.1451 | LR: 0.005
[851/1429] Loss: 0.9155 | LR: 0.005
[852/1429] Loss: 0.595 | LR: 0.005
[853/1429] Loss: 0.8026 | LR: 0.005
[854/1429] Loss: 0.7972 | LR: 0.005
[855/1429] Loss: 0.2817 | LR: 0.005
[856/1429] Loss: 0.3797 | LR: 0.005
[857/1429] Loss: 0.413 | LR: 0.005
[858/1429] Loss: 0.2694 | LR: 0.005
[859/1429] Loss: 0.8769 | LR: 0.005
[860/1429] Loss: 0.284 | LR: 0.005
[861/1429] Loss: 0.6374 | LR: 0.005
[862/1429] Loss: 0.7144 | LR: 0.005
[863/1429] Loss: 0.4339 | LR: 0.005
[864/1429] Loss: 0.942 | LR: 0.005
[865/1429] Loss: 0.8729 | LR: 0.005
[866/1429] Loss: 1.0308 | LR: 0.005
[867/1429] Loss: 0.2409 | LR: 0.005
[868/1429] Loss: 0.4598 | LR: 0.005
[869/1429] Loss: 0.2975 | LR: 0.005
[870/1429] Loss: 0.5152 | LR: 0.005
[871/1429] Loss: 0.7686 | LR: 0.005
[872/1429] Loss: 0.6214 | LR: 0.005
[873/1429] Loss: 0.5282 | LR: 0.005
[874/1429] Loss: 1.1281 | LR: 0.005
[875/1429] Loss: 1.0982 | LR: 0.005
[876/1429] Loss: 0.4166 | LR: 0.005
[877/1429] Loss: 0.8682 | LR: 0.005
[878/1429] Loss: 0.41 | LR: 0.005
[879/1429] Loss: 0.2817 | LR: 0.005
[880/1429] Loss: 0.9826 | LR: 0.005
[881/1429] Loss: 0.8369 | LR: 0.005
[882/1429] Loss: 1.1144 | LR: 0.005
[883/1429] Loss: 0.9561 | LR: 0.005
[884/1429] Loss: 0.8239 | LR: 0.005
[885/1429] Loss: 0.8798 | LR: 0.005
[886/1429] Loss: 0.9685 | LR: 0.005
[887/1429] Loss: 0.9195 | LR: 0.005
[888/1429] Loss: 0.8987 | LR: 0.005
[889/1429] Loss: 1.181 | LR: 0.005
[890/1429] Loss: 1.0006 | LR: 0.005
[891/1429] Loss: 0.9039 | LR: 0.005
[892/1429] Loss: 0.2422 | LR: 0.005
[893/1429] Loss: 0.2852 | LR: 0.005
[894/1429] Loss: 1.1665 | LR: 0.005
[895/1429] Loss: 1.046 | LR: 0.005
[896/1429] Loss: 0.4794 | LR: 0.005
[897/1429] Loss: 0.2591 | LR: 0.005
[898/1429] Loss: 0.7862 | LR: 0.005
[899/1429] Loss: 0.5954 | LR: 0.005
[900/1429] Loss: 1.0331 | LR: 0.005
[901/1429] Loss: 0.2536 | LR: 0.005
[902/1429] Loss: 0.4305 | LR: 0.005
[903/1429] Loss: 0.8162 | LR: 0.005
[904/1429] Loss: 1.1343 | LR: 0.005
[905/1429] Loss: 0.6504 | LR: 0.005
[906/1429] Loss: 0.6161 | LR: 0.005
[907/1429] Loss: 0.8052 | LR: 0.005
[908/1429] Loss: 0.7492 | LR: 0.005
[909/1429] Loss: 0.6575 | LR: 0.005
[910/1429] Loss: 0.2418 | LR: 0.005
[911/1429] Loss: 0.2981 | LR: 0.005
[912/1429] Loss: 1.1484 | LR: 0.005
[913/1429] Loss: 0.866 | LR: 0.005
[914/1429] Loss: 0.5258 | LR: 0.005
[915/1429] Loss: 0.3433 | LR: 0.005
[916/1429] Loss: 0.5895 | LR: 0.005
[917/1429] Loss: 0.3608 | LR: 0.005
[918/1429] Loss: 0.8556 | LR: 0.005
[919/1429] Loss: 0.5378 | LR: 0.005
[920/1429] Loss: 0.9984 | LR: 0.005
[921/1429] Loss: 0.4202 | LR: 0.005
[922/1429] Loss: 1.1496 | LR: 0.005
[923/1429] Loss: 0.8002 | LR: 0.005
[924/1429] Loss: 0.2618 | LR: 0.005
[925/1429] Loss: 1.1424 | LR: 0.005
[926/1429] Loss: 0.7384 | LR: 0.005
[927/1429] Loss: 0.832 | LR: 0.005
[928/1429] Loss: 0.2973 | LR: 0.005
[929/1429] Loss: 0.4016 | LR: 0.005
[930/1429] Loss: 1.073 | LR: 0.005
[931/1429] Loss: 0.7067 | LR: 0.005
[932/1429] Loss: 0.4653 | LR: 0.005
[933/1429] Loss: 1.1334 | LR: 0.005
[934/1429] Loss: 0.3865 | LR: 0.005
[935/1429] Loss: 0.521 | LR: 0.005
[936/1429] Loss: 0.984 | LR: 0.005
[937/1429] Loss: 0.919 | LR: 0.005
[938/1429] Loss: 0.5165 | LR: 0.005
[939/1429] Loss: 1.1124 | LR: 0.005
[940/1429] Loss: 0.2525 | LR: 0.005
[941/1429] Loss: 0.7851 | LR: 0.005
[942/1429] Loss: 0.8429 | LR: 0.005
[943/1429] Loss: 0.9432 | LR: 0.005
[944/1429] Loss: 0.824 | LR: 0.005
[945/1429] Loss: 0.4206 | LR: 0.005
[946/1429] Loss: 0.8263 | LR: 0.005
[947/1429] Loss: 1.1015 | LR: 0.005
[948/1429] Loss: 0.6991 | LR: 0.005
[949/1429] Loss: 1.0401 | LR: 0.005
[950/1429] Loss: 0.6319 | LR: 0.005
[951/1429] Loss: 1.1217 | LR: 0.005
[952/1429] Loss: 0.2822 | LR: 0.005
[953/1429] Loss: 0.2423 | LR: 0.005
[954/1429] Loss: 0.9503 | LR: 0.005
[955/1429] Loss: 0.9933 | LR: 0.005
[956/1429] Loss: 0.7629 | LR: 0.005
[957/1429] Loss: 1.1637 | LR: 0.005
[958/1429] Loss: 0.2192 | LR: 0.005
[959/1429] Loss: 0.2694 | LR: 0.005
[960/1429] Loss: 0.9554 | LR: 0.005
[961/1429] Loss: 0.32 | LR: 0.005
[962/1429] Loss: 0.5939 | LR: 0.005
[963/1429] Loss: 0.2561 | LR: 0.005
[964/1429] Loss: 0.8373 | LR: 0.005
[965/1429] Loss: 0.6829 | LR: 0.005
[966/1429] Loss: 1.0062 | LR: 0.005
[967/1429] Loss: 0.2901 | LR: 0.005
[968/1429] Loss: 0.5879 | LR: 0.005
[969/1429] Loss: 0.7796 | LR: 0.005
[970/1429] Loss: 1.135 | LR: 0.005
[971/1429] Loss: 0.2045 | LR: 0.005
[972/1429] Loss: 1.1072 | LR: 0.005
[973/1429] Loss: 0.71 | LR: 0.005
[974/1429] Loss: 0.3921 | LR: 0.005
[975/1429] Loss: 1.1629 | LR: 0.005
[976/1429] Loss: 0.9434 | LR: 0.005
[977/1429] Loss: 0.2312 | LR: 0.005
[978/1429] Loss: 1.138 | LR: 0.005
[979/1429] Loss: 0.5135 | LR: 0.005
[980/1429] Loss: 1.0781 | LR: 0.005
[981/1429] Loss: 0.4607 | LR: 0.005
[982/1429] Loss: 1.1607 | LR: 0.005
[983/1429] Loss: 1.055 | LR: 0.005
[984/1429] Loss: 0.6315 | LR: 0.005
[985/1429] Loss: 0.3546 | LR: 0.005
[986/1429] Loss: 0.7545 | LR: 0.005
[987/1429] Loss: 0.9709 | LR: 0.005
[988/1429] Loss: 0.3771 | LR: 0.005
[989/1429] Loss: 0.603 | LR: 0.005
[990/1429] Loss: 0.4198 | LR: 0.005
[991/1429] Loss: 0.9465 | LR: 0.005
[992/1429] Loss: 0.3095 | LR: 0.005
[993/1429] Loss: 1.1282 | LR: 0.005
[994/1429] Loss: 0.8879 | LR: 0.005
[995/1429] Loss: 1.0064 | LR: 0.005
[996/1429] Loss: 0.8222 | LR: 0.005
[997/1429] Loss: 0.8744 | LR: 0.005
[998/1429] Loss: 0.5726 | LR: 0.005
[999/1429] Loss: 0.5172 | LR: 0.005
[1000/1429] Loss: 1.1593 | LR: 0.005
[1001/1429] Loss: 0.4333 | LR: 0.005
[1002/1429] Loss: 1.0632 | LR: 0.005
[1003/1429] Loss: 0.8003 | LR: 0.005
[1004/1429] Loss: 0.6037 | LR: 0.005
[1005/1429] Loss: 0.5293 | LR: 0.005
[1006/1429] Loss: 0.5718 | LR: 0.005
[1007/1429] Loss: 0.5402 | LR: 0.005
[1008/1429] Loss: 1.0967 | LR: 0.005
[1009/1429] Loss: 0.6562 | LR: 0.005
[1010/1429] Loss: 0.3127 | LR: 0.005
[1011/1429] Loss: 0.5833 | LR: 0.005
[1012/1429] Loss: 1.0074 | LR: 0.005
[1013/1429] Loss: 0.9421 | LR: 0.005
[1014/1429] Loss: 0.4594 | LR: 0.005
[1015/1429] Loss: 0.5099 | LR: 0.005
[1016/1429] Loss: 0.4824 | LR: 0.005
[1017/1429] Loss: 0.9493 | LR: 0.005
[1018/1429] Loss: 0.9934 | LR: 0.005
[1019/1429] Loss: 0.4295 | LR: 0.005
[1020/1429] Loss: 1.0332 | LR: 0.005
[1021/1429] Loss: 0.4334 | LR: 0.005
[1022/1429] Loss: 0.9351 | LR: 0.005
[1023/1429] Loss: 0.3224 | LR: 0.005
[1024/1429] Loss: 1.0231 | LR: 0.005
[1025/1429] Loss: 0.2473 | LR: 0.005
[1026/1429] Loss: 0.4614 | LR: 0.005
[1027/1429] Loss: 0.5991 | LR: 0.005
[1028/1429] Loss: 1.1572 | LR: 0.005
[1029/1429] Loss: 0.6441 | LR: 0.005
[1030/1429] Loss: 0.5552 | LR: 0.005
[1031/1429] Loss: 0.2839 | LR: 0.005
[1032/1429] Loss: 0.8329 | LR: 0.005
[1033/1429] Loss: 1.0644 | LR: 0.005
[1034/1429] Loss: 0.8677 | LR: 0.005
[1035/1429] Loss: 0.2303 | LR: 0.005
[1036/1429] Loss: 0.9814 | LR: 0.005
[1037/1429] Loss: 1.1206 | LR: 0.005
[1038/1429] Loss: 0.3428 | LR: 0.005
[1039/1429] Loss: 0.4284 | LR: 0.005
[1040/1429] Loss: 1.1297 | LR: 0.005
[1041/1429] Loss: 1.1703 | LR: 0.005
[1042/1429] Loss: 0.7686 | LR: 0.005
[1043/1429] Loss: 0.2457 | LR: 0.005
[1044/1429] Loss: 0.7345 | LR: 0.005
[1045/1429] Loss: 0.6393 | LR: 0.005
[1046/1429] Loss: 0.5384 | LR: 0.005
[1047/1429] Loss: 0.3151 | LR: 0.005
[1048/1429] Loss: 1.1734 | LR: 0.005
[1049/1429] Loss: 0.7194 | LR: 0.005
[1050/1429] Loss: 0.9879 | LR: 0.005
[1051/1429] Loss: 0.386 | LR: 0.005
[1052/1429] Loss: 0.6008 | LR: 0.005
[1053/1429] Loss: 0.5911 | LR: 0.005
[1054/1429] Loss: 1.0871 | LR: 0.005
[1055/1429] Loss: 0.3811 | LR: 0.005
[1056/1429] Loss: 0.7863 | LR: 0.005
[1057/1429] Loss: 0.7688 | LR: 0.005
[1058/1429] Loss: 1.0217 | LR: 0.005
[1059/1429] Loss: 0.4018 | LR: 0.005
[1060/1429] Loss: 0.8007 | LR: 0.005
[1061/1429] Loss: 1.1668 | LR: 0.005
[1062/1429] Loss: 0.2602 | LR: 0.005
[1063/1429] Loss: 0.8575 | LR: 0.005
[1064/1429] Loss: 0.5118 | LR: 0.005
[1065/1429] Loss: 0.9204 | LR: 0.005
[1066/1429] Loss: 0.4004 | LR: 0.005
[1067/1429] Loss: 0.266 | LR: 0.005
[1068/1429] Loss: 0.7833 | LR: 0.005
[1069/1429] Loss: 0.4507 | LR: 0.005
[1070/1429] Loss: 0.6482 | LR: 0.005
[1071/1429] Loss: 0.9664 | LR: 0.005
[1072/1429] Loss: 0.774 | LR: 0.005
[1073/1429] Loss: 0.7848 | LR: 0.005
[1074/1429] Loss: 0.5536 | LR: 0.005
[1075/1429] Loss: 0.563 | LR: 0.005
[1076/1429] Loss: 0.4662 | LR: 0.005
[1077/1429] Loss: 0.8397 | LR: 0.005
[1078/1429] Loss: 0.3938 | LR: 0.005
[1079/1429] Loss: 0.991 | LR: 0.005
[1080/1429] Loss: 0.7452 | LR: 0.005
[1081/1429] Loss: 0.6425 | LR: 0.005
[1082/1429] Loss: 1.0366 | LR: 0.005
[1083/1429] Loss: 0.2746 | LR: 0.005
[1084/1429] Loss: 0.6387 | LR: 0.005
[1085/1429] Loss: 0.8978 | LR: 0.005
[1086/1429] Loss: 0.4506 | LR: 0.005
[1087/1429] Loss: 0.3142 | LR: 0.005
[1088/1429] Loss: 1.0109 | LR: 0.005
[1089/1429] Loss: 0.4041 | LR: 0.005
[1090/1429] Loss: 0.9871 | LR: 0.005
[1091/1429] Loss: 0.302 | LR: 0.005
[1092/1429] Loss: 0.6535 | LR: 0.005
[1093/1429] Loss: 0.4785 | LR: 0.005
[1094/1429] Loss: 0.9364 | LR: 0.005
[1095/1429] Loss: 0.2639 | LR: 0.005
[1096/1429] Loss: 0.6676 | LR: 0.005
[1097/1429] Loss: 0.8224 | LR: 0.005
[1098/1429] Loss: 1.1623 | LR: 0.005
[1099/1429] Loss: 0.8081 | LR: 0.005
[1100/1429] Loss: 1.1596 | LR: 0.005
[1101/1429] Loss: 0.3284 | LR: 0.005
[1102/1429] Loss: 0.3476 | LR: 0.005
[1103/1429] Loss: 0.5226 | LR: 0.005
[1104/1429] Loss: 1.1324 | LR: 0.005
[1105/1429] Loss: 0.7814 | LR: 0.005
[1106/1429] Loss: 0.8309 | LR: 0.005
[1107/1429] Loss: 0.4451 | LR: 0.005
[1108/1429] Loss: 0.409 | LR: 0.005
[1109/1429] Loss: 1.0156 | LR: 0.005
[1110/1429] Loss: 1.1162 | LR: 0.005
[1111/1429] Loss: 0.2989 | LR: 0.005
[1112/1429] Loss: 0.9486 | LR: 0.005
[1113/1429] Loss: 0.2064 | LR: 0.005
[1114/1429] Loss: 0.7537 | LR: 0.005
[1115/1429] Loss: 0.8394 | LR: 0.005
[1116/1429] Loss: 0.7553 | LR: 0.005
[1117/1429] Loss: 0.5866 | LR: 0.005
[1118/1429] Loss: 1.1306 | LR: 0.005
[1119/1429] Loss: 0.661 | LR: 0.005
[1120/1429] Loss: 0.8801 | LR: 0.005
[1121/1429] Loss: 0.5604 | LR: 0.005
[1122/1429] Loss: 0.2753 | LR: 0.005
[1123/1429] Loss: 0.3493 | LR: 0.005
[1124/1429] Loss: 0.5587 | LR: 0.005
[1125/1429] Loss: 0.2367 | LR: 0.005
[1126/1429] Loss: 0.4541 | LR: 0.005
[1127/1429] Loss: 0.2922 | LR: 0.005
[1128/1429] Loss: 0.7462 | LR: 0.005
[1129/1429] Loss: 0.499 | LR: 0.005
[1130/1429] Loss: 1.1656 | LR: 0.005
[1131/1429] Loss: 0.5089 | LR: 0.005
[1132/1429] Loss: 0.5698 | LR: 0.005
[1133/1429] Loss: 0.5719 | LR: 0.005
[1134/1429] Loss: 0.7962 | LR: 0.005
[1135/1429] Loss: 0.7119 | LR: 0.005
[1136/1429] Loss: 1.0457 | LR: 0.005
[1137/1429] Loss: 0.566 | LR: 0.005
[1138/1429] Loss: 1.1193 | LR: 0.005
[1139/1429] Loss: 0.3278 | LR: 0.005
[1140/1429] Loss: 1.1951 | LR: 0.005
[1141/1429] Loss: 0.6606 | LR: 0.005
[1142/1429] Loss: 0.7663 | LR: 0.005
[1143/1429] Loss: 0.2061 | LR: 0.005
[1144/1429] Loss: 0.7229 | LR: 0.005
[1145/1429] Loss: 0.2966 | LR: 0.005
[1146/1429] Loss: 1.0021 | LR: 0.005
[1147/1429] Loss: 1.0213 | LR: 0.005
[1148/1429] Loss: 1.028 | LR: 0.005
[1149/1429] Loss: 0.4298 | LR: 0.005
[1150/1429] Loss: 0.5581 | LR: 0.005
[1151/1429] Loss: 0.5238 | LR: 0.005
[1152/1429] Loss: 0.7856 | LR: 0.005
[1153/1429] Loss: 0.9312 | LR: 0.005
[1154/1429] Loss: 0.6371 | LR: 0.005
[1155/1429] Loss: 0.4591 | LR: 0.005
[1156/1429] Loss: 1.1199 | LR: 0.005
[1157/1429] Loss: 1.173 | LR: 0.005
[1158/1429] Loss: 1.1572 | LR: 0.005
[1159/1429] Loss: 1.1835 | LR: 0.005
[1160/1429] Loss: 0.6338 | LR: 0.005
[1161/1429] Loss: 0.7514 | LR: 0.005
[1162/1429] Loss: 0.9665 | LR: 0.005
[1163/1429] Loss: 0.2334 | LR: 0.005
[1164/1429] Loss: 0.3859 | LR: 0.005
[1165/1429] Loss: 0.3443 | LR: 0.005
[1166/1429] Loss: 0.7277 | LR: 0.005
[1167/1429] Loss: 0.8299 | LR: 0.005
[1168/1429] Loss: 0.5671 | LR: 0.005
[1169/1429] Loss: 0.7533 | LR: 0.005
[1170/1429] Loss: 0.505 | LR: 0.005
[1171/1429] Loss: 0.3265 | LR: 0.005
[1172/1429] Loss: 0.6104 | LR: 0.005
[1173/1429] Loss: 0.8862 | LR: 0.005
[1174/1429] Loss: 0.5269 | LR: 0.005
[1175/1429] Loss: 1.0454 | LR: 0.005
[1176/1429] Loss: 0.8433 | LR: 0.005
[1177/1429] Loss: 0.6167 | LR: 0.005
[1178/1429] Loss: 0.6322 | LR: 0.005
[1179/1429] Loss: 0.6097 | LR: 0.005
[1180/1429] Loss: 1.1824 | LR: 0.005
[1181/1429] Loss: 1.0924 | LR: 0.005
[1182/1429] Loss: 1.0738 | LR: 0.005
[1183/1429] Loss: 0.583 | LR: 0.005
[1184/1429] Loss: 0.3524 | LR: 0.005
[1185/1429] Loss: 0.6237 | LR: 0.005
[1186/1429] Loss: 1.1482 | LR: 0.005
[1187/1429] Loss: 1.1201 | LR: 0.005
[1188/1429] Loss: 1.1251 | LR: 0.005
[1189/1429] Loss: 0.4523 | LR: 0.005
[1190/1429] Loss: 0.523 | LR: 0.005
[1191/1429] Loss: 0.9479 | LR: 0.005
[1192/1429] Loss: 0.3938 | LR: 0.005
[1193/1429] Loss: 0.9287 | LR: 0.005
[1194/1429] Loss: 0.7468 | LR: 0.005
[1195/1429] Loss: 0.4425 | LR: 0.005
[1196/1429] Loss: 0.3336 | LR: 0.005
[1197/1429] Loss: 0.2891 | LR: 0.005
[1198/1429] Loss: 0.7124 | LR: 0.005
[1199/1429] Loss: 0.6153 | LR: 0.005
[1200/1429] Loss: 0.2986 | LR: 0.005
[1201/1429] Loss: 0.2755 | LR: 0.005
[1202/1429] Loss: 0.6258 | LR: 0.005
[1203/1429] Loss: 0.222 | LR: 0.005
[1204/1429] Loss: 0.2804 | LR: 0.005
[1205/1429] Loss: 0.2375 | LR: 0.005
[1206/1429] Loss: 0.688 | LR: 0.005
[1207/1429] Loss: 0.6911 | LR: 0.005
[1208/1429] Loss: 0.7309 | LR: 0.005
[1209/1429] Loss: 0.8283 | LR: 0.005
[1210/1429] Loss: 0.6497 | LR: 0.005
[1211/1429] Loss: 0.7864 | LR: 0.005
[1212/1429] Loss: 0.9766 | LR: 0.005
[1213/1429] Loss: 0.5065 | LR: 0.005
[1214/1429] Loss: 0.4764 | LR: 0.005
[1215/1429] Loss: 1.0915 | LR: 0.005
[1216/1429] Loss: 0.4107 | LR: 0.005
[1217/1429] Loss: 0.4632 | LR: 0.005
[1218/1429] Loss: 1.1924 | LR: 0.005
[1219/1429] Loss: 0.4259 | LR: 0.005
[1220/1429] Loss: 0.9615 | LR: 0.005
[1221/1429] Loss: 0.9121 | LR: 0.005
[1222/1429] Loss: 1.1944 | LR: 0.005
[1223/1429] Loss: 1.0202 | LR: 0.005
[1224/1429] Loss: 0.4287 | LR: 0.005
[1225/1429] Loss: 0.7206 | LR: 0.005
[1226/1429] Loss: 1.0911 | LR: 0.005
[1227/1429] Loss: 1.0302 | LR: 0.005
[1228/1429] Loss: 0.9323 | LR: 0.005
[1229/1429] Loss: 0.7477 | LR: 0.005
[1230/1429] Loss: 1.0283 | LR: 0.005
[1231/1429] Loss: 0.904 | LR: 0.005
[1232/1429] Loss: 1.0742 | LR: 0.005
[1233/1429] Loss: 0.2757 | LR: 0.005
[1234/1429] Loss: 1.0598 | LR: 0.005
[1235/1429] Loss: 0.995 | LR: 0.005
[1236/1429] Loss: 0.7502 | LR: 0.005
[1237/1429] Loss: 0.681 | LR: 0.005
[1238/1429] Loss: 0.76 | LR: 0.005
[1239/1429] Loss: 1.0556 | LR: 0.005
[1240/1429] Loss: 0.9553 | LR: 0.005
[1241/1429] Loss: 0.6408 | LR: 0.005
[1242/1429] Loss: 0.6239 | LR: 0.005
[1243/1429] Loss: 0.3565 | LR: 0.005
[1244/1429] Loss: 0.7063 | LR: 0.005
[1245/1429] Loss: 0.4926 | LR: 0.005
[1246/1429] Loss: 0.7641 | LR: 0.005
[1247/1429] Loss: 1.1194 | LR: 0.005
[1248/1429] Loss: 0.3574 | LR: 0.005
[1249/1429] Loss: 1.1297 | LR: 0.005
[1250/1429] Loss: 0.9303 | LR: 0.005
[1251/1429] Loss: 1.033 | LR: 0.005
[1252/1429] Loss: 0.8853 | LR: 0.005
[1253/1429] Loss: 0.9999 | LR: 0.005
[1254/1429] Loss: 1.059 | LR: 0.005
[1255/1429] Loss: 0.2001 | LR: 0.005
[1256/1429] Loss: 0.4001 | LR: 0.005
[1257/1429] Loss: 0.6437 | LR: 0.005
[1258/1429] Loss: 0.8887 | LR: 0.005
[1259/1429] Loss: 0.7634 | LR: 0.005
[1260/1429] Loss: 0.4833 | LR: 0.005
[1261/1429] Loss: 0.7601 | LR: 0.005
[1262/1429] Loss: 0.8014 | LR: 0.005
[1263/1429] Loss: 0.8909 | LR: 0.005
[1264/1429] Loss: 0.4273 | LR: 0.005
[1265/1429] Loss: 0.5041 | LR: 0.005
[1266/1429] Loss: 0.744 | LR: 0.005
[1267/1429] Loss: 0.5653 | LR: 0.005
[1268/1429] Loss: 0.2271 | LR: 0.005
[1269/1429] Loss: 0.8139 | LR: 0.005
[1270/1429] Loss: 0.8735 | LR: 0.005
[1271/1429] Loss: 0.3744 | LR: 0.005
[1272/1429] Loss: 0.9763 | LR: 0.005
[1273/1429] Loss: 0.2757 | LR: 0.005
[1274/1429] Loss: 0.4031 | LR: 0.005
[1275/1429] Loss: 0.2417 | LR: 0.005
[1276/1429] Loss: 0.3565 | LR: 0.005
[1277/1429] Loss: 0.2134 | LR: 0.005
[1278/1429] Loss: 1.1704 | LR: 0.005
[1279/1429] Loss: 0.4191 | LR: 0.005
[1280/1429] Loss: 1.0561 | LR: 0.005
[1281/1429] Loss: 0.2343 | LR: 0.005
[1282/1429] Loss: 0.3746 | LR: 0.005
[1283/1429] Loss: 0.7472 | LR: 0.005
[1284/1429] Loss: 1.0373 | LR: 0.005
[1285/1429] Loss: 0.2444 | LR: 0.005
[1286/1429] Loss: 0.9467 | LR: 0.005
[1287/1429] Loss: 0.414 | LR: 0.005
[1288/1429] Loss: 0.3293 | LR: 0.005
[1289/1429] Loss: 0.4105 | LR: 0.005
[1290/1429] Loss: 0.6905 | LR: 0.005
[1291/1429] Loss: 0.9201 | LR: 0.005
[1292/1429] Loss: 1.1149 | LR: 0.005
[1293/1429] Loss: 0.9611 | LR: 0.005
[1294/1429] Loss: 0.9269 | LR: 0.005
[1295/1429] Loss: 0.9244 | LR: 0.005
[1296/1429] Loss: 1.0244 | LR: 0.005
[1297/1429] Loss: 1.1133 | LR: 0.005
[1298/1429] Loss: 0.6396 | LR: 0.005
[1299/1429] Loss: 1.1093 | LR: 0.005
[1300/1429] Loss: 0.4065 | LR: 0.005
[1301/1429] Loss: 0.8166 | LR: 0.005
[1302/1429] Loss: 0.249 | LR: 0.005
[1303/1429] Loss: 0.9129 | LR: 0.005
[1304/1429] Loss: 0.9734 | LR: 0.005
[1305/1429] Loss: 0.4593 | LR: 0.005
[1306/1429] Loss: 0.4978 | LR: 0.005
[1307/1429] Loss: 0.4678 | LR: 0.005
[1308/1429] Loss: 0.7247 | LR: 0.005
[1309/1429] Loss: 0.7824 | LR: 0.005
[1310/1429] Loss: 1.0409 | LR: 0.005
[1311/1429] Loss: 0.5215 | LR: 0.005
[1312/1429] Loss: 1.196 | LR: 0.005
[1313/1429] Loss: 1.0948 | LR: 0.005
[1314/1429] Loss: 0.5483 | LR: 0.005
[1315/1429] Loss: 0.6377 | LR: 0.005
[1316/1429] Loss: 0.6153 | LR: 0.005
[1317/1429] Loss: 0.8206 | LR: 0.005
[1318/1429] Loss: 0.8051 | LR: 0.005
[1319/1429] Loss: 0.8332 | LR: 0.005
[1320/1429] Loss: 0.2047 | LR: 0.005
[1321/1429] Loss: 0.7156 | LR: 0.005
[1322/1429] Loss: 0.9119 | LR: 0.005
[1323/1429] Loss: 0.6917 | LR: 0.005
[1324/1429] Loss: 0.8214 | LR: 0.005
[1325/1429] Loss: 0.9775 | LR: 0.005
[1326/1429] Loss: 0.4946 | LR: 0.005
[1327/1429] Loss: 0.4828 | LR: 0.005
[1328/1429] Loss: 0.5281 | LR: 0.005
[1329/1429] Loss: 0.4307 | LR: 0.005
[1330/1429] Loss: 1.0296 | LR: 0.005
[1331/1429] Loss: 0.2956 | LR: 0.005
[1332/1429] Loss: 0.6354 | LR: 0.005
[1333/1429] Loss: 0.5048 | LR: 0.005
[1334/1429] Loss: 0.4659 | LR: 0.005
[1335/1429] Loss: 0.9548 | LR: 0.005
[1336/1429] Loss: 0.7876 | LR: 0.005
[1337/1429] Loss: 0.9296 | LR: 0.005
[1338/1429] Loss: 1.1779 | LR: 0.005
[1339/1429] Loss: 0.2015 | LR: 0.005
[1340/1429] Loss: 0.9252 | LR: 0.005
[1341/1429] Loss: 1.0268 | LR: 0.005
[1342/1429] Loss: 1.1418 | LR: 0.005
[1343/1429] Loss: 0.9931 | LR: 0.005
[1344/1429] Loss: 0.9423 | LR: 0.005
[1345/1429] Loss: 1.0625 | LR: 0.005
[1346/1429] Loss: 0.29 | LR: 0.005
[1347/1429] Loss: 0.9494 | LR: 0.005
[1348/1429] Loss: 0.5643 | LR: 0.005
[1349/1429] Loss: 0.8603 | LR: 0.005
[1350/1429] Loss: 0.3431 | LR: 0.005
[1351/1429] Loss: 0.7838 | LR: 0.005
[1352/1429] Loss: 0.3674 | LR: 0.005
[1353/1429] Loss: 1.0576 | LR: 0.005
[1354/1429] Loss: 0.7659 | LR: 0.005
[1355/1429] Loss: 0.2669 | LR: 0.005
[1356/1429] Loss: 1.1594 | LR: 0.005
[1357/1429] Loss: 1.0624 | LR: 0.005
[1358/1429] Loss: 0.3971 | LR: 0.005
[1359/1429] Loss: 0.8913 | LR: 0.005
[1360/1429] Loss: 1.1954 | LR: 0.005
[1361/1429] Loss: 0.4484 | LR: 0.005
[1362/1429] Loss: 0.6874 | LR: 0.005
[1363/1429] Loss: 0.6682 | LR: 0.005
[1364/1429] Loss: 1.0518 | LR: 0.005
[1365/1429] Loss: 0.3537 | LR: 0.005
[1366/1429] Loss: 0.9304 | LR: 0.005
[1367/1429] Loss: 1.061 | LR: 0.005
[1368/1429] Loss: 0.9573 | LR: 0.005
[1369/1429] Loss: 0.3069 | LR: 0.005
[1370/1429] Loss: 0.4256 | LR: 0.005
[1371/1429] Loss: 0.4098 | LR: 0.005
[1372/1429] Loss: 0.4008 | LR: 0.005
[1373/1429] Loss: 0.6489 | LR: 0.005
[1374/1429] Loss: 0.5957 | LR: 0.005
[1375/1429] Loss: 0.7669 | LR: 0.005
[1376/1429] Loss: 0.5137 | LR: 0.005
[1377/1429] Loss: 0.5521 | LR: 0.005
[1378/1429] Loss: 0.5138 | LR: 0.005
[1379/1429] Loss: 0.607 | LR: 0.005
[1380/1429] Loss: 0.4158 | LR: 0.005
[1381/1429] Loss: 0.6928 | LR: 0.005
[1382/1429] Loss: 1.0858 | LR: 0.005
[1383/1429] Loss: 0.4881 | LR: 0.005
[1384/1429] Loss: 0.6691 | LR: 0.005
[1385/1429] Loss: 0.973 | LR: 0.005
[1386/1429] Loss: 0.4976 | LR: 0.005
[1387/1429] Loss: 0.9005 | LR: 0.005
[1388/1429] Loss: 0.3045 | LR: 0.005
[1389/1429] Loss: 0.2922 | LR: 0.005
[1390/1429] Loss: 0.2925 | LR: 0.005
[1391/1429] Loss: 0.6957 | LR: 0.005
[1392/1429] Loss: 0.7318 | LR: 0.005
[1393/1429] Loss: 1.1914 | LR: 0.005
[1394/1429] Loss: 1.1884 | LR: 0.005
[1395/1429] Loss: 0.532 | LR: 0.005
[1396/1429] Loss: 0.3069 | LR: 0.005
[1397/1429] Loss: 1.0524 | LR: 0.005
[1398/1429] Loss: 0.2286 | LR: 0.005
[1399/1429] Loss: 1.0819 | LR: 0.005
[1400/1429] Loss: 0.9796 | LR: 0.005
[1401/1429] Loss: 0.269 | LR: 0.005
[1402/1429] Loss: 1.072 | LR: 0.005
[1403/1429] Loss: 1.1596 | LR: 0.005
[1404/1429] Loss: 0.2637 | LR: 0.005
[1405/1429] Loss: 0.3373 | LR: 0.005
[1406/1429] Loss: 0.7591 | LR: 0.005
[1407/1429] Loss: 0.5887 | LR: 0.005
[1408/1429] Loss: 0.7891 | LR: 0.005
[1409/1429] Loss: 0.294 | LR: 0.005
[1410/1429] Loss: 0.5571 | LR: 0.005
[1411/1429] Loss: 1.1337 | LR: 0.005
[1412/1429] Loss: 0.5993 | LR: 0.005
[1413/1429] Loss: 0.4628 | LR: 0.005
[1414/1429] Loss: 0.4302 | LR: 0.005
[1415/1429] Loss: 1.0165 | LR: 0.005
[1416/1429] Loss: 0.7516 | LR: 0.005
[1417/1429] Loss: 0.9964 | LR: 0.005
[1418/1429] Loss: 0.2989 | LR: 0.005
[1419/1429] Loss: 1.1604 | LR: 0.005
[1420/1429] Loss: 0.8009 | LR: 0.005
[1421/1429] Loss: 1.0087 | LR: 0.005
[1422/1429] Loss: 0.8114 | LR: 0.005
[1423/1429] Loss: 0.2822 | LR: 0.005
[1424/1429] Loss: 0.6364 | LR: 0.005
[1425/1429] Loss: 0.285 | LR: 0.005
[1426/1429] Loss: 0.5487 | LR: 0.005
[1427/1429] Loss: 0.784 | LR: 0.005
[1428/1429] Loss: 1.0247 | LR: 0.005
[1429/1429] Loss: 0.7136 | LR: 0.005
[1429/1429] Epoch Time: 11m 31s

Epoch 2/3
------------------------------------------------------------
[1/1429] Loss: 0.2792 | LR: 0.005
[2/1429] Loss: 0.1691 | LR: 0.005
[3/1429] Loss: 0.2244 | LR: 0.005
[4/1429] Loss: 0.6369 | LR: 0.005
[5/1429] Loss: 0.228 | LR: 0.005
[6/1429] Loss: 0.2273 | LR: 0.005
[7/1429] Loss: 0.7756 | LR: 0.005
[8/1429] Loss: 1.082 | LR: 0.005
[9/1429] Loss: 0.2335 | LR: 0.005
[10/1429] Loss: 0.9555 | LR: 0.005
[11/1429] Loss: 0.6467 | LR: 0.005
[12/1429] Loss: 0.6189 | LR: 0.005
[13/1429] Loss: 0.3427 | LR: 0.005
[14/1429] Loss: 0.9372 | LR: 0.005
[15/1429] Loss: 0.5341 | LR: 0.005
[16/1429] Loss: 0.3841 | LR: 0.005
[17/1429] Loss: 0.4641 | LR: 0.005
[18/1429] Loss: 0.3394 | LR: 0.005
[19/1429] Loss: 0.1721 | LR: 0.005
[20/1429] Loss: 0.2361 | LR: 0.005
[21/1429] Loss: 0.5785 | LR: 0.005
[22/1429] Loss: 0.2186 | LR: 0.005
[23/1429] Loss: 0.2463 | LR: 0.005
[24/1429] Loss: 0.1011 | LR: 0.005
[25/1429] Loss: 0.501 | LR: 0.005
[26/1429] Loss: 0.4294 | LR: 0.005
[27/1429] Loss: 0.1979 | LR: 0.005
[28/1429] Loss: 0.7731 | LR: 0.005
[29/1429] Loss: 0.6826 | LR: 0.005
[30/1429] Loss: 0.1502 | LR: 0.005
[31/1429] Loss: 0.9298 | LR: 0.005
[32/1429] Loss: 1.0842 | LR: 0.005
[33/1429] Loss: 0.1577 | LR: 0.005
[34/1429] Loss: 0.6921 | LR: 0.005
[35/1429] Loss: 0.1566 | LR: 0.005
[36/1429] Loss: 1.031 | LR: 0.005
[37/1429] Loss: 0.2581 | LR: 0.005
[38/1429] Loss: 0.5605 | LR: 0.005
[39/1429] Loss: 0.4123 | LR: 0.005
[40/1429] Loss: 0.8572 | LR: 0.005
[41/1429] Loss: 0.7291 | LR: 0.005
[42/1429] Loss: 0.6148 | LR: 0.005
[43/1429] Loss: 0.8736 | LR: 0.005
[44/1429] Loss: 0.6373 | LR: 0.005
[45/1429] Loss: 1.056 | LR: 0.005
[46/1429] Loss: 0.374 | LR: 0.005
[47/1429] Loss: 0.7056 | LR: 0.005
[48/1429] Loss: 0.2078 | LR: 0.005
[49/1429] Loss: 0.9315 | LR: 0.005
[50/1429] Loss: 0.1921 | LR: 0.005
[51/1429] Loss: 0.7758 | LR: 0.005
[52/1429] Loss: 0.3452 | LR: 0.005
[53/1429] Loss: 0.6023 | LR: 0.005
[54/1429] Loss: 0.4561 | LR: 0.005
[55/1429] Loss: 0.7949 | LR: 0.005
[56/1429] Loss: 1.0703 | LR: 0.005
[57/1429] Loss: 0.3367 | LR: 0.005
[58/1429] Loss: 0.1555 | LR: 0.005
[59/1429] Loss: 0.9781 | LR: 0.005
[60/1429] Loss: 0.4505 | LR: 0.005
[61/1429] Loss: 0.645 | LR: 0.005
[62/1429] Loss: 0.8579 | LR: 0.005
[63/1429] Loss: 0.4469 | LR: 0.005
[64/1429] Loss: 0.1962 | LR: 0.005
[65/1429] Loss: 0.9344 | LR: 0.005
[66/1429] Loss: 0.8705 | LR: 0.005
[67/1429] Loss: 0.991 | LR: 0.005
[68/1429] Loss: 0.3803 | LR: 0.005
[69/1429] Loss: 0.2895 | LR: 0.005
[70/1429] Loss: 0.1326 | LR: 0.005
[71/1429] Loss: 1.0624 | LR: 0.005
[72/1429] Loss: 0.6846 | LR: 0.005
[73/1429] Loss: 1.0338 | LR: 0.005
[74/1429] Loss: 0.7998 | LR: 0.005
[75/1429] Loss: 0.533 | LR: 0.005
[76/1429] Loss: 1.0165 | LR: 0.005
[77/1429] Loss: 1.0529 | LR: 0.005
[78/1429] Loss: 0.5341 | LR: 0.005
[79/1429] Loss: 0.2286 | LR: 0.005
[80/1429] Loss: 1.0465 | LR: 0.005
[81/1429] Loss: 0.2036 | LR: 0.005
[82/1429] Loss: 0.8372 | LR: 0.005
[83/1429] Loss: 0.3395 | LR: 0.005
[84/1429] Loss: 0.6776 | LR: 0.005
[85/1429] Loss: 0.8812 | LR: 0.005
[86/1429] Loss: 0.5398 | LR: 0.005
[87/1429] Loss: 0.2092 | LR: 0.005
[88/1429] Loss: 0.2394 | LR: 0.005
[89/1429] Loss: 0.9656 | LR: 0.005
[90/1429] Loss: 0.6321 | LR: 0.005
[91/1429] Loss: 1.0873 | LR: 0.005
[92/1429] Loss: 0.695 | LR: 0.005
[93/1429] Loss: 0.2758 | LR: 0.005
[94/1429] Loss: 0.5868 | LR: 0.005
[95/1429] Loss: 0.4144 | LR: 0.005
[96/1429] Loss: 0.7928 | LR: 0.005
[97/1429] Loss: 0.2222 | LR: 0.005
[98/1429] Loss: 0.7611 | LR: 0.005
[99/1429] Loss: 0.8621 | LR: 0.005
[100/1429] Loss: 0.8478 | LR: 0.005
[101/1429] Loss: 0.2702 | LR: 0.005
[102/1429] Loss: 0.647 | LR: 0.005
[103/1429] Loss: 0.4438 | LR: 0.005
[104/1429] Loss: 0.324 | LR: 0.005
[105/1429] Loss: 0.1891 | LR: 0.005
[106/1429] Loss: 0.6894 | LR: 0.005
[107/1429] Loss: 0.1885 | LR: 0.005
[108/1429] Loss: 0.964 | LR: 0.005
[109/1429] Loss: 0.5321 | LR: 0.005
[110/1429] Loss: 0.9674 | LR: 0.005
[111/1429] Loss: 0.9183 | LR: 0.005
[112/1429] Loss: 0.7196 | LR: 0.005
[113/1429] Loss: 0.3183 | LR: 0.005
[114/1429] Loss: 0.4647 | LR: 0.005
[115/1429] Loss: 0.7223 | LR: 0.005
[116/1429] Loss: 0.3963 | LR: 0.005
[117/1429] Loss: 0.1529 | LR: 0.005
[118/1429] Loss: 0.3686 | LR: 0.005
[119/1429] Loss: 0.5979 | LR: 0.005
[120/1429] Loss: 0.9959 | LR: 0.005
[121/1429] Loss: 0.6747 | LR: 0.005
[122/1429] Loss: 1.003 | LR: 0.005
[123/1429] Loss: 0.363 | LR: 0.005
[124/1429] Loss: 0.1703 | LR: 0.005
[125/1429] Loss: 0.8359 | LR: 0.005
[126/1429] Loss: 0.9379 | LR: 0.005
[127/1429] Loss: 0.6083 | LR: 0.005
[128/1429] Loss: 0.6118 | LR: 0.005
[129/1429] Loss: 0.8475 | LR: 0.005
[130/1429] Loss: 0.3905 | LR: 0.005
[131/1429] Loss: 1.001 | LR: 0.005
[132/1429] Loss: 0.8651 | LR: 0.005
[133/1429] Loss: 0.197 | LR: 0.005
[134/1429] Loss: 1.03 | LR: 0.005
[135/1429] Loss: 0.8563 | LR: 0.005
[136/1429] Loss: 0.153 | LR: 0.005
[137/1429] Loss: 1.0399 | LR: 0.005
[138/1429] Loss: 0.4305 | LR: 0.005
[139/1429] Loss: 0.4527 | LR: 0.005
[140/1429] Loss: 0.6477 | LR: 0.005
[141/1429] Loss: 0.7965 | LR: 0.005
[142/1429] Loss: 0.7688 | LR: 0.005
[143/1429] Loss: 0.3302 | LR: 0.005
[144/1429] Loss: 0.7574 | LR: 0.005
[145/1429] Loss: 0.9846 | LR: 0.005
[146/1429] Loss: 0.6323 | LR: 0.005
[147/1429] Loss: 0.586 | LR: 0.005
[148/1429] Loss: 0.2901 | LR: 0.005
[149/1429] Loss: 0.7755 | LR: 0.005
[150/1429] Loss: 1.0622 | LR: 0.005
[151/1429] Loss: 0.2089 | LR: 0.005
[152/1429] Loss: 0.8918 | LR: 0.005
[153/1429] Loss: 0.1318 | LR: 0.005
[154/1429] Loss: 0.1214 | LR: 0.005
[155/1429] Loss: 1.0157 | LR: 0.005
[156/1429] Loss: 0.4205 | LR: 0.005
[157/1429] Loss: 0.3748 | LR: 0.005
[158/1429] Loss: 0.5913 | LR: 0.005
[159/1429] Loss: 0.2167 | LR: 0.005
[160/1429] Loss: 0.5598 | LR: 0.005
[161/1429] Loss: 1.048 | LR: 0.005
[162/1429] Loss: 0.6045 | LR: 0.005
[163/1429] Loss: 0.5392 | LR: 0.005
[164/1429] Loss: 0.9731 | LR: 0.005
[165/1429] Loss: 0.2671 | LR: 0.005
[166/1429] Loss: 0.7086 | LR: 0.005
[167/1429] Loss: 0.5444 | LR: 0.005
[168/1429] Loss: 0.1935 | LR: 0.005
[169/1429] Loss: 0.2447 | LR: 0.005
[170/1429] Loss: 0.4472 | LR: 0.005
[171/1429] Loss: 0.1163 | LR: 0.005
[172/1429] Loss: 0.5617 | LR: 0.005
[173/1429] Loss: 0.1284 | LR: 0.005
[174/1429] Loss: 0.214 | LR: 0.005
[175/1429] Loss: 0.8345 | LR: 0.005
[176/1429] Loss: 0.6929 | LR: 0.005
[177/1429] Loss: 0.767 | LR: 0.005
[178/1429] Loss: 0.1748 | LR: 0.005
[179/1429] Loss: 0.1336 | LR: 0.005
[180/1429] Loss: 0.9079 | LR: 0.005
[181/1429] Loss: 0.3092 | LR: 0.005
[182/1429] Loss: 0.7498 | LR: 0.005
[183/1429] Loss: 0.33 | LR: 0.005
[184/1429] Loss: 1.0252 | LR: 0.005
[185/1429] Loss: 0.5745 | LR: 0.005
[186/1429] Loss: 0.3396 | LR: 0.005
[187/1429] Loss: 0.1202 | LR: 0.005
[188/1429] Loss: 0.9436 | LR: 0.005
[189/1429] Loss: 0.5064 | LR: 0.005
[190/1429] Loss: 0.5112 | LR: 0.005
[191/1429] Loss: 0.7365 | LR: 0.005
[192/1429] Loss: 0.4618 | LR: 0.005
[193/1429] Loss: 0.4193 | LR: 0.005
[194/1429] Loss: 0.6461 | LR: 0.005
[195/1429] Loss: 0.9497 | LR: 0.005
[196/1429] Loss: 0.6643 | LR: 0.005
[197/1429] Loss: 0.3072 | LR: 0.005
[198/1429] Loss: 0.7559 | LR: 0.005
[199/1429] Loss: 0.4424 | LR: 0.005
[200/1429] Loss: 0.1991 | LR: 0.005
[201/1429] Loss: 0.9119 | LR: 0.005
[202/1429] Loss: 0.9033 | LR: 0.005
[203/1429] Loss: 0.4088 | LR: 0.005
[204/1429] Loss: 0.8167 | LR: 0.005
[205/1429] Loss: 0.882 | LR: 0.005
[206/1429] Loss: 0.1339 | LR: 0.005
[207/1429] Loss: 0.7702 | LR: 0.005
[208/1429] Loss: 0.9783 | LR: 0.005
[209/1429] Loss: 0.5718 | LR: 0.005
[210/1429] Loss: 0.6282 | LR: 0.005
[211/1429] Loss: 0.5433 | LR: 0.005
[212/1429] Loss: 0.8694 | LR: 0.005
[213/1429] Loss: 0.9064 | LR: 0.005
[214/1429] Loss: 0.8372 | LR: 0.005
[215/1429] Loss: 0.848 | LR: 0.005
[216/1429] Loss: 0.6706 | LR: 0.005
[217/1429] Loss: 0.3732 | LR: 0.005
[218/1429] Loss: 0.7881 | LR: 0.005
[219/1429] Loss: 0.3073 | LR: 0.005
[220/1429] Loss: 0.7016 | LR: 0.005
[221/1429] Loss: 0.3476 | LR: 0.005
[222/1429] Loss: 0.5515 | LR: 0.005
[223/1429] Loss: 0.2269 | LR: 0.005
[224/1429] Loss: 0.1469 | LR: 0.005
[225/1429] Loss: 0.2796 | LR: 0.005
[226/1429] Loss: 0.2464 | LR: 0.005
[227/1429] Loss: 0.3679 | LR: 0.005
[228/1429] Loss: 0.7885 | LR: 0.005
[229/1429] Loss: 0.5534 | LR: 0.005
[230/1429] Loss: 0.3634 | LR: 0.005
[231/1429] Loss: 1.0806 | LR: 0.005
[232/1429] Loss: 0.6558 | LR: 0.005
[233/1429] Loss: 0.3141 | LR: 0.005
[234/1429] Loss: 0.3172 | LR: 0.005
[235/1429] Loss: 0.7915 | LR: 0.005
[236/1429] Loss: 0.9757 | LR: 0.005
[237/1429] Loss: 0.8448 | LR: 0.005
[238/1429] Loss: 0.5872 | LR: 0.005
[239/1429] Loss: 0.515 | LR: 0.005
[240/1429] Loss: 0.9448 | LR: 0.005
[241/1429] Loss: 0.7792 | LR: 0.005
[242/1429] Loss: 1.0369 | LR: 0.005
[243/1429] Loss: 0.9661 | LR: 0.005
[244/1429] Loss: 0.3141 | LR: 0.005
[245/1429] Loss: 0.6434 | LR: 0.005
[246/1429] Loss: 1.0392 | LR: 0.005
[247/1429] Loss: 0.2863 | LR: 0.005
[248/1429] Loss: 0.7736 | LR: 0.005
[249/1429] Loss: 1.0343 | LR: 0.005
[250/1429] Loss: 0.3127 | LR: 0.005
[251/1429] Loss: 1.0014 | LR: 0.005
[252/1429] Loss: 0.685 | LR: 0.005
[253/1429] Loss: 0.8265 | LR: 0.005
[254/1429] Loss: 0.7645 | LR: 0.005
[255/1429] Loss: 0.3392 | LR: 0.005
[256/1429] Loss: 1.0519 | LR: 0.005
[257/1429] Loss: 0.6544 | LR: 0.005
[258/1429] Loss: 0.4089 | LR: 0.005
[259/1429] Loss: 0.9732 | LR: 0.005
[260/1429] Loss: 1.0745 | LR: 0.005
[261/1429] Loss: 1.0192 | LR: 0.005
[262/1429] Loss: 0.7674 | LR: 0.005
[263/1429] Loss: 0.164 | LR: 0.005
[264/1429] Loss: 0.5922 | LR: 0.005
[265/1429] Loss: 0.4288 | LR: 0.005
[266/1429] Loss: 0.8421 | LR: 0.005
[267/1429] Loss: 0.5983 | LR: 0.005
[268/1429] Loss: 0.9327 | LR: 0.005
[269/1429] Loss: 0.4103 | LR: 0.005
[270/1429] Loss: 0.7289 | LR: 0.005
[271/1429] Loss: 0.6093 | LR: 0.005
[272/1429] Loss: 0.2064 | LR: 0.005
[273/1429] Loss: 0.4951 | LR: 0.005
[274/1429] Loss: 0.494 | LR: 0.005
[275/1429] Loss: 0.4314 | LR: 0.005
[276/1429] Loss: 0.4918 | LR: 0.005
[277/1429] Loss: 0.118 | LR: 0.005
[278/1429] Loss: 0.705 | LR: 0.005
[279/1429] Loss: 0.5644 | LR: 0.005
[280/1429] Loss: 0.3752 | LR: 0.005
[281/1429] Loss: 0.307 | LR: 0.005
[282/1429] Loss: 0.7491 | LR: 0.005
[283/1429] Loss: 0.8507 | LR: 0.005
[284/1429] Loss: 0.971 | LR: 0.005
[285/1429] Loss: 0.1317 | LR: 0.005
[286/1429] Loss: 0.2936 | LR: 0.005
[287/1429] Loss: 0.7074 | LR: 0.005
[288/1429] Loss: 0.8076 | LR: 0.005
[289/1429] Loss: 0.321 | LR: 0.005
[290/1429] Loss: 0.7569 | LR: 0.005
[291/1429] Loss: 0.8477 | LR: 0.005
[292/1429] Loss: 0.9119 | LR: 0.005
[293/1429] Loss: 0.5801 | LR: 0.005
[294/1429] Loss: 0.321 | LR: 0.005
[295/1429] Loss: 0.6111 | LR: 0.005
[296/1429] Loss: 0.2887 | LR: 0.005
[297/1429] Loss: 0.289 | LR: 0.005
[298/1429] Loss: 0.666 | LR: 0.005
[299/1429] Loss: 0.5833 | LR: 0.005
[300/1429] Loss: 0.9733 | LR: 0.005
[301/1429] Loss: 0.3744 | LR: 0.005
[302/1429] Loss: 0.3669 | LR: 0.005
[303/1429] Loss: 0.2206 | LR: 0.005
[304/1429] Loss: 1.0758 | LR: 0.005
[305/1429] Loss: 0.398 | LR: 0.005
[306/1429] Loss: 0.2725 | LR: 0.005
[307/1429] Loss: 0.9208 | LR: 0.005
[308/1429] Loss: 1.0093 | LR: 0.005
[309/1429] Loss: 0.2469 | LR: 0.005
[310/1429] Loss: 0.1766 | LR: 0.005
[311/1429] Loss: 0.3356 | LR: 0.005
[312/1429] Loss: 0.2743 | LR: 0.005
[313/1429] Loss: 0.4103 | LR: 0.005
[314/1429] Loss: 0.8733 | LR: 0.005
[315/1429] Loss: 0.3907 | LR: 0.005
[316/1429] Loss: 0.7384 | LR: 0.005
[317/1429] Loss: 1.0171 | LR: 0.005
[318/1429] Loss: 0.9073 | LR: 0.005
[319/1429] Loss: 0.7382 | LR: 0.005
[320/1429] Loss: 0.6906 | LR: 0.005
[321/1429] Loss: 0.5561 | LR: 0.005
[322/1429] Loss: 0.403 | LR: 0.005
[323/1429] Loss: 0.1449 | LR: 0.005
[324/1429] Loss: 0.4645 | LR: 0.005
[325/1429] Loss: 0.5504 | LR: 0.005
[326/1429] Loss: 0.9073 | LR: 0.005
[327/1429] Loss: 0.3082 | LR: 0.005
[328/1429] Loss: 0.1672 | LR: 0.005
[329/1429] Loss: 0.9134 | LR: 0.005
[330/1429] Loss: 0.1891 | LR: 0.005
[331/1429] Loss: 0.5301 | LR: 0.005
[332/1429] Loss: 0.8846 | LR: 0.005
[333/1429] Loss: 0.1943 | LR: 0.005
[334/1429] Loss: 0.7621 | LR: 0.005
[335/1429] Loss: 0.9841 | LR: 0.005
[336/1429] Loss: 1.0369 | LR: 0.005
[337/1429] Loss: 0.4206 | LR: 0.005
[338/1429] Loss: 0.6426 | LR: 0.005
[339/1429] Loss: 0.7649 | LR: 0.005
[340/1429] Loss: 0.8624 | LR: 0.005
[341/1429] Loss: 0.26 | LR: 0.005
[342/1429] Loss: 0.6482 | LR: 0.005
[343/1429] Loss: 0.8988 | LR: 0.005
[344/1429] Loss: 0.365 | LR: 0.005
[345/1429] Loss: 1.0485 | LR: 0.005
[346/1429] Loss: 0.7282 | LR: 0.005
[347/1429] Loss: 1.0998 | LR: 0.005
[348/1429] Loss: 1.0814 | LR: 0.005
[349/1429] Loss: 0.2267 | LR: 0.005
[350/1429] Loss: 0.5253 | LR: 0.005
[351/1429] Loss: 0.638 | LR: 0.005
[352/1429] Loss: 0.6738 | LR: 0.005
[353/1429] Loss: 0.3309 | LR: 0.005
[354/1429] Loss: 0.1188 | LR: 0.005
[355/1429] Loss: 0.2629 | LR: 0.005
[356/1429] Loss: 0.9882 | LR: 0.005
[357/1429] Loss: 0.4306 | LR: 0.005
[358/1429] Loss: 0.6775 | LR: 0.005
[359/1429] Loss: 0.2456 | LR: 0.005
[360/1429] Loss: 0.262 | LR: 0.005
[361/1429] Loss: 0.7203 | LR: 0.005
[362/1429] Loss: 0.2405 | LR: 0.005
[363/1429] Loss: 0.1333 | LR: 0.005
[364/1429] Loss: 0.8212 | LR: 0.005
[365/1429] Loss: 0.4735 | LR: 0.005
[366/1429] Loss: 0.7644 | LR: 0.005
[367/1429] Loss: 0.7108 | LR: 0.005
[368/1429] Loss: 0.284 | LR: 0.005
[369/1429] Loss: 0.5606 | LR: 0.005
[370/1429] Loss: 0.9879 | LR: 0.005
[371/1429] Loss: 0.7785 | LR: 0.005
[372/1429] Loss: 0.3284 | LR: 0.005
[373/1429] Loss: 0.1715 | LR: 0.005
[374/1429] Loss: 0.7146 | LR: 0.005
[375/1429] Loss: 0.3379 | LR: 0.005
[376/1429] Loss: 1.0855 | LR: 0.005
[377/1429] Loss: 0.1659 | LR: 0.005
[378/1429] Loss: 0.4382 | LR: 0.005
[379/1429] Loss: 0.7802 | LR: 0.005
[380/1429] Loss: 0.3884 | LR: 0.005
[381/1429] Loss: 0.102 | LR: 0.005
[382/1429] Loss: 0.2378 | LR: 0.005
[383/1429] Loss: 0.9626 | LR: 0.005
[384/1429] Loss: 0.7308 | LR: 0.005
[385/1429] Loss: 0.9775 | LR: 0.005
[386/1429] Loss: 0.3313 | LR: 0.005
[387/1429] Loss: 0.2477 | LR: 0.005
[388/1429] Loss: 0.3699 | LR: 0.005
[389/1429] Loss: 0.9532 | LR: 0.005
[390/1429] Loss: 0.7776 | LR: 0.005
[391/1429] Loss: 0.8674 | LR: 0.005
[392/1429] Loss: 0.2734 | LR: 0.005
[393/1429] Loss: 0.2199 | LR: 0.005
[394/1429] Loss: 1.0335 | LR: 0.005
[395/1429] Loss: 0.857 | LR: 0.005
[396/1429] Loss: 0.2652 | LR: 0.005
[397/1429] Loss: 0.4963 | LR: 0.005
[398/1429] Loss: 0.2604 | LR: 0.005
[399/1429] Loss: 0.9896 | LR: 0.005
[400/1429] Loss: 0.2871 | LR: 0.005
[401/1429] Loss: 0.4031 | LR: 0.005
[402/1429] Loss: 0.3414 | LR: 0.005
[403/1429] Loss: 0.7124 | LR: 0.005
[404/1429] Loss: 0.7664 | LR: 0.005
[405/1429] Loss: 0.2015 | LR: 0.005
[406/1429] Loss: 0.5814 | LR: 0.005
[407/1429] Loss: 0.1335 | LR: 0.005
[408/1429] Loss: 0.6091 | LR: 0.005
[409/1429] Loss: 0.3647 | LR: 0.005
[410/1429] Loss: 0.4628 | LR: 0.005
[411/1429] Loss: 0.8368 | LR: 0.005
[412/1429] Loss: 0.6533 | LR: 0.005
[413/1429] Loss: 0.3194 | LR: 0.005
[414/1429] Loss: 1.0314 | LR: 0.005
[415/1429] Loss: 0.9142 | LR: 0.005
[416/1429] Loss: 0.598 | LR: 0.005
[417/1429] Loss: 0.9968 | LR: 0.005
[418/1429] Loss: 0.4623 | LR: 0.005
[419/1429] Loss: 0.8029 | LR: 0.005
[420/1429] Loss: 1.0329 | LR: 0.005
[421/1429] Loss: 0.7578 | LR: 0.005
[422/1429] Loss: 0.7684 | LR: 0.005
[423/1429] Loss: 0.2445 | LR: 0.005
[424/1429] Loss: 0.9181 | LR: 0.005
[425/1429] Loss: 0.4818 | LR: 0.005
[426/1429] Loss: 1.0212 | LR: 0.005
[427/1429] Loss: 0.1798 | LR: 0.005
[428/1429] Loss: 0.4646 | LR: 0.005
[429/1429] Loss: 0.2726 | LR: 0.005
[430/1429] Loss: 0.5327 | LR: 0.005
[431/1429] Loss: 0.8215 | LR: 0.005
[432/1429] Loss: 0.7738 | LR: 0.005
[433/1429] Loss: 0.6466 | LR: 0.005
[434/1429] Loss: 0.4425 | LR: 0.005
[435/1429] Loss: 0.3323 | LR: 0.005
[436/1429] Loss: 0.5315 | LR: 0.005
[437/1429] Loss: 0.384 | LR: 0.005
[438/1429] Loss: 1.0181 | LR: 0.005
[439/1429] Loss: 0.7199 | LR: 0.005
[440/1429] Loss: 0.7976 | LR: 0.005
[441/1429] Loss: 0.5531 | LR: 0.005
[442/1429] Loss: 0.9648 | LR: 0.005
[443/1429] Loss: 0.2544 | LR: 0.005
[444/1429] Loss: 0.9027 | LR: 0.005
[445/1429] Loss: 1.03 | LR: 0.005
[446/1429] Loss: 0.1139 | LR: 0.005
[447/1429] Loss: 0.9241 | LR: 0.005
[448/1429] Loss: 0.1367 | LR: 0.005
[449/1429] Loss: 0.5303 | LR: 0.005
[450/1429] Loss: 0.4408 | LR: 0.005
[451/1429] Loss: 0.4784 | LR: 0.005
[452/1429] Loss: 0.801 | LR: 0.005
[453/1429] Loss: 1.0671 | LR: 0.005
[454/1429] Loss: 1.0791 | LR: 0.005
[455/1429] Loss: 0.1821 | LR: 0.005
[456/1429] Loss: 0.4619 | LR: 0.005
[457/1429] Loss: 0.9984 | LR: 0.005
[458/1429] Loss: 0.7319 | LR: 0.005
[459/1429] Loss: 0.8669 | LR: 0.005
[460/1429] Loss: 0.5285 | LR: 0.005
[461/1429] Loss: 0.8173 | LR: 0.005
[462/1429] Loss: 0.8924 | LR: 0.005
[463/1429] Loss: 0.6274 | LR: 0.005
[464/1429] Loss: 1.0262 | LR: 0.005
[465/1429] Loss: 0.9162 | LR: 0.005
[466/1429] Loss: 0.2191 | LR: 0.005
[467/1429] Loss: 0.631 | LR: 0.005
[468/1429] Loss: 1.0001 | LR: 0.005
[469/1429] Loss: 0.7068 | LR: 0.005
[470/1429] Loss: 1.0501 | LR: 0.005
[471/1429] Loss: 0.5873 | LR: 0.005
[472/1429] Loss: 0.7404 | LR: 0.005
[473/1429] Loss: 1.051 | LR: 0.005
[474/1429] Loss: 0.5228 | LR: 0.005
[475/1429] Loss: 0.9797 | LR: 0.005
[476/1429] Loss: 0.6522 | LR: 0.005
[477/1429] Loss: 0.8355 | LR: 0.005
[478/1429] Loss: 0.3963 | LR: 0.005
[479/1429] Loss: 0.8275 | LR: 0.005
[480/1429] Loss: 0.2492 | LR: 0.005
[481/1429] Loss: 0.9571 | LR: 0.005
[482/1429] Loss: 0.7641 | LR: 0.005
[483/1429] Loss: 0.675 | LR: 0.005
[484/1429] Loss: 0.2234 | LR: 0.005
[485/1429] Loss: 0.765 | LR: 0.005
[486/1429] Loss: 0.1472 | LR: 0.005
[487/1429] Loss: 0.2983 | LR: 0.005
[488/1429] Loss: 0.5369 | LR: 0.005
[489/1429] Loss: 0.7003 | LR: 0.005
[490/1429] Loss: 0.4756 | LR: 0.005
[491/1429] Loss: 0.7758 | LR: 0.005
[492/1429] Loss: 0.5456 | LR: 0.005
[493/1429] Loss: 0.9746 | LR: 0.005
[494/1429] Loss: 1.0183 | LR: 0.005
[495/1429] Loss: 0.2093 | LR: 0.005
[496/1429] Loss: 0.6957 | LR: 0.005
[497/1429] Loss: 1.0682 | LR: 0.005
[498/1429] Loss: 0.8097 | LR: 0.005
[499/1429] Loss: 0.1285 | LR: 0.005
[500/1429] Loss: 0.9589 | LR: 0.005
[501/1429] Loss: 0.8792 | LR: 0.005
[502/1429] Loss: 0.2303 | LR: 0.005
[503/1429] Loss: 0.3693 | LR: 0.005
[504/1429] Loss: 0.2976 | LR: 0.005
[505/1429] Loss: 0.4013 | LR: 0.005
[506/1429] Loss: 1.0851 | LR: 0.005
[507/1429] Loss: 0.4037 | LR: 0.005
[508/1429] Loss: 0.7996 | LR: 0.005
[509/1429] Loss: 0.413 | LR: 0.005
[510/1429] Loss: 0.1644 | LR: 0.005
[511/1429] Loss: 0.8037 | LR: 0.005
[512/1429] Loss: 0.4287 | LR: 0.005
[513/1429] Loss: 0.9918 | LR: 0.005
[514/1429] Loss: 0.2919 | LR: 0.005
[515/1429] Loss: 0.594 | LR: 0.005
[516/1429] Loss: 0.5132 | LR: 0.005
[517/1429] Loss: 0.4336 | LR: 0.005
[518/1429] Loss: 0.617 | LR: 0.005
[519/1429] Loss: 0.3445 | LR: 0.005
[520/1429] Loss: 0.8213 | LR: 0.005
[521/1429] Loss: 0.6064 | LR: 0.005
[522/1429] Loss: 0.4077 | LR: 0.005
[523/1429] Loss: 0.8919 | LR: 0.005
[524/1429] Loss: 0.2519 | LR: 0.005
[525/1429] Loss: 0.8175 | LR: 0.005
[526/1429] Loss: 1.0147 | LR: 0.005
[527/1429] Loss: 0.6034 | LR: 0.005
[528/1429] Loss: 0.6072 | LR: 0.005
[529/1429] Loss: 0.9885 | LR: 0.005
[530/1429] Loss: 0.2243 | LR: 0.005
[531/1429] Loss: 0.6521 | LR: 0.005
[532/1429] Loss: 0.4504 | LR: 0.005
[533/1429] Loss: 0.9763 | LR: 0.005
[534/1429] Loss: 1.0762 | LR: 0.005
[535/1429] Loss: 0.9644 | LR: 0.005
[536/1429] Loss: 0.8536 | LR: 0.005
[537/1429] Loss: 0.144 | LR: 0.005
[538/1429] Loss: 0.5909 | LR: 0.005
[539/1429] Loss: 0.8111 | LR: 0.005
[540/1429] Loss: 0.3678 | LR: 0.005
[541/1429] Loss: 0.3396 | LR: 0.005
[542/1429] Loss: 0.7883 | LR: 0.005
[543/1429] Loss: 0.8871 | LR: 0.005
[544/1429] Loss: 0.5744 | LR: 0.005
[545/1429] Loss: 0.6412 | LR: 0.005
[546/1429] Loss: 0.8847 | LR: 0.005
[547/1429] Loss: 1.0194 | LR: 0.005
[548/1429] Loss: 0.7427 | LR: 0.005
[549/1429] Loss: 0.5822 | LR: 0.005
[550/1429] Loss: 0.1674 | LR: 0.005
[551/1429] Loss: 1.0993 | LR: 0.005
[552/1429] Loss: 0.6135 | LR: 0.005
[553/1429] Loss: 0.4987 | LR: 0.005
[554/1429] Loss: 0.241 | LR: 0.005
[555/1429] Loss: 1.0866 | LR: 0.005
[556/1429] Loss: 0.623 | LR: 0.005
[557/1429] Loss: 1.0286 | LR: 0.005
[558/1429] Loss: 0.6841 | LR: 0.005
[559/1429] Loss: 0.2951 | LR: 0.005
[560/1429] Loss: 0.7799 | LR: 0.005
[561/1429] Loss: 0.5618 | LR: 0.005
[562/1429] Loss: 0.4482 | LR: 0.005
[563/1429] Loss: 0.284 | LR: 0.005
[564/1429] Loss: 1.0401 | LR: 0.005
[565/1429] Loss: 0.9823 | LR: 0.005
[566/1429] Loss: 0.614 | LR: 0.005
[567/1429] Loss: 0.6393 | LR: 0.005
[568/1429] Loss: 0.464 | LR: 0.005
[569/1429] Loss: 0.2634 | LR: 0.005
[570/1429] Loss: 0.7551 | LR: 0.005
[571/1429] Loss: 0.1544 | LR: 0.005
[572/1429] Loss: 0.9168 | LR: 0.005
[573/1429] Loss: 1.0511 | LR: 0.005
[574/1429] Loss: 0.399 | LR: 0.005
[575/1429] Loss: 0.3447 | LR: 0.005
[576/1429] Loss: 0.1082 | LR: 0.005
[577/1429] Loss: 0.9636 | LR: 0.005
[578/1429] Loss: 0.8519 | LR: 0.005
[579/1429] Loss: 0.9642 | LR: 0.005
[580/1429] Loss: 0.8178 | LR: 0.005
[581/1429] Loss: 0.6666 | LR: 0.005
[582/1429] Loss: 0.1433 | LR: 0.005
[583/1429] Loss: 0.4767 | LR: 0.005
[584/1429] Loss: 0.1418 | LR: 0.005
[585/1429] Loss: 0.6117 | LR: 0.005
[586/1429] Loss: 0.419 | LR: 0.005
[587/1429] Loss: 0.9808 | LR: 0.005
[588/1429] Loss: 0.409 | LR: 0.005
[589/1429] Loss: 0.6344 | LR: 0.005
[590/1429] Loss: 0.9797 | LR: 0.005
[591/1429] Loss: 1.0521 | LR: 0.005
[592/1429] Loss: 0.4897 | LR: 0.005
[593/1429] Loss: 0.1784 | LR: 0.005
[594/1429] Loss: 0.8171 | LR: 0.005
[595/1429] Loss: 0.7766 | LR: 0.005
[596/1429] Loss: 0.9176 | LR: 0.005
[597/1429] Loss: 0.7838 | LR: 0.005
[598/1429] Loss: 0.1241 | LR: 0.005
[599/1429] Loss: 1.0326 | LR: 0.005
[600/1429] Loss: 0.3363 | LR: 0.005
[601/1429] Loss: 0.552 | LR: 0.005
[602/1429] Loss: 0.9174 | LR: 0.005
[603/1429] Loss: 0.5033 | LR: 0.005
[604/1429] Loss: 0.2641 | LR: 0.005
[605/1429] Loss: 0.4719 | LR: 0.005
[606/1429] Loss: 0.2422 | LR: 0.005
[607/1429] Loss: 0.5587 | LR: 0.005
[608/1429] Loss: 0.2113 | LR: 0.005
[609/1429] Loss: 0.8764 | LR: 0.005
[610/1429] Loss: 0.9017 | LR: 0.005
[611/1429] Loss: 0.6959 | LR: 0.005
[612/1429] Loss: 1.0243 | LR: 0.005
[613/1429] Loss: 0.7784 | LR: 0.005
[614/1429] Loss: 0.9409 | LR: 0.005
[615/1429] Loss: 0.3916 | LR: 0.005
[616/1429] Loss: 0.3187 | LR: 0.005
[617/1429] Loss: 1.0926 | LR: 0.005
[618/1429] Loss: 0.419 | LR: 0.005
[619/1429] Loss: 0.8976 | LR: 0.005
[620/1429] Loss: 0.9701 | LR: 0.005
[621/1429] Loss: 0.1482 | LR: 0.005
[622/1429] Loss: 0.483 | LR: 0.005
[623/1429] Loss: 0.2057 | LR: 0.005
[624/1429] Loss: 0.2191 | LR: 0.005
[625/1429] Loss: 0.8066 | LR: 0.005
[626/1429] Loss: 0.7138 | LR: 0.005
[627/1429] Loss: 0.5132 | LR: 0.005
[628/1429] Loss: 0.5743 | LR: 0.005
[629/1429] Loss: 0.2743 | LR: 0.005
[630/1429] Loss: 0.2873 | LR: 0.005
[631/1429] Loss: 0.3936 | LR: 0.005
[632/1429] Loss: 0.4929 | LR: 0.005
[633/1429] Loss: 0.9695 | LR: 0.005
[634/1429] Loss: 0.9089 | LR: 0.005
[635/1429] Loss: 0.7443 | LR: 0.005
[636/1429] Loss: 1.0914 | LR: 0.005
[637/1429] Loss: 0.9744 | LR: 0.005
[638/1429] Loss: 0.2182 | LR: 0.005
[639/1429] Loss: 0.6514 | LR: 0.005
[640/1429] Loss: 0.36 | LR: 0.005
[641/1429] Loss: 0.6213 | LR: 0.005
[642/1429] Loss: 1.0965 | LR: 0.005
[643/1429] Loss: 0.7263 | LR: 0.005
[644/1429] Loss: 0.4975 | LR: 0.005
[645/1429] Loss: 0.7253 | LR: 0.005
[646/1429] Loss: 1.0159 | LR: 0.005
[647/1429] Loss: 0.2713 | LR: 0.005
[648/1429] Loss: 0.7906 | LR: 0.005
[649/1429] Loss: 0.1103 | LR: 0.005
[650/1429] Loss: 0.3837 | LR: 0.005
[651/1429] Loss: 0.3737 | LR: 0.005
[652/1429] Loss: 0.6233 | LR: 0.005
[653/1429] Loss: 0.1183 | LR: 0.005
[654/1429] Loss: 0.4219 | LR: 0.005
[655/1429] Loss: 0.5171 | LR: 0.005
[656/1429] Loss: 0.1514 | LR: 0.005
[657/1429] Loss: 0.8837 | LR: 0.005
[658/1429] Loss: 0.5129 | LR: 0.005
[659/1429] Loss: 0.995 | LR: 0.005
[660/1429] Loss: 0.839 | LR: 0.005
[661/1429] Loss: 0.9712 | LR: 0.005
[662/1429] Loss: 0.967 | LR: 0.005
[663/1429] Loss: 0.8967 | LR: 0.005
[664/1429] Loss: 0.9202 | LR: 0.005
[665/1429] Loss: 0.3188 | LR: 0.005
[666/1429] Loss: 0.1271 | LR: 0.005
[667/1429] Loss: 0.3011 | LR: 0.005
[668/1429] Loss: 0.7926 | LR: 0.005
[669/1429] Loss: 1.0022 | LR: 0.005
[670/1429] Loss: 0.4756 | LR: 0.005
[671/1429] Loss: 0.2243 | LR: 0.005
[672/1429] Loss: 0.7496 | LR: 0.005
[673/1429] Loss: 0.4651 | LR: 0.005
[674/1429] Loss: 0.794 | LR: 0.005
[675/1429] Loss: 0.4356 | LR: 0.005
[676/1429] Loss: 0.2803 | LR: 0.005
[677/1429] Loss: 0.3303 | LR: 0.005
[678/1429] Loss: 0.8238 | LR: 0.005
[679/1429] Loss: 0.9941 | LR: 0.005
[680/1429] Loss: 0.2525 | LR: 0.005
[681/1429] Loss: 0.3539 | LR: 0.005
[682/1429] Loss: 0.8177 | LR: 0.005
[683/1429] Loss: 0.2279 | LR: 0.005
[684/1429] Loss: 0.3519 | LR: 0.005
[685/1429] Loss: 0.9999 | LR: 0.005
[686/1429] Loss: 0.4871 | LR: 0.005
[687/1429] Loss: 0.6365 | LR: 0.005
[688/1429] Loss: 0.2719 | LR: 0.005
[689/1429] Loss: 0.7266 | LR: 0.005
[690/1429] Loss: 1.0736 | LR: 0.005
[691/1429] Loss: 0.1399 | LR: 0.005
[692/1429] Loss: 0.9054 | LR: 0.005
[693/1429] Loss: 0.9104 | LR: 0.005
[694/1429] Loss: 0.731 | LR: 0.005
[695/1429] Loss: 0.9415 | LR: 0.005
[696/1429] Loss: 1.0069 | LR: 0.005
[697/1429] Loss: 0.6604 | LR: 0.005
[698/1429] Loss: 1.0342 | LR: 0.005
[699/1429] Loss: 0.7072 | LR: 0.005
[700/1429] Loss: 0.6792 | LR: 0.005
[701/1429] Loss: 0.8282 | LR: 0.005
[702/1429] Loss: 0.6821 | LR: 0.005
[703/1429] Loss: 0.8979 | LR: 0.005
[704/1429] Loss: 0.7104 | LR: 0.005
[705/1429] Loss: 0.9266 | LR: 0.005
[706/1429] Loss: 0.9677 | LR: 0.005
[707/1429] Loss: 0.2066 | LR: 0.005
[708/1429] Loss: 0.8892 | LR: 0.005
[709/1429] Loss: 0.9066 | LR: 0.005
[710/1429] Loss: 0.3442 | LR: 0.005
[711/1429] Loss: 1.0854 | LR: 0.005
[712/1429] Loss: 0.3703 | LR: 0.005
[713/1429] Loss: 0.1325 | LR: 0.005
[714/1429] Loss: 0.9255 | LR: 0.005
[715/1429] Loss: 0.6269 | LR: 0.005
[716/1429] Loss: 1.0208 | LR: 0.005
[717/1429] Loss: 0.5393 | LR: 0.005
[718/1429] Loss: 0.2635 | LR: 0.005
[719/1429] Loss: 1.0275 | LR: 0.005
[720/1429] Loss: 0.4178 | LR: 0.005
[721/1429] Loss: 0.1023 | LR: 0.005
[722/1429] Loss: 0.7165 | LR: 0.005
[723/1429] Loss: 0.2056 | LR: 0.005
[724/1429] Loss: 0.3091 | LR: 0.005
[725/1429] Loss: 0.9733 | LR: 0.005
[726/1429] Loss: 0.5983 | LR: 0.005
[727/1429] Loss: 0.3539 | LR: 0.005
[728/1429] Loss: 0.1722 | LR: 0.005
[729/1429] Loss: 0.7159 | LR: 0.005
[730/1429] Loss: 0.8214 | LR: 0.005
[731/1429] Loss: 0.4943 | LR: 0.005
[732/1429] Loss: 0.8904 | LR: 0.005
[733/1429] Loss: 0.5924 | LR: 0.005
[734/1429] Loss: 0.4601 | LR: 0.005
[735/1429] Loss: 0.2729 | LR: 0.005
[736/1429] Loss: 0.8289 | LR: 0.005
[737/1429] Loss: 0.6105 | LR: 0.005
[738/1429] Loss: 0.3269 | LR: 0.005
[739/1429] Loss: 1.0247 | LR: 0.005
[740/1429] Loss: 0.7639 | LR: 0.005
[741/1429] Loss: 0.1624 | LR: 0.005
[742/1429] Loss: 0.2745 | LR: 0.005
[743/1429] Loss: 1.0424 | LR: 0.005
[744/1429] Loss: 0.7792 | LR: 0.005
[745/1429] Loss: 0.1207 | LR: 0.005
[746/1429] Loss: 0.6226 | LR: 0.005
[747/1429] Loss: 0.7918 | LR: 0.005
[748/1429] Loss: 0.6107 | LR: 0.005
[749/1429] Loss: 0.1987 | LR: 0.005
[750/1429] Loss: 0.309 | LR: 0.005
[751/1429] Loss: 0.7586 | LR: 0.005
[752/1429] Loss: 0.9718 | LR: 0.005
[753/1429] Loss: 0.5574 | LR: 0.005
[754/1429] Loss: 0.4552 | LR: 0.005
[755/1429] Loss: 0.8853 | LR: 0.005
[756/1429] Loss: 1.0079 | LR: 0.005
[757/1429] Loss: 0.1779 | LR: 0.005
[758/1429] Loss: 0.8893 | LR: 0.005
[759/1429] Loss: 0.5459 | LR: 0.005
[760/1429] Loss: 0.1754 | LR: 0.005
[761/1429] Loss: 0.973 | LR: 0.005
[762/1429] Loss: 1.0426 | LR: 0.005
[763/1429] Loss: 0.2945 | LR: 0.005
[764/1429] Loss: 0.6631 | LR: 0.005
[765/1429] Loss: 0.3799 | LR: 0.005
[766/1429] Loss: 0.121 | LR: 0.005
[767/1429] Loss: 1.0733 | LR: 0.005
[768/1429] Loss: 0.3203 | LR: 0.005
[769/1429] Loss: 0.6208 | LR: 0.005
[770/1429] Loss: 0.7748 | LR: 0.005
[771/1429] Loss: 0.7542 | LR: 0.005
[772/1429] Loss: 0.5048 | LR: 0.005
[773/1429] Loss: 0.9829 | LR: 0.005
[774/1429] Loss: 0.1405 | LR: 0.005
[775/1429] Loss: 0.9347 | LR: 0.005
[776/1429] Loss: 0.4574 | LR: 0.005
[777/1429] Loss: 0.1099 | LR: 0.005
[778/1429] Loss: 0.2188 | LR: 0.005
[779/1429] Loss: 1.0128 | LR: 0.005
[780/1429] Loss: 0.5751 | LR: 0.005
[781/1429] Loss: 1.0126 | LR: 0.005
[782/1429] Loss: 0.7167 | LR: 0.005
[783/1429] Loss: 0.6795 | LR: 0.005
[784/1429] Loss: 0.2735 | LR: 0.005
[785/1429] Loss: 0.7629 | LR: 0.005
[786/1429] Loss: 1.0571 | LR: 0.005
[787/1429] Loss: 0.4794 | LR: 0.005
[788/1429] Loss: 1.0989 | LR: 0.005
[789/1429] Loss: 0.7331 | LR: 0.005
[790/1429] Loss: 0.5079 | LR: 0.005
[791/1429] Loss: 0.2015 | LR: 0.005
[792/1429] Loss: 1.0952 | LR: 0.005
[793/1429] Loss: 0.8679 | LR: 0.005
[794/1429] Loss: 0.2817 | LR: 0.005
[795/1429] Loss: 0.412 | LR: 0.005
[796/1429] Loss: 0.2491 | LR: 0.005
[797/1429] Loss: 0.3482 | LR: 0.005
[798/1429] Loss: 1.0276 | LR: 0.005
[799/1429] Loss: 0.1071 | LR: 0.005
[800/1429] Loss: 0.3909 | LR: 0.005
[801/1429] Loss: 0.8331 | LR: 0.005
[802/1429] Loss: 0.3242 | LR: 0.005
[803/1429] Loss: 0.3193 | LR: 0.005
[804/1429] Loss: 0.2066 | LR: 0.005
[805/1429] Loss: 1.0275 | LR: 0.005
[806/1429] Loss: 0.1664 | LR: 0.005
[807/1429] Loss: 0.5985 | LR: 0.005
[808/1429] Loss: 0.8837 | LR: 0.005
[809/1429] Loss: 0.9585 | LR: 0.005
[810/1429] Loss: 0.1049 | LR: 0.005
[811/1429] Loss: 0.578 | LR: 0.005
[812/1429] Loss: 0.7337 | LR: 0.005
[813/1429] Loss: 0.778 | LR: 0.005
[814/1429] Loss: 0.8164 | LR: 0.005
[815/1429] Loss: 0.5866 | LR: 0.005
[816/1429] Loss: 0.3975 | LR: 0.005
[817/1429] Loss: 0.2355 | LR: 0.005
[818/1429] Loss: 0.3094 | LR: 0.005
[819/1429] Loss: 1.0354 | LR: 0.005
[820/1429] Loss: 0.662 | LR: 0.005
[821/1429] Loss: 0.9635 | LR: 0.005
[822/1429] Loss: 0.8264 | LR: 0.005
[823/1429] Loss: 0.3526 | LR: 0.005
[824/1429] Loss: 0.1063 | LR: 0.005
[825/1429] Loss: 0.9293 | LR: 0.005
[826/1429] Loss: 0.5758 | LR: 0.005
[827/1429] Loss: 0.4323 | LR: 0.005
[828/1429] Loss: 0.3646 | LR: 0.005
[829/1429] Loss: 0.184 | LR: 0.005
[830/1429] Loss: 0.1187 | LR: 0.005
[831/1429] Loss: 0.9829 | LR: 0.005
[832/1429] Loss: 0.1688 | LR: 0.005
[833/1429] Loss: 0.255 | LR: 0.005
[834/1429] Loss: 0.7493 | LR: 0.005
[835/1429] Loss: 0.6864 | LR: 0.005
[836/1429] Loss: 0.5916 | LR: 0.005
[837/1429] Loss: 0.4328 | LR: 0.005
[838/1429] Loss: 0.1317 | LR: 0.005
[839/1429] Loss: 0.7069 | LR: 0.005
[840/1429] Loss: 0.1146 | LR: 0.005
[841/1429] Loss: 0.6058 | LR: 0.005
[842/1429] Loss: 0.4691 | LR: 0.005
[843/1429] Loss: 0.46 | LR: 0.005
[844/1429] Loss: 0.4157 | LR: 0.005
[845/1429] Loss: 0.3224 | LR: 0.005
[846/1429] Loss: 0.719 | LR: 0.005
[847/1429] Loss: 1.0256 | LR: 0.005
[848/1429] Loss: 0.7278 | LR: 0.005
[849/1429] Loss: 0.3819 | LR: 0.005
[850/1429] Loss: 0.9493 | LR: 0.005
[851/1429] Loss: 0.6519 | LR: 0.005
[852/1429] Loss: 0.5158 | LR: 0.005
[853/1429] Loss: 0.7656 | LR: 0.005
[854/1429] Loss: 0.4156 | LR: 0.005
[855/1429] Loss: 0.8443 | LR: 0.005
[856/1429] Loss: 0.1362 | LR: 0.005
[857/1429] Loss: 0.8114 | LR: 0.005
[858/1429] Loss: 0.6738 | LR: 0.005
[859/1429] Loss: 0.8825 | LR: 0.005
[860/1429] Loss: 0.9954 | LR: 0.005
[861/1429] Loss: 0.6228 | LR: 0.005
[862/1429] Loss: 1.0159 | LR: 0.005
[863/1429] Loss: 0.5461 | LR: 0.005
[864/1429] Loss: 0.2951 | LR: 0.005
[865/1429] Loss: 0.4345 | LR: 0.005
[866/1429] Loss: 0.1805 | LR: 0.005
[867/1429] Loss: 0.5115 | LR: 0.005
[868/1429] Loss: 0.2568 | LR: 0.005
[869/1429] Loss: 0.6115 | LR: 0.005
[870/1429] Loss: 0.164 | LR: 0.005
[871/1429] Loss: 0.7667 | LR: 0.005
[872/1429] Loss: 0.842 | LR: 0.005
[873/1429] Loss: 0.6499 | LR: 0.005
[874/1429] Loss: 0.5831 | LR: 0.005
[875/1429] Loss: 1.0129 | LR: 0.005
[876/1429] Loss: 0.8385 | LR: 0.005
[877/1429] Loss: 0.5184 | LR: 0.005
[878/1429] Loss: 0.1695 | LR: 0.005
[879/1429] Loss: 0.2653 | LR: 0.005
[880/1429] Loss: 0.7641 | LR: 0.005
[881/1429] Loss: 0.8022 | LR: 0.005
[882/1429] Loss: 0.7372 | LR: 0.005
[883/1429] Loss: 0.3817 | LR: 0.005
[884/1429] Loss: 0.7727 | LR: 0.005
[885/1429] Loss: 0.3587 | LR: 0.005
[886/1429] Loss: 1.0661 | LR: 0.005
[887/1429] Loss: 0.6769 | LR: 0.005
[888/1429] Loss: 0.8158 | LR: 0.005
[889/1429] Loss: 0.5181 | LR: 0.005
[890/1429] Loss: 0.1407 | LR: 0.005
[891/1429] Loss: 0.9822 | LR: 0.005
[892/1429] Loss: 0.8828 | LR: 0.005
[893/1429] Loss: 0.7611 | LR: 0.005
[894/1429] Loss: 0.6957 | LR: 0.005
[895/1429] Loss: 1.0735 | LR: 0.005
[896/1429] Loss: 0.6986 | LR: 0.005
[897/1429] Loss: 0.8187 | LR: 0.005
[898/1429] Loss: 0.791 | LR: 0.005
[899/1429] Loss: 0.9663 | LR: 0.005
[900/1429] Loss: 0.2456 | LR: 0.005
[901/1429] Loss: 0.1951 | LR: 0.005
[902/1429] Loss: 0.1296 | LR: 0.005
[903/1429] Loss: 0.3724 | LR: 0.005
[904/1429] Loss: 0.8098 | LR: 0.005
[905/1429] Loss: 1.0926 | LR: 0.005
[906/1429] Loss: 0.3662 | LR: 0.005
[907/1429] Loss: 0.7437 | LR: 0.005
[908/1429] Loss: 1.0394 | LR: 0.005
[909/1429] Loss: 0.8584 | LR: 0.005
[910/1429] Loss: 0.1314 | LR: 0.005
[911/1429] Loss: 0.7839 | LR: 0.005
[912/1429] Loss: 0.5243 | LR: 0.005
[913/1429] Loss: 0.3333 | LR: 0.005
[914/1429] Loss: 0.1344 | LR: 0.005
[915/1429] Loss: 0.6213 | LR: 0.005
[916/1429] Loss: 0.2326 | LR: 0.005
[917/1429] Loss: 0.7654 | LR: 0.005
[918/1429] Loss: 0.2443 | LR: 0.005
[919/1429] Loss: 0.833 | LR: 0.005
[920/1429] Loss: 0.4763 | LR: 0.005
[921/1429] Loss: 0.449 | LR: 0.005
[922/1429] Loss: 0.4604 | LR: 0.005
[923/1429] Loss: 0.2912 | LR: 0.005
[924/1429] Loss: 0.539 | LR: 0.005
[925/1429] Loss: 0.3359 | LR: 0.005
[926/1429] Loss: 0.14 | LR: 0.005
[927/1429] Loss: 0.112 | LR: 0.005
[928/1429] Loss: 0.4217 | LR: 0.005
[929/1429] Loss: 0.7278 | LR: 0.005
[930/1429] Loss: 0.9669 | LR: 0.005
[931/1429] Loss: 0.1709 | LR: 0.005
[932/1429] Loss: 0.4156 | LR: 0.005
[933/1429] Loss: 0.8801 | LR: 0.005
[934/1429] Loss: 0.3154 | LR: 0.005
[935/1429] Loss: 0.2659 | LR: 0.005
[936/1429] Loss: 0.6546 | LR: 0.005
[937/1429] Loss: 0.5023 | LR: 0.005
[938/1429] Loss: 0.3737 | LR: 0.005
[939/1429] Loss: 0.3922 | LR: 0.005
[940/1429] Loss: 0.989 | LR: 0.005
[941/1429] Loss: 0.6634 | LR: 0.005
[942/1429] Loss: 0.3228 | LR: 0.005
[943/1429] Loss: 0.2362 | LR: 0.005
[944/1429] Loss: 0.9461 | LR: 0.005
[945/1429] Loss: 0.6925 | LR: 0.005
[946/1429] Loss: 0.7014 | LR: 0.005
[947/1429] Loss: 0.1557 | LR: 0.005
[948/1429] Loss: 0.3415 | LR: 0.005
[949/1429] Loss: 0.5349 | LR: 0.005
[950/1429] Loss: 0.9608 | LR: 0.005
[951/1429] Loss: 0.5104 | LR: 0.005
[952/1429] Loss: 0.3841 | LR: 0.005
[953/1429] Loss: 0.3651 | LR: 0.005
[954/1429] Loss: 0.1624 | LR: 0.005
[955/1429] Loss: 0.7347 | LR: 0.005
[956/1429] Loss: 0.2686 | LR: 0.005
[957/1429] Loss: 1.0543 | LR: 0.005
[958/1429] Loss: 0.887 | LR: 0.005
[959/1429] Loss: 0.5982 | LR: 0.005
[960/1429] Loss: 0.7962 | LR: 0.005
[961/1429] Loss: 1.062 | LR: 0.005
[962/1429] Loss: 0.4933 | LR: 0.005
[963/1429] Loss: 0.3952 | LR: 0.005
[964/1429] Loss: 0.8893 | LR: 0.005
[965/1429] Loss: 0.5563 | LR: 0.005
[966/1429] Loss: 0.5954 | LR: 0.005
[967/1429] Loss: 0.9022 | LR: 0.005
[968/1429] Loss: 0.3679 | LR: 0.005
[969/1429] Loss: 0.6783 | LR: 0.005
[970/1429] Loss: 0.7472 | LR: 0.005
[971/1429] Loss: 0.4668 | LR: 0.005
[972/1429] Loss: 0.5485 | LR: 0.005
[973/1429] Loss: 0.5694 | LR: 0.005
[974/1429] Loss: 0.4929 | LR: 0.005
[975/1429] Loss: 0.1136 | LR: 0.005
[976/1429] Loss: 0.5479 | LR: 0.005
[977/1429] Loss: 0.1211 | LR: 0.005
[978/1429] Loss: 0.6972 | LR: 0.005
[979/1429] Loss: 0.8368 | LR: 0.005
[980/1429] Loss: 0.7309 | LR: 0.005
[981/1429] Loss: 1.0512 | LR: 0.005
[982/1429] Loss: 0.8449 | LR: 0.005
[983/1429] Loss: 0.695 | LR: 0.005
[984/1429] Loss: 0.586 | LR: 0.005
[985/1429] Loss: 0.5516 | LR: 0.005
[986/1429] Loss: 0.6423 | LR: 0.005
[987/1429] Loss: 0.3393 | LR: 0.005
[988/1429] Loss: 0.8462 | LR: 0.005
[989/1429] Loss: 0.888 | LR: 0.005
[990/1429] Loss: 0.1195 | LR: 0.005
[991/1429] Loss: 0.2685 | LR: 0.005
[992/1429] Loss: 0.7769 | LR: 0.005
[993/1429] Loss: 0.497 | LR: 0.005
[994/1429] Loss: 1.0625 | LR: 0.005
[995/1429] Loss: 0.6113 | LR: 0.005
[996/1429] Loss: 0.4698 | LR: 0.005
[997/1429] Loss: 0.1147 | LR: 0.005
[998/1429] Loss: 0.8392 | LR: 0.005
[999/1429] Loss: 0.1132 | LR: 0.005
[1000/1429] Loss: 0.948 | LR: 0.005
[1001/1429] Loss: 0.8026 | LR: 0.005
[1002/1429] Loss: 0.6388 | LR: 0.005
[1003/1429] Loss: 0.8088 | LR: 0.005
[1004/1429] Loss: 0.5087 | LR: 0.005
[1005/1429] Loss: 0.9199 | LR: 0.005
[1006/1429] Loss: 0.7061 | LR: 0.005
[1007/1429] Loss: 0.5554 | LR: 0.005
[1008/1429] Loss: 0.9329 | LR: 0.005
[1009/1429] Loss: 0.8181 | LR: 0.005
[1010/1429] Loss: 0.5721 | LR: 0.005
[1011/1429] Loss: 1.085 | LR: 0.005
[1012/1429] Loss: 0.9967 | LR: 0.005
[1013/1429] Loss: 0.8341 | LR: 0.005
[1014/1429] Loss: 0.1822 | LR: 0.005
[1015/1429] Loss: 0.4029 | LR: 0.005
[1016/1429] Loss: 0.6107 | LR: 0.005
[1017/1429] Loss: 0.8861 | LR: 0.005
[1018/1429] Loss: 0.9386 | LR: 0.005
[1019/1429] Loss: 0.5888 | LR: 0.005
[1020/1429] Loss: 0.8969 | LR: 0.005
[1021/1429] Loss: 0.3698 | LR: 0.005
[1022/1429] Loss: 0.2458 | LR: 0.005
[1023/1429] Loss: 0.6681 | LR: 0.005
[1024/1429] Loss: 0.3912 | LR: 0.005
[1025/1429] Loss: 0.274 | LR: 0.005
[1026/1429] Loss: 0.8651 | LR: 0.005
[1027/1429] Loss: 0.1131 | LR: 0.005
[1028/1429] Loss: 0.9992 | LR: 0.005
[1029/1429] Loss: 0.5387 | LR: 0.005
[1030/1429] Loss: 0.4122 | LR: 0.005
[1031/1429] Loss: 0.3955 | LR: 0.005
[1032/1429] Loss: 0.7233 | LR: 0.005
[1033/1429] Loss: 1.0651 | LR: 0.005
[1034/1429] Loss: 0.3678 | LR: 0.005
[1035/1429] Loss: 0.1524 | LR: 0.005
[1036/1429] Loss: 0.3309 | LR: 0.005
[1037/1429] Loss: 0.2303 | LR: 0.005
[1038/1429] Loss: 0.3038 | LR: 0.005
[1039/1429] Loss: 0.5731 | LR: 0.005
[1040/1429] Loss: 0.4794 | LR: 0.005
[1041/1429] Loss: 0.375 | LR: 0.005
[1042/1429] Loss: 0.9464 | LR: 0.005
[1043/1429] Loss: 1.0595 | LR: 0.005
[1044/1429] Loss: 0.7625 | LR: 0.005
[1045/1429] Loss: 0.5751 | LR: 0.005
[1046/1429] Loss: 0.4619 | LR: 0.005
[1047/1429] Loss: 1.0261 | LR: 0.005
[1048/1429] Loss: 0.4401 | LR: 0.005
[1049/1429] Loss: 0.8233 | LR: 0.005
[1050/1429] Loss: 0.8517 | LR: 0.005
[1051/1429] Loss: 0.4149 | LR: 0.005
[1052/1429] Loss: 0.1224 | LR: 0.005
[1053/1429] Loss: 0.4199 | LR: 0.005
[1054/1429] Loss: 0.2709 | LR: 0.005
[1055/1429] Loss: 0.6119 | LR: 0.005
[1056/1429] Loss: 0.6673 | LR: 0.005
[1057/1429] Loss: 0.3358 | LR: 0.005
[1058/1429] Loss: 0.9731 | LR: 0.005
[1059/1429] Loss: 0.168 | LR: 0.005
[1060/1429] Loss: 0.5345 | LR: 0.005
[1061/1429] Loss: 0.3905 | LR: 0.005
[1062/1429] Loss: 0.7933 | LR: 0.005
[1063/1429] Loss: 0.8109 | LR: 0.005
[1064/1429] Loss: 0.4446 | LR: 0.005
[1065/1429] Loss: 0.5095 | LR: 0.005
[1066/1429] Loss: 0.4813 | LR: 0.005
[1067/1429] Loss: 0.8834 | LR: 0.005
[1068/1429] Loss: 0.6641 | LR: 0.005
[1069/1429] Loss: 0.4282 | LR: 0.005
[1070/1429] Loss: 0.5993 | LR: 0.005
[1071/1429] Loss: 0.2972 | LR: 0.005
[1072/1429] Loss: 0.2514 | LR: 0.005
[1073/1429] Loss: 0.9169 | LR: 0.005
[1074/1429] Loss: 0.8252 | LR: 0.005
[1075/1429] Loss: 0.7136 | LR: 0.005
[1076/1429] Loss: 0.385 | LR: 0.005
[1077/1429] Loss: 0.8327 | LR: 0.005
[1078/1429] Loss: 1.056 | LR: 0.005
[1079/1429] Loss: 0.8867 | LR: 0.005
[1080/1429] Loss: 1.0076 | LR: 0.005
[1081/1429] Loss: 0.4426 | LR: 0.005
[1082/1429] Loss: 1.0633 | LR: 0.005
[1083/1429] Loss: 0.9205 | LR: 0.005
[1084/1429] Loss: 0.9714 | LR: 0.005
[1085/1429] Loss: 0.1144 | LR: 0.005
[1086/1429] Loss: 0.2315 | LR: 0.005
[1087/1429] Loss: 0.1675 | LR: 0.005
[1088/1429] Loss: 1.0255 | LR: 0.005
[1089/1429] Loss: 0.7456 | LR: 0.005
[1090/1429] Loss: 0.4128 | LR: 0.005
[1091/1429] Loss: 0.8687 | LR: 0.005
[1092/1429] Loss: 0.8775 | LR: 0.005
[1093/1429] Loss: 0.716 | LR: 0.005
[1094/1429] Loss: 0.3709 | LR: 0.005
[1095/1429] Loss: 0.3531 | LR: 0.005
[1096/1429] Loss: 0.1703 | LR: 0.005
[1097/1429] Loss: 0.8098 | LR: 0.005
[1098/1429] Loss: 0.7679 | LR: 0.005
[1099/1429] Loss: 0.4714 | LR: 0.005
[1100/1429] Loss: 0.4985 | LR: 0.005
[1101/1429] Loss: 0.2579 | LR: 0.005
[1102/1429] Loss: 1.0097 | LR: 0.005
[1103/1429] Loss: 0.7648 | LR: 0.005
[1104/1429] Loss: 0.9981 | LR: 0.005
[1105/1429] Loss: 0.5127 | LR: 0.005
[1106/1429] Loss: 0.6083 | LR: 0.005
[1107/1429] Loss: 1.0025 | LR: 0.005
[1108/1429] Loss: 0.3117 | LR: 0.005
[1109/1429] Loss: 0.6549 | LR: 0.005
[1110/1429] Loss: 0.7953 | LR: 0.005
[1111/1429] Loss: 0.4102 | LR: 0.005
[1112/1429] Loss: 0.9506 | LR: 0.005
[1113/1429] Loss: 0.3007 | LR: 0.005
[1114/1429] Loss: 0.7177 | LR: 0.005
[1115/1429] Loss: 0.6142 | LR: 0.005
[1116/1429] Loss: 0.7792 | LR: 0.005
[1117/1429] Loss: 0.9868 | LR: 0.005
[1118/1429] Loss: 0.466 | LR: 0.005
[1119/1429] Loss: 0.8466 | LR: 0.005
[1120/1429] Loss: 0.9633 | LR: 0.005
[1121/1429] Loss: 0.3358 | LR: 0.005
[1122/1429] Loss: 0.3874 | LR: 0.005
[1123/1429] Loss: 0.1782 | LR: 0.005
[1124/1429] Loss: 0.7528 | LR: 0.005
[1125/1429] Loss: 0.6785 | LR: 0.005
[1126/1429] Loss: 0.2996 | LR: 0.005
[1127/1429] Loss: 0.4134 | LR: 0.005
[1128/1429] Loss: 0.8705 | LR: 0.005
[1129/1429] Loss: 0.6163 | LR: 0.005
[1130/1429] Loss: 0.1549 | LR: 0.005
[1131/1429] Loss: 0.4594 | LR: 0.005
[1132/1429] Loss: 0.5989 | LR: 0.005
[1133/1429] Loss: 1.0595 | LR: 0.005
[1134/1429] Loss: 0.8858 | LR: 0.005
[1135/1429] Loss: 0.4083 | LR: 0.005
[1136/1429] Loss: 0.8669 | LR: 0.005
[1137/1429] Loss: 0.877 | LR: 0.005
[1138/1429] Loss: 0.4237 | LR: 0.005
[1139/1429] Loss: 0.5445 | LR: 0.005
[1140/1429] Loss: 0.4714 | LR: 0.005
[1141/1429] Loss: 0.3699 | LR: 0.005
[1142/1429] Loss: 0.2383 | LR: 0.005
[1143/1429] Loss: 0.1345 | LR: 0.005
[1144/1429] Loss: 0.347 | LR: 0.005
[1145/1429] Loss: 0.9709 | LR: 0.005
[1146/1429] Loss: 0.148 | LR: 0.005
[1147/1429] Loss: 1.0847 | LR: 0.005
[1148/1429] Loss: 0.4998 | LR: 0.005
[1149/1429] Loss: 0.9752 | LR: 0.005
[1150/1429] Loss: 0.4047 | LR: 0.005
[1151/1429] Loss: 0.3191 | LR: 0.005
[1152/1429] Loss: 0.1798 | LR: 0.005
[1153/1429] Loss: 0.7682 | LR: 0.005
[1154/1429] Loss: 0.1533 | LR: 0.005
[1155/1429] Loss: 0.1721 | LR: 0.005
[1156/1429] Loss: 0.9578 | LR: 0.005
[1157/1429] Loss: 0.2742 | LR: 0.005
[1158/1429] Loss: 0.914 | LR: 0.005
[1159/1429] Loss: 0.6153 | LR: 0.005
[1160/1429] Loss: 0.613 | LR: 0.005
[1161/1429] Loss: 0.5057 | LR: 0.005
[1162/1429] Loss: 0.8969 | LR: 0.005
[1163/1429] Loss: 0.3978 | LR: 0.005
[1164/1429] Loss: 0.1211 | LR: 0.005
[1165/1429] Loss: 0.6971 | LR: 0.005
[1166/1429] Loss: 0.4282 | LR: 0.005
[1167/1429] Loss: 1.0183 | LR: 0.005
[1168/1429] Loss: 0.4644 | LR: 0.005
[1169/1429] Loss: 0.8853 | LR: 0.005
[1170/1429] Loss: 1.0831 | LR: 0.005
[1171/1429] Loss: 0.5229 | LR: 0.005
[1172/1429] Loss: 0.4019 | LR: 0.005
[1173/1429] Loss: 1.0217 | LR: 0.005
[1174/1429] Loss: 0.9007 | LR: 0.005
[1175/1429] Loss: 0.6521 | LR: 0.005
[1176/1429] Loss: 0.3634 | LR: 0.005
[1177/1429] Loss: 0.8946 | LR: 0.005
[1178/1429] Loss: 0.2439 | LR: 0.005
[1179/1429] Loss: 0.3686 | LR: 0.005
[1180/1429] Loss: 0.4789 | LR: 0.005
[1181/1429] Loss: 0.8086 | LR: 0.005
[1182/1429] Loss: 0.8617 | LR: 0.005
[1183/1429] Loss: 0.6767 | LR: 0.005
[1184/1429] Loss: 0.9198 | LR: 0.005
[1185/1429] Loss: 0.4391 | LR: 0.005
[1186/1429] Loss: 0.751 | LR: 0.005
[1187/1429] Loss: 0.1129 | LR: 0.005
[1188/1429] Loss: 1.0304 | LR: 0.005
[1189/1429] Loss: 0.5524 | LR: 0.005
[1190/1429] Loss: 0.4912 | LR: 0.005
[1191/1429] Loss: 0.3594 | LR: 0.005
[1192/1429] Loss: 0.9015 | LR: 0.005
[1193/1429] Loss: 0.3452 | LR: 0.005
[1194/1429] Loss: 0.4921 | LR: 0.005
[1195/1429] Loss: 0.8056 | LR: 0.005
[1196/1429] Loss: 0.4137 | LR: 0.005
[1197/1429] Loss: 0.7676 | LR: 0.005
[1198/1429] Loss: 0.3997 | LR: 0.005
[1199/1429] Loss: 0.8085 | LR: 0.005
[1200/1429] Loss: 0.881 | LR: 0.005
[1201/1429] Loss: 0.4852 | LR: 0.005
[1202/1429] Loss: 1.0201 | LR: 0.005
[1203/1429] Loss: 1.0383 | LR: 0.005
[1204/1429] Loss: 0.4675 | LR: 0.005
[1205/1429] Loss: 0.5177 | LR: 0.005
[1206/1429] Loss: 0.6348 | LR: 0.005
[1207/1429] Loss: 0.331 | LR: 0.005
[1208/1429] Loss: 0.6476 | LR: 0.005
[1209/1429] Loss: 1.0635 | LR: 0.005
[1210/1429] Loss: 0.3827 | LR: 0.005
[1211/1429] Loss: 0.3673 | LR: 0.005
[1212/1429] Loss: 0.3463 | LR: 0.005
[1213/1429] Loss: 0.9746 | LR: 0.005
[1214/1429] Loss: 0.8322 | LR: 0.005
[1215/1429] Loss: 0.6874 | LR: 0.005
[1216/1429] Loss: 0.9602 | LR: 0.005
[1217/1429] Loss: 0.7516 | LR: 0.005
[1218/1429] Loss: 0.8425 | LR: 0.005
[1219/1429] Loss: 0.5517 | LR: 0.005
[1220/1429] Loss: 0.2163 | LR: 0.005
[1221/1429] Loss: 0.3159 | LR: 0.005
[1222/1429] Loss: 0.986 | LR: 0.005
[1223/1429] Loss: 0.661 | LR: 0.005
[1224/1429] Loss: 0.4148 | LR: 0.005
[1225/1429] Loss: 0.2393 | LR: 0.005
[1226/1429] Loss: 0.2692 | LR: 0.005
[1227/1429] Loss: 0.8443 | LR: 0.005
[1228/1429] Loss: 0.7223 | LR: 0.005
[1229/1429] Loss: 0.1946 | LR: 0.005
[1230/1429] Loss: 1.0526 | LR: 0.005
[1231/1429] Loss: 0.3604 | LR: 0.005
[1232/1429] Loss: 0.2545 | LR: 0.005
[1233/1429] Loss: 0.1106 | LR: 0.005
[1234/1429] Loss: 0.5761 | LR: 0.005
[1235/1429] Loss: 0.3559 | LR: 0.005
[1236/1429] Loss: 0.1251 | LR: 0.005
[1237/1429] Loss: 0.7703 | LR: 0.005
[1238/1429] Loss: 0.3073 | LR: 0.005
[1239/1429] Loss: 0.2542 | LR: 0.005
[1240/1429] Loss: 0.5425 | LR: 0.005
[1241/1429] Loss: 0.4491 | LR: 0.005
[1242/1429] Loss: 0.2419 | LR: 0.005
[1243/1429] Loss: 0.2792 | LR: 0.005
[1244/1429] Loss: 0.473 | LR: 0.005
[1245/1429] Loss: 0.2837 | LR: 0.005
[1246/1429] Loss: 0.5625 | LR: 0.005
[1247/1429] Loss: 0.8657 | LR: 0.005
[1248/1429] Loss: 0.2192 | LR: 0.005
[1249/1429] Loss: 1.0293 | LR: 0.005
[1250/1429] Loss: 0.5054 | LR: 0.005
[1251/1429] Loss: 0.687 | LR: 0.005
[1252/1429] Loss: 0.328 | LR: 0.005
[1253/1429] Loss: 0.9026 | LR: 0.005
[1254/1429] Loss: 1.0132 | LR: 0.005
[1255/1429] Loss: 0.415 | LR: 0.005
[1256/1429] Loss: 0.2663 | LR: 0.005
[1257/1429] Loss: 0.3026 | LR: 0.005
[1258/1429] Loss: 0.7678 | LR: 0.005
[1259/1429] Loss: 0.4133 | LR: 0.005
[1260/1429] Loss: 0.6944 | LR: 0.005
[1261/1429] Loss: 0.9189 | LR: 0.005
[1262/1429] Loss: 0.7828 | LR: 0.005
[1263/1429] Loss: 0.1259 | LR: 0.005
[1264/1429] Loss: 0.2419 | LR: 0.005
[1265/1429] Loss: 0.2149 | LR: 0.005
[1266/1429] Loss: 0.4144 | LR: 0.005
[1267/1429] Loss: 0.9987 | LR: 0.005
[1268/1429] Loss: 0.1318 | LR: 0.005
[1269/1429] Loss: 0.3376 | LR: 0.005
[1270/1429] Loss: 0.1621 | LR: 0.005
[1271/1429] Loss: 0.9304 | LR: 0.005
[1272/1429] Loss: 0.4809 | LR: 0.005
[1273/1429] Loss: 0.1638 | LR: 0.005
[1274/1429] Loss: 0.6012 | LR: 0.005
[1275/1429] Loss: 0.523 | LR: 0.005
[1276/1429] Loss: 0.3491 | LR: 0.005
[1277/1429] Loss: 0.2471 | LR: 0.005
[1278/1429] Loss: 0.5353 | LR: 0.005
[1279/1429] Loss: 0.213 | LR: 0.005
[1280/1429] Loss: 1.0464 | LR: 0.005
[1281/1429] Loss: 0.1019 | LR: 0.005
[1282/1429] Loss: 0.9242 | LR: 0.005
[1283/1429] Loss: 0.779 | LR: 0.005
[1284/1429] Loss: 0.5295 | LR: 0.005
[1285/1429] Loss: 0.1278 | LR: 0.005
[1286/1429] Loss: 0.965 | LR: 0.005
[1287/1429] Loss: 0.8706 | LR: 0.005
[1288/1429] Loss: 0.7362 | LR: 0.005
[1289/1429] Loss: 0.6352 | LR: 0.005
[1290/1429] Loss: 0.852 | LR: 0.005
[1291/1429] Loss: 0.6701 | LR: 0.005
[1292/1429] Loss: 0.7099 | LR: 0.005
[1293/1429] Loss: 0.4676 | LR: 0.005
[1294/1429] Loss: 0.5609 | LR: 0.005
[1295/1429] Loss: 0.1033 | LR: 0.005
[1296/1429] Loss: 0.3923 | LR: 0.005
[1297/1429] Loss: 0.9278 | LR: 0.005
[1298/1429] Loss: 0.3868 | LR: 0.005
[1299/1429] Loss: 1.0722 | LR: 0.005
[1300/1429] Loss: 0.176 | LR: 0.005
[1301/1429] Loss: 0.6478 | LR: 0.005
[1302/1429] Loss: 1.0215 | LR: 0.005
[1303/1429] Loss: 0.1071 | LR: 0.005
[1304/1429] Loss: 0.7791 | LR: 0.005
[1305/1429] Loss: 0.5085 | LR: 0.005
[1306/1429] Loss: 0.2765 | LR: 0.005
[1307/1429] Loss: 0.8444 | LR: 0.005
[1308/1429] Loss: 0.6569 | LR: 0.005
[1309/1429] Loss: 0.2587 | LR: 0.005
[1310/1429] Loss: 0.9419 | LR: 0.005
[1311/1429] Loss: 1.0903 | LR: 0.005
[1312/1429] Loss: 0.7847 | LR: 0.005
[1313/1429] Loss: 0.7498 | LR: 0.005
[1314/1429] Loss: 0.5508 | LR: 0.005
[1315/1429] Loss: 0.5588 | LR: 0.005
[1316/1429] Loss: 0.1789 | LR: 0.005
[1317/1429] Loss: 1.0711 | LR: 0.005
[1318/1429] Loss: 0.8817 | LR: 0.005
[1319/1429] Loss: 0.4759 | LR: 0.005
[1320/1429] Loss: 0.6177 | LR: 0.005
[1321/1429] Loss: 0.5402 | LR: 0.005
[1322/1429] Loss: 1.0272 | LR: 0.005
[1323/1429] Loss: 0.2 | LR: 0.005
[1324/1429] Loss: 0.5009 | LR: 0.005
[1325/1429] Loss: 0.657 | LR: 0.005
[1326/1429] Loss: 0.9115 | LR: 0.005
[1327/1429] Loss: 0.977 | LR: 0.005
[1328/1429] Loss: 0.1121 | LR: 0.005
[1329/1429] Loss: 0.5473 | LR: 0.005
[1330/1429] Loss: 0.9553 | LR: 0.005
[1331/1429] Loss: 0.8376 | LR: 0.005
[1332/1429] Loss: 0.2675 | LR: 0.005
[1333/1429] Loss: 0.2169 | LR: 0.005
[1334/1429] Loss: 0.6667 | LR: 0.005
[1335/1429] Loss: 0.6445 | LR: 0.005
[1336/1429] Loss: 0.4023 | LR: 0.005
[1337/1429] Loss: 0.4018 | LR: 0.005
[1338/1429] Loss: 0.3257 | LR: 0.005
[1339/1429] Loss: 0.2374 | LR: 0.005
[1340/1429] Loss: 0.4291 | LR: 0.005
[1341/1429] Loss: 0.2327 | LR: 0.005
[1342/1429] Loss: 1.0099 | LR: 0.005
[1343/1429] Loss: 1.0187 | LR: 0.005
[1344/1429] Loss: 0.3259 | LR: 0.005
[1345/1429] Loss: 0.6952 | LR: 0.005
[1346/1429] Loss: 0.93 | LR: 0.005
[1347/1429] Loss: 0.1793 | LR: 0.005
[1348/1429] Loss: 0.9156 | LR: 0.005
[1349/1429] Loss: 0.5854 | LR: 0.005
[1350/1429] Loss: 0.5669 | LR: 0.005
[1351/1429] Loss: 0.7372 | LR: 0.005
[1352/1429] Loss: 0.6989 | LR: 0.005
[1353/1429] Loss: 1.0358 | LR: 0.005
[1354/1429] Loss: 0.7446 | LR: 0.005
[1355/1429] Loss: 0.251 | LR: 0.005
[1356/1429] Loss: 0.6931 | LR: 0.005
[1357/1429] Loss: 0.6673 | LR: 0.005
[1358/1429] Loss: 0.6253 | LR: 0.005
[1359/1429] Loss: 0.5023 | LR: 0.005
[1360/1429] Loss: 0.726 | LR: 0.005
[1361/1429] Loss: 0.3978 | LR: 0.005
[1362/1429] Loss: 0.4205 | LR: 0.005
[1363/1429] Loss: 0.8366 | LR: 0.005
[1364/1429] Loss: 0.1356 | LR: 0.005
[1365/1429] Loss: 0.949 | LR: 0.005
[1366/1429] Loss: 0.9503 | LR: 0.005
[1367/1429] Loss: 0.9208 | LR: 0.005
[1368/1429] Loss: 0.5137 | LR: 0.005
[1369/1429] Loss: 0.5187 | LR: 0.005
[1370/1429] Loss: 0.9008 | LR: 0.005
[1371/1429] Loss: 0.9867 | LR: 0.005
[1372/1429] Loss: 0.5241 | LR: 0.005
[1373/1429] Loss: 0.538 | LR: 0.005
[1374/1429] Loss: 0.3409 | LR: 0.005
[1375/1429] Loss: 0.3622 | LR: 0.005
[1376/1429] Loss: 0.8263 | LR: 0.005
[1377/1429] Loss: 0.4541 | LR: 0.005
[1378/1429] Loss: 0.9489 | LR: 0.005
[1379/1429] Loss: 0.2735 | LR: 0.005
[1380/1429] Loss: 0.9937 | LR: 0.005
[1381/1429] Loss: 0.524 | LR: 0.005
[1382/1429] Loss: 0.631 | LR: 0.005
[1383/1429] Loss: 1.0805 | LR: 0.005
[1384/1429] Loss: 0.7325 | LR: 0.005
[1385/1429] Loss: 0.3621 | LR: 0.005
[1386/1429] Loss: 0.955 | LR: 0.005
[1387/1429] Loss: 0.6902 | LR: 0.005
[1388/1429] Loss: 0.7533 | LR: 0.005
[1389/1429] Loss: 1.0777 | LR: 0.005
[1390/1429] Loss: 0.2163 | LR: 0.005
[1391/1429] Loss: 0.7652 | LR: 0.005
[1392/1429] Loss: 0.8125 | LR: 0.005
[1393/1429] Loss: 1.0447 | LR: 0.005
[1394/1429] Loss: 1.0792 | LR: 0.005
[1395/1429] Loss: 0.4745 | LR: 0.005
[1396/1429] Loss: 0.7712 | LR: 0.005
[1397/1429] Loss: 0.3218 | LR: 0.005
[1398/1429] Loss: 0.3694 | LR: 0.005
[1399/1429] Loss: 0.9645 | LR: 0.005
[1400/1429] Loss: 1.0834 | LR: 0.005
[1401/1429] Loss: 0.8343 | LR: 0.005
[1402/1429] Loss: 0.5143 | LR: 0.005
[1403/1429] Loss: 1.0528 | LR: 0.005
[1404/1429] Loss: 0.1437 | LR: 0.005
[1405/1429] Loss: 0.6451 | LR: 0.005
[1406/1429] Loss: 0.162 | LR: 0.005
[1407/1429] Loss: 0.279 | LR: 0.005
[1408/1429] Loss: 0.1716 | LR: 0.005
[1409/1429] Loss: 0.3722 | LR: 0.005
[1410/1429] Loss: 0.9649 | LR: 0.005
[1411/1429] Loss: 0.4971 | LR: 0.005
[1412/1429] Loss: 0.5147 | LR: 0.005
[1413/1429] Loss: 0.3824 | LR: 0.005
[1414/1429] Loss: 0.9015 | LR: 0.005
[1415/1429] Loss: 0.4219 | LR: 0.005
[1416/1429] Loss: 0.9883 | LR: 0.005
[1417/1429] Loss: 0.1431 | LR: 0.005
[1418/1429] Loss: 0.1249 | LR: 0.005
[1419/1429] Loss: 0.3225 | LR: 0.005
[1420/1429] Loss: 0.6818 | LR: 0.005
[1421/1429] Loss: 0.3177 | LR: 0.005
[1422/1429] Loss: 0.2362 | LR: 0.005
[1423/1429] Loss: 0.2893 | LR: 0.005
[1424/1429] Loss: 0.8124 | LR: 0.005
[1425/1429] Loss: 0.4158 | LR: 0.005
[1426/1429] Loss: 0.5739 | LR: 0.005
[1427/1429] Loss: 0.3588 | LR: 0.005
[1428/1429] Loss: 0.3576 | LR: 0.005
[1429/1429] Loss: 0.4929 | LR: 0.005
[1429/1429] Epoch Time: 11m 32s

Epoch 3/3
------------------------------------------------------------
[1/1429] Loss: 0.2953 | LR: 0.005
[2/1429] Loss: 0.5614 | LR: 0.005
[3/1429] Loss: 0.6134 | LR: 0.005
[4/1429] Loss: 0.5898 | LR: 0.005
[5/1429] Loss: 0.2003 | LR: 0.005
[6/1429] Loss: 0.6005 | LR: 0.005
[7/1429] Loss: 0.0902 | LR: 0.005
[8/1429] Loss: 0.5001 | LR: 0.005
[9/1429] Loss: 0.5905 | LR: 0.005
[10/1429] Loss: 0.5368 | LR: 0.005
[11/1429] Loss: 0.7895 | LR: 0.005
[12/1429] Loss: 0.4898 | LR: 0.005
[13/1429] Loss: 0.2281 | LR: 0.005
[14/1429] Loss: 0.6176 | LR: 0.005
[15/1429] Loss: 0.3589 | LR: 0.005
[16/1429] Loss: 0.1692 | LR: 0.005
[17/1429] Loss: 0.0588 | LR: 0.005
[18/1429] Loss: 0.0764 | LR: 0.005
[19/1429] Loss: 0.3399 | LR: 0.005
[20/1429] Loss: 0.0773 | LR: 0.005
[21/1429] Loss: 0.5423 | LR: 0.005
[22/1429] Loss: 0.5843 | LR: 0.005
[23/1429] Loss: 0.0625 | LR: 0.005
[24/1429] Loss: 0.7265 | LR: 0.005
[25/1429] Loss: 0.6693 | LR: 0.005
[26/1429] Loss: 0.2492 | LR: 0.005
[27/1429] Loss: 0.8246 | LR: 0.005
[28/1429] Loss: 0.8639 | LR: 0.005
[29/1429] Loss: 0.1601 | LR: 0.005
[30/1429] Loss: 0.4521 | LR: 0.005
[31/1429] Loss: 0.4584 | LR: 0.005
[32/1429] Loss: 0.1866 | LR: 0.005
[33/1429] Loss: 0.101 | LR: 0.005
[34/1429] Loss: 0.5335 | LR: 0.005
[35/1429] Loss: 0.2871 | LR: 0.005
[36/1429] Loss: 0.4338 | LR: 0.005
[37/1429] Loss: 0.8443 | LR: 0.005
[38/1429] Loss: 0.9665 | LR: 0.005
[39/1429] Loss: 0.1313 | LR: 0.005
[40/1429] Loss: 0.5406 | LR: 0.005
[41/1429] Loss: 0.8005 | LR: 0.005
[42/1429] Loss: 0.0903 | LR: 0.005
[43/1429] Loss: 0.9521 | LR: 0.005
[44/1429] Loss: 0.6902 | LR: 0.005
[45/1429] Loss: 0.3869 | LR: 0.005
[46/1429] Loss: 0.7575 | LR: 0.005
[47/1429] Loss: 0.4194 | LR: 0.005
[48/1429] Loss: 0.4169 | LR: 0.005
[49/1429] Loss: 0.6507 | LR: 0.005
[50/1429] Loss: 0.8656 | LR: 0.005
[51/1429] Loss: 0.5427 | LR: 0.005
[52/1429] Loss: 0.2447 | LR: 0.005
[53/1429] Loss: 0.5966 | LR: 0.005
[54/1429] Loss: 0.2767 | LR: 0.005
[55/1429] Loss: 0.3687 | LR: 0.005
[56/1429] Loss: 0.0634 | LR: 0.005
[57/1429] Loss: 0.8672 | LR: 0.005
[58/1429] Loss: 0.121 | LR: 0.005
[59/1429] Loss: 0.8926 | LR: 0.005
[60/1429] Loss: 0.5267 | LR: 0.005
[61/1429] Loss: 0.0412 | LR: 0.005
[62/1429] Loss: 0.6272 | LR: 0.005
[63/1429] Loss: 0.9632 | LR: 0.005
[64/1429] Loss: 0.1317 | LR: 0.005
[65/1429] Loss: 0.5415 | LR: 0.005
[66/1429] Loss: 0.0181 | LR: 0.005
[67/1429] Loss: 0.992 | LR: 0.005
[68/1429] Loss: 0.3041 | LR: 0.005
[69/1429] Loss: 0.1334 | LR: 0.005
[70/1429] Loss: 0.9806 | LR: 0.005
[71/1429] Loss: 0.3534 | LR: 0.005
[72/1429] Loss: 0.4606 | LR: 0.005
[73/1429] Loss: 0.66 | LR: 0.005
[74/1429] Loss: 0.9561 | LR: 0.005
[75/1429] Loss: 0.8789 | LR: 0.005
[76/1429] Loss: 0.609 | LR: 0.005
[77/1429] Loss: 0.3492 | LR: 0.005
[78/1429] Loss: 0.1302 | LR: 0.005
[79/1429] Loss: 0.6253 | LR: 0.005
[80/1429] Loss: 0.0552 | LR: 0.005
[81/1429] Loss: 0.719 | LR: 0.005
[82/1429] Loss: 0.6115 | LR: 0.005
[83/1429] Loss: 0.4533 | LR: 0.005
[84/1429] Loss: 0.7949 | LR: 0.005
[85/1429] Loss: 0.5916 | LR: 0.005
[86/1429] Loss: 0.7194 | LR: 0.005
[87/1429] Loss: 0.6746 | LR: 0.005
[88/1429] Loss: 0.0637 | LR: 0.005
[89/1429] Loss: 0.6713 | LR: 0.005
[90/1429] Loss: 0.8043 | LR: 0.005
[91/1429] Loss: 0.1985 | LR: 0.005
[92/1429] Loss: 0.6761 | LR: 0.005
[93/1429] Loss: 0.3078 | LR: 0.005
[94/1429] Loss: 0.9599 | LR: 0.005
[95/1429] Loss: 0.2721 | LR: 0.005
[96/1429] Loss: 0.9991 | LR: 0.005
[97/1429] Loss: 0.3163 | LR: 0.005
[98/1429] Loss: 0.969 | LR: 0.005
[99/1429] Loss: 0.3733 | LR: 0.005
[100/1429] Loss: 0.2853 | LR: 0.005
[101/1429] Loss: 0.6767 | LR: 0.005
[102/1429] Loss: 0.4584 | LR: 0.005
[103/1429] Loss: 0.4459 | LR: 0.005
[104/1429] Loss: 0.7168 | LR: 0.005
[105/1429] Loss: 0.4336 | LR: 0.005
[106/1429] Loss: 0.5043 | LR: 0.005
[107/1429] Loss: 0.5005 | LR: 0.005
[108/1429] Loss: 0.8601 | LR: 0.005
[109/1429] Loss: 0.7708 | LR: 0.005
[110/1429] Loss: 0.751 | LR: 0.005
[111/1429] Loss: 0.7332 | LR: 0.005
[112/1429] Loss: 0.2295 | LR: 0.005
[113/1429] Loss: 0.6323 | LR: 0.005
[114/1429] Loss: 0.6099 | LR: 0.005
[115/1429] Loss: 0.6359 | LR: 0.005
[116/1429] Loss: 0.185 | LR: 0.005
[117/1429] Loss: 0.4626 | LR: 0.005
[118/1429] Loss: 0.6184 | LR: 0.005
[119/1429] Loss: 0.7955 | LR: 0.005
[120/1429] Loss: 0.396 | LR: 0.005
[121/1429] Loss: 0.3306 | LR: 0.005
[122/1429] Loss: 0.052 | LR: 0.005
[123/1429] Loss: 0.9651 | LR: 0.005
[124/1429] Loss: 0.1576 | LR: 0.005
[125/1429] Loss: 0.4029 | LR: 0.005
[126/1429] Loss: 0.2854 | LR: 0.005
[127/1429] Loss: 0.0978 | LR: 0.005
[128/1429] Loss: 0.4647 | LR: 0.005
[129/1429] Loss: 0.4846 | LR: 0.005
[130/1429] Loss: 0.238 | LR: 0.005
[131/1429] Loss: 0.9983 | LR: 0.005
[132/1429] Loss: 0.1051 | LR: 0.005
[133/1429] Loss: 0.485 | LR: 0.005
[134/1429] Loss: 0.6514 | LR: 0.005
[135/1429] Loss: 0.6124 | LR: 0.005
[136/1429] Loss: 0.3375 | LR: 0.005
[137/1429] Loss: 0.8271 | LR: 0.005
[138/1429] Loss: 0.7051 | LR: 0.005
[139/1429] Loss: 0.7654 | LR: 0.005
[140/1429] Loss: 0.0156 | LR: 0.005
[141/1429] Loss: 0.7548 | LR: 0.005
[142/1429] Loss: 0.2563 | LR: 0.005
[143/1429] Loss: 0.7056 | LR: 0.005
[144/1429] Loss: 0.1754 | LR: 0.005
[145/1429] Loss: 0.0358 | LR: 0.005
[146/1429] Loss: 0.3002 | LR: 0.005
[147/1429] Loss: 0.1541 | LR: 0.005
[148/1429] Loss: 0.3331 | LR: 0.005
[149/1429] Loss: 0.1255 | LR: 0.005
[150/1429] Loss: 0.5457 | LR: 0.005
[151/1429] Loss: 0.2661 | LR: 0.005
[152/1429] Loss: 0.0162 | LR: 0.005
[153/1429] Loss: 0.1312 | LR: 0.005
[154/1429] Loss: 0.7366 | LR: 0.005
[155/1429] Loss: 0.3902 | LR: 0.005
[156/1429] Loss: 0.59 | LR: 0.005
[157/1429] Loss: 0.4308 | LR: 0.005
[158/1429] Loss: 0.0083 | LR: 0.005
[159/1429] Loss: 0.0911 | LR: 0.005
[160/1429] Loss: 0.2015 | LR: 0.005
[161/1429] Loss: 0.8076 | LR: 0.005
[162/1429] Loss: 0.3937 | LR: 0.005
[163/1429] Loss: 0.8611 | LR: 0.005
[164/1429] Loss: 0.9475 | LR: 0.005
[165/1429] Loss: 0.3643 | LR: 0.005
[166/1429] Loss: 0.0095 | LR: 0.005
[167/1429] Loss: 0.9072 | LR: 0.005
[168/1429] Loss: 0.7057 | LR: 0.005
[169/1429] Loss: 0.3007 | LR: 0.005
[170/1429] Loss: 0.8392 | LR: 0.005
[171/1429] Loss: 0.3476 | LR: 0.005
[172/1429] Loss: 0.8896 | LR: 0.005
[173/1429] Loss: 0.1916 | LR: 0.005
[174/1429] Loss: 0.0149 | LR: 0.005
[175/1429] Loss: 0.8341 | LR: 0.005
[176/1429] Loss: 0.1192 | LR: 0.005
[177/1429] Loss: 0.6941 | LR: 0.005
[178/1429] Loss: 0.422 | LR: 0.005
[179/1429] Loss: 0.6727 | LR: 0.005
[180/1429] Loss: 0.6529 | LR: 0.005
[181/1429] Loss: 0.821 | LR: 0.005
[182/1429] Loss: 0.5412 | LR: 0.005
[183/1429] Loss: 0.3566 | LR: 0.005
[184/1429] Loss: 0.5474 | LR: 0.005
[185/1429] Loss: 0.9066 | LR: 0.005
[186/1429] Loss: 0.5714 | LR: 0.005
[187/1429] Loss: 0.3126 | LR: 0.005
[188/1429] Loss: 0.6712 | LR: 0.005
[189/1429] Loss: 0.4958 | LR: 0.005
[190/1429] Loss: 0.7924 | LR: 0.005
[191/1429] Loss: 0.4055 | LR: 0.005
[192/1429] Loss: 0.8026 | LR: 0.005
[193/1429] Loss: 0.3662 | LR: 0.005
[194/1429] Loss: 0.7324 | LR: 0.005
[195/1429] Loss: 0.123 | LR: 0.005
[196/1429] Loss: 0.0063 | LR: 0.005
[197/1429] Loss: 0.6926 | LR: 0.005
[198/1429] Loss: 0.7217 | LR: 0.005
[199/1429] Loss: 0.6461 | LR: 0.005
[200/1429] Loss: 0.5568 | LR: 0.005
[201/1429] Loss: 0.6852 | LR: 0.005
[202/1429] Loss: 0.6053 | LR: 0.005
[203/1429] Loss: 0.7201 | LR: 0.005
[204/1429] Loss: 0.9151 | LR: 0.005
[205/1429] Loss: 0.5932 | LR: 0.005
[206/1429] Loss: 0.7761 | LR: 0.005
[207/1429] Loss: 0.7952 | LR: 0.005
[208/1429] Loss: 0.2524 | LR: 0.005
[209/1429] Loss: 0.1175 | LR: 0.005
[210/1429] Loss: 0.8517 | LR: 0.005
[211/1429] Loss: 0.925 | LR: 0.005
[212/1429] Loss: 0.9174 | LR: 0.005
[213/1429] Loss: 0.5984 | LR: 0.005
[214/1429] Loss: 0.7545 | LR: 0.005
[215/1429] Loss: 0.5323 | LR: 0.005
[216/1429] Loss: 0.9695 | LR: 0.005
[217/1429] Loss: 0.464 | LR: 0.005
[218/1429] Loss: 0.7804 | LR: 0.005
[219/1429] Loss: 0.6107 | LR: 0.005
[220/1429] Loss: 0.8687 | LR: 0.005
[221/1429] Loss: 0.1498 | LR: 0.005
[222/1429] Loss: 0.3987 | LR: 0.005
[223/1429] Loss: 0.5767 | LR: 0.005
[224/1429] Loss: 0.4494 | LR: 0.005
[225/1429] Loss: 0.3432 | LR: 0.005
[226/1429] Loss: 0.7722 | LR: 0.005
[227/1429] Loss: 0.821 | LR: 0.005
[228/1429] Loss: 0.6388 | LR: 0.005
[229/1429] Loss: 0.7977 | LR: 0.005
[230/1429] Loss: 0.2731 | LR: 0.005
[231/1429] Loss: 0.5521 | LR: 0.005
[232/1429] Loss: 0.8596 | LR: 0.005
[233/1429] Loss: 0.0426 | LR: 0.005
[234/1429] Loss: 0.4946 | LR: 0.005
[235/1429] Loss: 0.7179 | LR: 0.005
[236/1429] Loss: 0.9049 | LR: 0.005
[237/1429] Loss: 0.5157 | LR: 0.005
[238/1429] Loss: 0.08 | LR: 0.005
[239/1429] Loss: 0.9096 | LR: 0.005
[240/1429] Loss: 0.7472 | LR: 0.005
[241/1429] Loss: 0.3792 | LR: 0.005
[242/1429] Loss: 0.6184 | LR: 0.005
[243/1429] Loss: 0.0284 | LR: 0.005
[244/1429] Loss: 0.6159 | LR: 0.005
[245/1429] Loss: 0.0775 | LR: 0.005
[246/1429] Loss: 0.2017 | LR: 0.005
[247/1429] Loss: 0.066 | LR: 0.005
[248/1429] Loss: 0.5287 | LR: 0.005
[249/1429] Loss: 0.7129 | LR: 0.005
[250/1429] Loss: 0.7672 | LR: 0.005
[251/1429] Loss: 0.7204 | LR: 0.005
[252/1429] Loss: 0.3916 | LR: 0.005
[253/1429] Loss: 0.6531 | LR: 0.005
[254/1429] Loss: 0.3814 | LR: 0.005
[255/1429] Loss: 0.0254 | LR: 0.005
[256/1429] Loss: 0.9608 | LR: 0.005
[257/1429] Loss: 0.6234 | LR: 0.005
[258/1429] Loss: 0.4361 | LR: 0.005
[259/1429] Loss: 0.0181 | LR: 0.005
[260/1429] Loss: 0.3763 | LR: 0.005
[261/1429] Loss: 0.2111 | LR: 0.005
[262/1429] Loss: 0.9964 | LR: 0.005
[263/1429] Loss: 0.0358 | LR: 0.005
[264/1429] Loss: 0.0773 | LR: 0.005
[265/1429] Loss: 0.6986 | LR: 0.005
[266/1429] Loss: 0.1142 | LR: 0.005
[267/1429] Loss: 0.1352 | LR: 0.005
[268/1429] Loss: 0.0156 | LR: 0.005
[269/1429] Loss: 0.5677 | LR: 0.005
[270/1429] Loss: 0.8378 | LR: 0.005
[271/1429] Loss: 0.042 | LR: 0.005
[272/1429] Loss: 0.7921 | LR: 0.005
[273/1429] Loss: 0.1166 | LR: 0.005
[274/1429] Loss: 0.5573 | LR: 0.005
[275/1429] Loss: 0.6025 | LR: 0.005
[276/1429] Loss: 0.0476 | LR: 0.005
[277/1429] Loss: 0.5753 | LR: 0.005
[278/1429] Loss: 0.107 | LR: 0.005
[279/1429] Loss: 0.9466 | LR: 0.005
[280/1429] Loss: 0.3 | LR: 0.005
[281/1429] Loss: 0.2285 | LR: 0.005
[282/1429] Loss: 0.3572 | LR: 0.005
[283/1429] Loss: 0.833 | LR: 0.005
[284/1429] Loss: 0.7779 | LR: 0.005
[285/1429] Loss: 0.0733 | LR: 0.005
[286/1429] Loss: 0.974 | LR: 0.005
[287/1429] Loss: 0.1163 | LR: 0.005
[288/1429] Loss: 0.8486 | LR: 0.005
[289/1429] Loss: 0.9463 | LR: 0.005
[290/1429] Loss: 0.4443 | LR: 0.005
[291/1429] Loss: 0.2475 | LR: 0.005
[292/1429] Loss: 0.8183 | LR: 0.005
[293/1429] Loss: 0.3011 | LR: 0.005
[294/1429] Loss: 0.5683 | LR: 0.005
[295/1429] Loss: 0.7041 | LR: 0.005
[296/1429] Loss: 0.4195 | LR: 0.005
[297/1429] Loss: 0.4621 | LR: 0.005
[298/1429] Loss: 0.0716 | LR: 0.005
[299/1429] Loss: 0.9413 | LR: 0.005
[300/1429] Loss: 0.7663 | LR: 0.005
[301/1429] Loss: 0.4551 | LR: 0.005
[302/1429] Loss: 0.5911 | LR: 0.005
[303/1429] Loss: 0.5456 | LR: 0.005
[304/1429] Loss: 0.7903 | LR: 0.005
[305/1429] Loss: 0.4392 | LR: 0.005
[306/1429] Loss: 0.358 | LR: 0.005
[307/1429] Loss: 0.5062 | LR: 0.005
[308/1429] Loss: 0.6721 | LR: 0.005
[309/1429] Loss: 0.6478 | LR: 0.005
[310/1429] Loss: 0.9175 | LR: 0.005
[311/1429] Loss: 0.7872 | LR: 0.005
[312/1429] Loss: 0.6463 | LR: 0.005
[313/1429] Loss: 0.2249 | LR: 0.005
[314/1429] Loss: 0.8873 | LR: 0.005
[315/1429] Loss: 0.4268 | LR: 0.005
[316/1429] Loss: 0.1712 | LR: 0.005
[317/1429] Loss: 0.4062 | LR: 0.005
[318/1429] Loss: 0.1786 | LR: 0.005
[319/1429] Loss: 0.784 | LR: 0.005
[320/1429] Loss: 0.684 | LR: 0.005
[321/1429] Loss: 0.8188 | LR: 0.005
[322/1429] Loss: 0.2953 | LR: 0.005
[323/1429] Loss: 0.7597 | LR: 0.005
[324/1429] Loss: 0.9805 | LR: 0.005
[325/1429] Loss: 0.3622 | LR: 0.005
[326/1429] Loss: 0.8592 | LR: 0.005
[327/1429] Loss: 0.1677 | LR: 0.005
[328/1429] Loss: 0.2746 | LR: 0.005
[329/1429] Loss: 0.9762 | LR: 0.005
[330/1429] Loss: 0.5605 | LR: 0.005
[331/1429] Loss: 0.4114 | LR: 0.005
[332/1429] Loss: 0.0617 | LR: 0.005
[333/1429] Loss: 0.8224 | LR: 0.005
[334/1429] Loss: 0.5014 | LR: 0.005
[335/1429] Loss: 0.2959 | LR: 0.005
[336/1429] Loss: 0.0875 | LR: 0.005
[337/1429] Loss: 0.1605 | LR: 0.005
[338/1429] Loss: 0.4569 | LR: 0.005
[339/1429] Loss: 0.9991 | LR: 0.005
[340/1429] Loss: 0.437 | LR: 0.005
[341/1429] Loss: 0.7634 | LR: 0.005
[342/1429] Loss: 0.1812 | LR: 0.005
[343/1429] Loss: 0.3213 | LR: 0.005
[344/1429] Loss: 0.7804 | LR: 0.005
[345/1429] Loss: 0.8027 | LR: 0.005
[346/1429] Loss: 0.826 | LR: 0.005
[347/1429] Loss: 0.1797 | LR: 0.005
[348/1429] Loss: 0.0733 | LR: 0.005
[349/1429] Loss: 0.8735 | LR: 0.005
[350/1429] Loss: 0.0534 | LR: 0.005
[351/1429] Loss: 0.6308 | LR: 0.005
[352/1429] Loss: 0.1296 | LR: 0.005
[353/1429] Loss: 0.5672 | LR: 0.005
[354/1429] Loss: 0.8251 | LR: 0.005
[355/1429] Loss: 0.7328 | LR: 0.005
[356/1429] Loss: 0.5887 | LR: 0.005
[357/1429] Loss: 0.812 | LR: 0.005
[358/1429] Loss: 0.6092 | LR: 0.005
[359/1429] Loss: 0.1546 | LR: 0.005
[360/1429] Loss: 0.8349 | LR: 0.005
[361/1429] Loss: 0.2538 | LR: 0.005
[362/1429] Loss: 0.6542 | LR: 0.005
[363/1429] Loss: 0.9877 | LR: 0.005
[364/1429] Loss: 0.6615 | LR: 0.005
[365/1429] Loss: 0.2644 | LR: 0.005
[366/1429] Loss: 0.5106 | LR: 0.005
[367/1429] Loss: 0.9165 | LR: 0.005
[368/1429] Loss: 0.3048 | LR: 0.005
[369/1429] Loss: 0.5348 | LR: 0.005
[370/1429] Loss: 0.6005 | LR: 0.005
[371/1429] Loss: 0.7016 | LR: 0.005
[372/1429] Loss: 0.7671 | LR: 0.005
[373/1429] Loss: 0.9382 | LR: 0.005
[374/1429] Loss: 0.9005 | LR: 0.005
[375/1429] Loss: 0.6377 | LR: 0.005
[376/1429] Loss: 0.3046 | LR: 0.005
[377/1429] Loss: 0.68 | LR: 0.005
[378/1429] Loss: 0.7219 | LR: 0.005
[379/1429] Loss: 0.5714 | LR: 0.005
[380/1429] Loss: 0.8282 | LR: 0.005
[381/1429] Loss: 0.821 | LR: 0.005
[382/1429] Loss: 0.2676 | LR: 0.005
[383/1429] Loss: 0.8914 | LR: 0.005
[384/1429] Loss: 0.8391 | LR: 0.005
[385/1429] Loss: 0.7623 | LR: 0.005
[386/1429] Loss: 0.014 | LR: 0.005
[387/1429] Loss: 0.6388 | LR: 0.005
[388/1429] Loss: 0.3517 | LR: 0.005
[389/1429] Loss: 0.6177 | LR: 0.005
[390/1429] Loss: 0.0418 | LR: 0.005
[391/1429] Loss: 0.4758 | LR: 0.005
[392/1429] Loss: 0.0833 | LR: 0.005
[393/1429] Loss: 0.8599 | LR: 0.005
[394/1429] Loss: 0.083 | LR: 0.005
[395/1429] Loss: 0.9661 | LR: 0.005
[396/1429] Loss: 0.7443 | LR: 0.005
[397/1429] Loss: 0.3085 | LR: 0.005
[398/1429] Loss: 0.0326 | LR: 0.005
[399/1429] Loss: 0.7859 | LR: 0.005
[400/1429] Loss: 0.126 | LR: 0.005
[401/1429] Loss: 0.5907 | LR: 0.005
[402/1429] Loss: 0.7105 | LR: 0.005
[403/1429] Loss: 0.7809 | LR: 0.005
[404/1429] Loss: 0.8906 | LR: 0.005
[405/1429] Loss: 0.6789 | LR: 0.005
[406/1429] Loss: 0.2212 | LR: 0.005
[407/1429] Loss: 0.2864 | LR: 0.005
[408/1429] Loss: 0.5423 | LR: 0.005
[409/1429] Loss: 0.4704 | LR: 0.005
[410/1429] Loss: 0.9171 | LR: 0.005
[411/1429] Loss: 0.2962 | LR: 0.005
[412/1429] Loss: 0.6675 | LR: 0.005
[413/1429] Loss: 0.1928 | LR: 0.005
[414/1429] Loss: 0.9463 | LR: 0.005
[415/1429] Loss: 0.3934 | LR: 0.005
[416/1429] Loss: 0.5271 | LR: 0.005
[417/1429] Loss: 0.1663 | LR: 0.005
[418/1429] Loss: 0.2307 | LR: 0.005
[419/1429] Loss: 0.4402 | LR: 0.005
[420/1429] Loss: 0.4037 | LR: 0.005
[421/1429] Loss: 0.8634 | LR: 0.005
[422/1429] Loss: 0.3766 | LR: 0.005
[423/1429] Loss: 0.9531 | LR: 0.005
[424/1429] Loss: 0.4904 | LR: 0.005
[425/1429] Loss: 0.6264 | LR: 0.005
[426/1429] Loss: 0.0135 | LR: 0.005
[427/1429] Loss: 0.7689 | LR: 0.005
[428/1429] Loss: 0.53 | LR: 0.005
[429/1429] Loss: 0.2593 | LR: 0.005
[430/1429] Loss: 0.4226 | LR: 0.005
[431/1429] Loss: 0.4001 | LR: 0.005
[432/1429] Loss: 0.8204 | LR: 0.005
[433/1429] Loss: 0.1941 | LR: 0.005
[434/1429] Loss: 0.1066 | LR: 0.005
[435/1429] Loss: 0.2467 | LR: 0.005
[436/1429] Loss: 0.1385 | LR: 0.005
[437/1429] Loss: 0.7101 | LR: 0.005
[438/1429] Loss: 0.0838 | LR: 0.005
[439/1429] Loss: 0.4442 | LR: 0.005
[440/1429] Loss: 0.3485 | LR: 0.005
[441/1429] Loss: 0.681 | LR: 0.005
[442/1429] Loss: 0.8728 | LR: 0.005
[443/1429] Loss: 0.4639 | LR: 0.005
[444/1429] Loss: 0.8997 | LR: 0.005
[445/1429] Loss: 0.5783 | LR: 0.005
[446/1429] Loss: 0.6017 | LR: 0.005
[447/1429] Loss: 0.439 | LR: 0.005
[448/1429] Loss: 0.0686 | LR: 0.005
[449/1429] Loss: 0.7011 | LR: 0.005
[450/1429] Loss: 0.3968 | LR: 0.005
[451/1429] Loss: 0.2864 | LR: 0.005
[452/1429] Loss: 0.4189 | LR: 0.005
[453/1429] Loss: 0.4813 | LR: 0.005
[454/1429] Loss: 0.8548 | LR: 0.005
[455/1429] Loss: 0.8675 | LR: 0.005
[456/1429] Loss: 0.7177 | LR: 0.005
[457/1429] Loss: 0.41 | LR: 0.005
[458/1429] Loss: 0.576 | LR: 0.005
[459/1429] Loss: 0.9732 | LR: 0.005
[460/1429] Loss: 0.0514 | LR: 0.005
[461/1429] Loss: 0.9642 | LR: 0.005
[462/1429] Loss: 0.0719 | LR: 0.005
[463/1429] Loss: 0.2533 | LR: 0.005
[464/1429] Loss: 0.6019 | LR: 0.005
[465/1429] Loss: 0.2262 | LR: 0.005
[466/1429] Loss: 0.7127 | LR: 0.005
[467/1429] Loss: 0.4288 | LR: 0.005
[468/1429] Loss: 0.5979 | LR: 0.005
[469/1429] Loss: 0.0264 | LR: 0.005
[470/1429] Loss: 0.6201 | LR: 0.005
[471/1429] Loss: 0.9625 | LR: 0.005
[472/1429] Loss: 0.8848 | LR: 0.005
[473/1429] Loss: 0.5783 | LR: 0.005
[474/1429] Loss: 0.5381 | LR: 0.005
[475/1429] Loss: 0.8862 | LR: 0.005
[476/1429] Loss: 0.6798 | LR: 0.005
[477/1429] Loss: 0.8033 | LR: 0.005
[478/1429] Loss: 0.1615 | LR: 0.005
[479/1429] Loss: 0.0395 | LR: 0.005
[480/1429] Loss: 0.2792 | LR: 0.005
[481/1429] Loss: 0.6605 | LR: 0.005
[482/1429] Loss: 0.0025 | LR: 0.005
[483/1429] Loss: 0.7123 | LR: 0.005
[484/1429] Loss: 0.8766 | LR: 0.005
[485/1429] Loss: 0.1091 | LR: 0.005
[486/1429] Loss: 0.6451 | LR: 0.005
[487/1429] Loss: 0.5562 | LR: 0.005
[488/1429] Loss: 0.7104 | LR: 0.005
[489/1429] Loss: 0.3756 | LR: 0.005
[490/1429] Loss: 0.2888 | LR: 0.005
[491/1429] Loss: 0.5114 | LR: 0.005
[492/1429] Loss: 0.2877 | LR: 0.005
[493/1429] Loss: 0.0415 | LR: 0.005
[494/1429] Loss: 0.8899 | LR: 0.005
[495/1429] Loss: 0.1575 | LR: 0.005
[496/1429] Loss: 0.4929 | LR: 0.005
[497/1429] Loss: 0.6568 | LR: 0.005
[498/1429] Loss: 0.938 | LR: 0.005
[499/1429] Loss: 0.7259 | LR: 0.005
[500/1429] Loss: 0.6799 | LR: 0.005
[501/1429] Loss: 0.7841 | LR: 0.005
[502/1429] Loss: 0.0908 | LR: 0.005
[503/1429] Loss: 0.2139 | LR: 0.005
[504/1429] Loss: 0.5807 | LR: 0.005
[505/1429] Loss: 0.273 | LR: 0.005
[506/1429] Loss: 0.9261 | LR: 0.005
[507/1429] Loss: 0.9071 | LR: 0.005
[508/1429] Loss: 0.5506 | LR: 0.005
[509/1429] Loss: 0.0608 | LR: 0.005
[510/1429] Loss: 0.8918 | LR: 0.005
[511/1429] Loss: 0.2892 | LR: 0.005
[512/1429] Loss: 0.3239 | LR: 0.005
[513/1429] Loss: 0.2078 | LR: 0.005
[514/1429] Loss: 0.9254 | LR: 0.005
[515/1429] Loss: 0.56 | LR: 0.005
[516/1429] Loss: 0.3134 | LR: 0.005
[517/1429] Loss: 0.8687 | LR: 0.005
[518/1429] Loss: 0.0568 | LR: 0.005
[519/1429] Loss: 0.5917 | LR: 0.005
[520/1429] Loss: 0.0605 | LR: 0.005
[521/1429] Loss: 0.1321 | LR: 0.005
[522/1429] Loss: 0.1371 | LR: 0.005
[523/1429] Loss: 0.9011 | LR: 0.005
[524/1429] Loss: 0.5293 | LR: 0.005
[525/1429] Loss: 0.599 | LR: 0.005
[526/1429] Loss: 0.3733 | LR: 0.005
[527/1429] Loss: 0.8079 | LR: 0.005
[528/1429] Loss: 0.1842 | LR: 0.005
[529/1429] Loss: 0.4497 | LR: 0.005
[530/1429] Loss: 0.7346 | LR: 0.005
[531/1429] Loss: 0.818 | LR: 0.005
[532/1429] Loss: 0.8002 | LR: 0.005
[533/1429] Loss: 0.5645 | LR: 0.005
[534/1429] Loss: 0.0302 | LR: 0.005
[535/1429] Loss: 0.6196 | LR: 0.005
[536/1429] Loss: 0.0084 | LR: 0.005
[537/1429] Loss: 0.7498 | LR: 0.005
[538/1429] Loss: 0.4654 | LR: 0.005
[539/1429] Loss: 0.417 | LR: 0.005
[540/1429] Loss: 0.53 | LR: 0.005
[541/1429] Loss: 0.3258 | LR: 0.005
[542/1429] Loss: 0.0908 | LR: 0.005
[543/1429] Loss: 0.3388 | LR: 0.005
[544/1429] Loss: 0.2745 | LR: 0.005
[545/1429] Loss: 0.3586 | LR: 0.005
[546/1429] Loss: 0.1384 | LR: 0.005
[547/1429] Loss: 0.7427 | LR: 0.005
[548/1429] Loss: 0.4552 | LR: 0.005
[549/1429] Loss: 0.4846 | LR: 0.005
[550/1429] Loss: 0.1164 | LR: 0.005
[551/1429] Loss: 0.5314 | LR: 0.005
[552/1429] Loss: 0.4101 | LR: 0.005
[553/1429] Loss: 0.2044 | LR: 0.005
[554/1429] Loss: 0.4418 | LR: 0.005
[555/1429] Loss: 0.7205 | LR: 0.005
[556/1429] Loss: 0.5479 | LR: 0.005
[557/1429] Loss: 0.3043 | LR: 0.005
[558/1429] Loss: 0.8282 | LR: 0.005
[559/1429] Loss: 0.8541 | LR: 0.005
[560/1429] Loss: 0.9406 | LR: 0.005
[561/1429] Loss: 0.1293 | LR: 0.005
[562/1429] Loss: 0.2915 | LR: 0.005
[563/1429] Loss: 0.2878 | LR: 0.005
[564/1429] Loss: 0.2993 | LR: 0.005
[565/1429] Loss: 0.1791 | LR: 0.005
[566/1429] Loss: 0.5915 | LR: 0.005
[567/1429] Loss: 0.8925 | LR: 0.005
[568/1429] Loss: 0.9462 | LR: 0.005
[569/1429] Loss: 0.5747 | LR: 0.005
[570/1429] Loss: 0.1064 | LR: 0.005
[571/1429] Loss: 0.9006 | LR: 0.005
[572/1429] Loss: 0.9964 | LR: 0.005
[573/1429] Loss: 0.7046 | LR: 0.005
[574/1429] Loss: 0.754 | LR: 0.005
[575/1429] Loss: 0.1424 | LR: 0.005
[576/1429] Loss: 0.9329 | LR: 0.005
[577/1429] Loss: 0.1344 | LR: 0.005
[578/1429] Loss: 0.9509 | LR: 0.005
[579/1429] Loss: 0.32 | LR: 0.005
[580/1429] Loss: 0.6494 | LR: 0.005
[581/1429] Loss: 0.4633 | LR: 0.005
[582/1429] Loss: 0.4177 | LR: 0.005
[583/1429] Loss: 0.7347 | LR: 0.005
[584/1429] Loss: 0.1444 | LR: 0.005
[585/1429] Loss: 0.0818 | LR: 0.005
[586/1429] Loss: 0.935 | LR: 0.005
[587/1429] Loss: 0.6881 | LR: 0.005
[588/1429] Loss: 0.4011 | LR: 0.005
[589/1429] Loss: 0.6405 | LR: 0.005
[590/1429] Loss: 0.2661 | LR: 0.005
[591/1429] Loss: 0.2069 | LR: 0.005
[592/1429] Loss: 0.0619 | LR: 0.005
[593/1429] Loss: 0.0057 | LR: 0.005
[594/1429] Loss: 0.6419 | LR: 0.005
[595/1429] Loss: 0.9373 | LR: 0.005
[596/1429] Loss: 0.1697 | LR: 0.005
[597/1429] Loss: 0.2956 | LR: 0.005
[598/1429] Loss: 0.0469 | LR: 0.005
[599/1429] Loss: 0.7678 | LR: 0.005
[600/1429] Loss: 0.0444 | LR: 0.005
[601/1429] Loss: 0.0863 | LR: 0.005
[602/1429] Loss: 0.5939 | LR: 0.005
[603/1429] Loss: 0.1039 | LR: 0.005
[604/1429] Loss: 0.2101 | LR: 0.005
[605/1429] Loss: 0.7779 | LR: 0.005
[606/1429] Loss: 0.1862 | LR: 0.005
[607/1429] Loss: 0.3574 | LR: 0.005
[608/1429] Loss: 0.5585 | LR: 0.005
[609/1429] Loss: 0.8499 | LR: 0.005
[610/1429] Loss: 0.3739 | LR: 0.005
[611/1429] Loss: 0.8996 | LR: 0.005
[612/1429] Loss: 0.5095 | LR: 0.005
[613/1429] Loss: 0.8146 | LR: 0.005
[614/1429] Loss: 0.6088 | LR: 0.005
[615/1429] Loss: 0.9656 | LR: 0.005
[616/1429] Loss: 0.8878 | LR: 0.005
[617/1429] Loss: 0.6016 | LR: 0.005
[618/1429] Loss: 0.1044 | LR: 0.005
[619/1429] Loss: 0.6493 | LR: 0.005
[620/1429] Loss: 0.5952 | LR: 0.005
[621/1429] Loss: 0.0327 | LR: 0.005
[622/1429] Loss: 0.135 | LR: 0.005
[623/1429] Loss: 0.5875 | LR: 0.005
[624/1429] Loss: 0.9375 | LR: 0.005
[625/1429] Loss: 0.2784 | LR: 0.005
[626/1429] Loss: 0.88 | LR: 0.005
[627/1429] Loss: 0.5784 | LR: 0.005
[628/1429] Loss: 0.7604 | LR: 0.005
[629/1429] Loss: 0.1043 | LR: 0.005
[630/1429] Loss: 0.8834 | LR: 0.005
[631/1429] Loss: 0.5372 | LR: 0.005
[632/1429] Loss: 0.1963 | LR: 0.005
[633/1429] Loss: 0.7442 | LR: 0.005
[634/1429] Loss: 0.6919 | LR: 0.005
[635/1429] Loss: 0.3746 | LR: 0.005
[636/1429] Loss: 0.1384 | LR: 0.005
[637/1429] Loss: 0.6483 | LR: 0.005
[638/1429] Loss: 0.9592 | LR: 0.005
[639/1429] Loss: 0.3025 | LR: 0.005
[640/1429] Loss: 0.2674 | LR: 0.005
[641/1429] Loss: 0.4101 | LR: 0.005
[642/1429] Loss: 0.5376 | LR: 0.005
[643/1429] Loss: 0.9711 | LR: 0.005
[644/1429] Loss: 0.8636 | LR: 0.005
[645/1429] Loss: 0.8015 | LR: 0.005
[646/1429] Loss: 0.8328 | LR: 0.005
[647/1429] Loss: 0.9911 | LR: 0.005
[648/1429] Loss: 0.1456 | LR: 0.005
[649/1429] Loss: 0.3649 | LR: 0.005
[650/1429] Loss: 0.5371 | LR: 0.005
[651/1429] Loss: 0.0813 | LR: 0.005
[652/1429] Loss: 0.4354 | LR: 0.005
[653/1429] Loss: 0.7756 | LR: 0.005
[654/1429] Loss: 0.676 | LR: 0.005
[655/1429] Loss: 0.1789 | LR: 0.005
[656/1429] Loss: 0.8513 | LR: 0.005
[657/1429] Loss: 0.0317 | LR: 0.005
[658/1429] Loss: 0.5284 | LR: 0.005
[659/1429] Loss: 0.6419 | LR: 0.005
[660/1429] Loss: 0.0066 | LR: 0.005
[661/1429] Loss: 0.4488 | LR: 0.005
[662/1429] Loss: 0.6969 | LR: 0.005
[663/1429] Loss: 0.2326 | LR: 0.005
[664/1429] Loss: 0.7169 | LR: 0.005
[665/1429] Loss: 0.4578 | LR: 0.005
[666/1429] Loss: 0.5369 | LR: 0.005
[667/1429] Loss: 0.0441 | LR: 0.005
[668/1429] Loss: 0.4613 | LR: 0.005
[669/1429] Loss: 0.3281 | LR: 0.005
[670/1429] Loss: 0.7979 | LR: 0.005
[671/1429] Loss: 0.7051 | LR: 0.005
[672/1429] Loss: 0.3978 | LR: 0.005
[673/1429] Loss: 0.5686 | LR: 0.005
[674/1429] Loss: 0.9171 | LR: 0.005
[675/1429] Loss: 0.0282 | LR: 0.005
[676/1429] Loss: 0.4629 | LR: 0.005
[677/1429] Loss: 0.5222 | LR: 0.005
[678/1429] Loss: 0.4501 | LR: 0.005
[679/1429] Loss: 0.5041 | LR: 0.005
[680/1429] Loss: 0.3399 | LR: 0.005
[681/1429] Loss: 0.0497 | LR: 0.005
[682/1429] Loss: 0.8511 | LR: 0.005
[683/1429] Loss: 0.7906 | LR: 0.005
[684/1429] Loss: 0.2736 | LR: 0.005
[685/1429] Loss: 0.5226 | LR: 0.005
[686/1429] Loss: 0.4187 | LR: 0.005
[687/1429] Loss: 0.5521 | LR: 0.005
[688/1429] Loss: 0.7636 | LR: 0.005
[689/1429] Loss: 0.4758 | LR: 0.005
[690/1429] Loss: 0.9785 | LR: 0.005
[691/1429] Loss: 0.6815 | LR: 0.005
[692/1429] Loss: 0.6235 | LR: 0.005
[693/1429] Loss: 0.9091 | LR: 0.005
[694/1429] Loss: 0.7753 | LR: 0.005
[695/1429] Loss: 0.5652 | LR: 0.005
[696/1429] Loss: 0.1872 | LR: 0.005
[697/1429] Loss: 0.1806 | LR: 0.005
[698/1429] Loss: 0.4653 | LR: 0.005
[699/1429] Loss: 0.5604 | LR: 0.005
[700/1429] Loss: 0.1286 | LR: 0.005
[701/1429] Loss: 0.8252 | LR: 0.005
[702/1429] Loss: 0.6969 | LR: 0.005
[703/1429] Loss: 0.7132 | LR: 0.005
[704/1429] Loss: 0.0418 | LR: 0.005
[705/1429] Loss: 0.3389 | LR: 0.005
[706/1429] Loss: 0.3387 | LR: 0.005
[707/1429] Loss: 0.9087 | LR: 0.005
[708/1429] Loss: 0.71 | LR: 0.005
[709/1429] Loss: 0.4249 | LR: 0.005
[710/1429] Loss: 0.4055 | LR: 0.005
[711/1429] Loss: 0.8025 | LR: 0.005
[712/1429] Loss: 0.2759 | LR: 0.005
[713/1429] Loss: 0.8533 | LR: 0.005
[714/1429] Loss: 0.2904 | LR: 0.005
[715/1429] Loss: 0.3907 | LR: 0.005
[716/1429] Loss: 0.7459 | LR: 0.005
[717/1429] Loss: 0.4223 | LR: 0.005
[718/1429] Loss: 0.5206 | LR: 0.005
[719/1429] Loss: 0.7332 | LR: 0.005
[720/1429] Loss: 0.0711 | LR: 0.005
[721/1429] Loss: 0.9089 | LR: 0.005
[722/1429] Loss: 0.3103 | LR: 0.005
[723/1429] Loss: 0.3155 | LR: 0.005
[724/1429] Loss: 0.9774 | LR: 0.005
[725/1429] Loss: 0.6395 | LR: 0.005
[726/1429] Loss: 0.3163 | LR: 0.005
[727/1429] Loss: 0.0924 | LR: 0.005
[728/1429] Loss: 0.7456 | LR: 0.005
[729/1429] Loss: 0.1624 | LR: 0.005
[730/1429] Loss: 0.4206 | LR: 0.005
[731/1429] Loss: 0.2996 | LR: 0.005
[732/1429] Loss: 0.8908 | LR: 0.005
[733/1429] Loss: 0.4257 | LR: 0.005
[734/1429] Loss: 0.175 | LR: 0.005
[735/1429] Loss: 0.6724 | LR: 0.005
[736/1429] Loss: 0.0584 | LR: 0.005
[737/1429] Loss: 0.7525 | LR: 0.005
[738/1429] Loss: 0.567 | LR: 0.005
[739/1429] Loss: 0.9304 | LR: 0.005
[740/1429] Loss: 0.3613 | LR: 0.005
[741/1429] Loss: 0.3144 | LR: 0.005
[742/1429] Loss: 0.0056 | LR: 0.005
[743/1429] Loss: 0.2451 | LR: 0.005
[744/1429] Loss: 0.08 | LR: 0.005
[745/1429] Loss: 0.5643 | LR: 0.005
[746/1429] Loss: 0.2343 | LR: 0.005
[747/1429] Loss: 0.6032 | LR: 0.005
[748/1429] Loss: 0.6284 | LR: 0.005
[749/1429] Loss: 0.6449 | LR: 0.005
[750/1429] Loss: 0.5709 | LR: 0.005
[751/1429] Loss: 0.0018 | LR: 0.005
[752/1429] Loss: 0.2152 | LR: 0.005
[753/1429] Loss: 0.0368 | LR: 0.005
[754/1429] Loss: 0.3154 | LR: 0.005
[755/1429] Loss: 0.8049 | LR: 0.005
[756/1429] Loss: 0.7122 | LR: 0.005
[757/1429] Loss: 0.0384 | LR: 0.005
[758/1429] Loss: 0.04 | LR: 0.005
[759/1429] Loss: 0.8559 | LR: 0.005
[760/1429] Loss: 0.1062 | LR: 0.005
[761/1429] Loss: 0.3179 | LR: 0.005
[762/1429] Loss: 0.1883 | LR: 0.005
[763/1429] Loss: 0.4256 | LR: 0.005
[764/1429] Loss: 0.2741 | LR: 0.005
[765/1429] Loss: 0.5016 | LR: 0.005
[766/1429] Loss: 0.5825 | LR: 0.005
[767/1429] Loss: 0.8492 | LR: 0.005
[768/1429] Loss: 0.7808 | LR: 0.005
[769/1429] Loss: 0.1735 | LR: 0.005
[770/1429] Loss: 0.604 | LR: 0.005
[771/1429] Loss: 0.1088 | LR: 0.005
[772/1429] Loss: 0.7338 | LR: 0.005
[773/1429] Loss: 0.813 | LR: 0.005
[774/1429] Loss: 0.9131 | LR: 0.005
[775/1429] Loss: 0.8888 | LR: 0.005
[776/1429] Loss: 0.1466 | LR: 0.005
[777/1429] Loss: 0.2032 | LR: 0.005
[778/1429] Loss: 0.9645 | LR: 0.005
[779/1429] Loss: 0.408 | LR: 0.005
[780/1429] Loss: 0.992 | LR: 0.005
[781/1429] Loss: 0.3796 | LR: 0.005
[782/1429] Loss: 0.4783 | LR: 0.005
[783/1429] Loss: 0.5006 | LR: 0.005
[784/1429] Loss: 0.2801 | LR: 0.005
[785/1429] Loss: 0.5435 | LR: 0.005
[786/1429] Loss: 0.3542 | LR: 0.005
[787/1429] Loss: 0.7925 | LR: 0.005
[788/1429] Loss: 0.7266 | LR: 0.005
[789/1429] Loss: 0.7 | LR: 0.005
[790/1429] Loss: 0.9803 | LR: 0.005
[791/1429] Loss: 0.3013 | LR: 0.005
[792/1429] Loss: 0.943 | LR: 0.005
[793/1429] Loss: 0.3665 | LR: 0.005
[794/1429] Loss: 0.6607 | LR: 0.005
[795/1429] Loss: 0.6514 | LR: 0.005
[796/1429] Loss: 0.948 | LR: 0.005
[797/1429] Loss: 0.2987 | LR: 0.005
[798/1429] Loss: 0.7628 | LR: 0.005
[799/1429] Loss: 0.8544 | LR: 0.005
[800/1429] Loss: 0.8728 | LR: 0.005
[801/1429] Loss: 0.1686 | LR: 0.005
[802/1429] Loss: 0.0232 | LR: 0.005
[803/1429] Loss: 0.1121 | LR: 0.005
[804/1429] Loss: 0.4546 | LR: 0.005
[805/1429] Loss: 0.6723 | LR: 0.005
[806/1429] Loss: 0.5336 | LR: 0.005
[807/1429] Loss: 0.6399 | LR: 0.005
[808/1429] Loss: 0.5167 | LR: 0.005
[809/1429] Loss: 0.9361 | LR: 0.005
[810/1429] Loss: 0.4401 | LR: 0.005
[811/1429] Loss: 0.6549 | LR: 0.005
[812/1429] Loss: 0.2517 | LR: 0.005
[813/1429] Loss: 0.4572 | LR: 0.005
[814/1429] Loss: 0.5222 | LR: 0.005
[815/1429] Loss: 0.0694 | LR: 0.005
[816/1429] Loss: 0.3891 | LR: 0.005
[817/1429] Loss: 0.5422 | LR: 0.005
[818/1429] Loss: 0.0205 | LR: 0.005
[819/1429] Loss: 0.7846 | LR: 0.005
[820/1429] Loss: 0.0256 | LR: 0.005
[821/1429] Loss: 0.2836 | LR: 0.005
[822/1429] Loss: 0.0156 | LR: 0.005
[823/1429] Loss: 0.0235 | LR: 0.005
[824/1429] Loss: 0.474 | LR: 0.005
[825/1429] Loss: 0.6822 | LR: 0.005
[826/1429] Loss: 0.8677 | LR: 0.005
[827/1429] Loss: 0.8106 | LR: 0.005
[828/1429] Loss: 0.0043 | LR: 0.005
[829/1429] Loss: 0.048 | LR: 0.005
[830/1429] Loss: 0.2455 | LR: 0.005
[831/1429] Loss: 0.7904 | LR: 0.005
[832/1429] Loss: 0.6609 | LR: 0.005
[833/1429] Loss: 0.304 | LR: 0.005
[834/1429] Loss: 0.8726 | LR: 0.005
[835/1429] Loss: 0.6995 | LR: 0.005
[836/1429] Loss: 0.0057 | LR: 0.005
[837/1429] Loss: 0.0453 | LR: 0.005
[838/1429] Loss: 0.8298 | LR: 0.005
[839/1429] Loss: 0.3255 | LR: 0.005
[840/1429] Loss: 0.7462 | LR: 0.005
[841/1429] Loss: 0.2702 | LR: 0.005
[842/1429] Loss: 0.8205 | LR: 0.005
[843/1429] Loss: 0.058 | LR: 0.005
[844/1429] Loss: 0.5232 | LR: 0.005
[845/1429] Loss: 0.1564 | LR: 0.005
[846/1429] Loss: 0.6346 | LR: 0.005
[847/1429] Loss: 0.9099 | LR: 0.005
[848/1429] Loss: 0.5992 | LR: 0.005
[849/1429] Loss: 0.3198 | LR: 0.005
[850/1429] Loss: 0.5871 | LR: 0.005
[851/1429] Loss: 0.331 | LR: 0.005
[852/1429] Loss: 0.9179 | LR: 0.005
[853/1429] Loss: 0.8834 | LR: 0.005
[854/1429] Loss: 0.8248 | LR: 0.005
[855/1429] Loss: 0.4339 | LR: 0.005
[856/1429] Loss: 0.2273 | LR: 0.005
[857/1429] Loss: 0.0278 | LR: 0.005
[858/1429] Loss: 0.3046 | LR: 0.005
[859/1429] Loss: 0.1014 | LR: 0.005
[860/1429] Loss: 0.3551 | LR: 0.005
[861/1429] Loss: 0.2063 | LR: 0.005
[862/1429] Loss: 0.0855 | LR: 0.005
[863/1429] Loss: 0.7117 | LR: 0.005
[864/1429] Loss: 0.4494 | LR: 0.005
[865/1429] Loss: 0.5969 | LR: 0.005
[866/1429] Loss: 0.8723 | LR: 0.005
[867/1429] Loss: 0.9189 | LR: 0.005
[868/1429] Loss: 0.4931 | LR: 0.005
[869/1429] Loss: 0.279 | LR: 0.005
[870/1429] Loss: 0.7851 | LR: 0.005
[871/1429] Loss: 0.1088 | LR: 0.005
[872/1429] Loss: 0.7323 | LR: 0.005
[873/1429] Loss: 0.6164 | LR: 0.005
[874/1429] Loss: 0.5833 | LR: 0.005
[875/1429] Loss: 0.2218 | LR: 0.005
[876/1429] Loss: 0.7175 | LR: 0.005
[877/1429] Loss: 0.9449 | LR: 0.005
[878/1429] Loss: 0.9447 | LR: 0.005
[879/1429] Loss: 0.2227 | LR: 0.005
[880/1429] Loss: 0.3873 | LR: 0.005
[881/1429] Loss: 0.3526 | LR: 0.005
[882/1429] Loss: 0.6918 | LR: 0.005
[883/1429] Loss: 0.6992 | LR: 0.005
[884/1429] Loss: 0.1041 | LR: 0.005
[885/1429] Loss: 0.9453 | LR: 0.005
[886/1429] Loss: 0.046 | LR: 0.005
[887/1429] Loss: 0.4525 | LR: 0.005
[888/1429] Loss: 0.5751 | LR: 0.005
[889/1429] Loss: 0.5428 | LR: 0.005
[890/1429] Loss: 0.7445 | LR: 0.005
[891/1429] Loss: 0.6742 | LR: 0.005
[892/1429] Loss: 0.2913 | LR: 0.005
[893/1429] Loss: 0.2129 | LR: 0.005
[894/1429] Loss: 0.3142 | LR: 0.005
[895/1429] Loss: 0.0446 | LR: 0.005
[896/1429] Loss: 0.8852 | LR: 0.005
[897/1429] Loss: 0.4147 | LR: 0.005
[898/1429] Loss: 0.9537 | LR: 0.005
[899/1429] Loss: 0.018 | LR: 0.005
[900/1429] Loss: 0.7359 | LR: 0.005
[901/1429] Loss: 0.0943 | LR: 0.005
[902/1429] Loss: 0.3215 | LR: 0.005
[903/1429] Loss: 0.03 | LR: 0.005
[904/1429] Loss: 0.2293 | LR: 0.005
[905/1429] Loss: 0.6677 | LR: 0.005
[906/1429] Loss: 0.7856 | LR: 0.005
[907/1429] Loss: 0.384 | LR: 0.005
[908/1429] Loss: 0.2101 | LR: 0.005
[909/1429] Loss: 0.6205 | LR: 0.005
[910/1429] Loss: 0.1493 | LR: 0.005
[911/1429] Loss: 0.8942 | LR: 0.005
[912/1429] Loss: 0.3727 | LR: 0.005
[913/1429] Loss: 0.7932 | LR: 0.005
[914/1429] Loss: 0.7069 | LR: 0.005
[915/1429] Loss: 0.0391 | LR: 0.005
[916/1429] Loss: 0.1299 | LR: 0.005
[917/1429] Loss: 0.4026 | LR: 0.005
[918/1429] Loss: 0.2517 | LR: 0.005
[919/1429] Loss: 0.955 | LR: 0.005
[920/1429] Loss: 0.0553 | LR: 0.005
[921/1429] Loss: 0.3653 | LR: 0.005
[922/1429] Loss: 0.1424 | LR: 0.005
[923/1429] Loss: 0.4273 | LR: 0.005
[924/1429] Loss: 0.8269 | LR: 0.005
[925/1429] Loss: 0.5828 | LR: 0.005
[926/1429] Loss: 0.0546 | LR: 0.005
[927/1429] Loss: 0.7929 | LR: 0.005
[928/1429] Loss: 0.8872 | LR: 0.005
[929/1429] Loss: 0.3434 | LR: 0.005
[930/1429] Loss: 0.7921 | LR: 0.005
[931/1429] Loss: 0.1584 | LR: 0.005
[932/1429] Loss: 0.8473 | LR: 0.005
[933/1429] Loss: 0.1251 | LR: 0.005
[934/1429] Loss: 0.2143 | LR: 0.005
[935/1429] Loss: 0.5828 | LR: 0.005
[936/1429] Loss: 0.2987 | LR: 0.005
[937/1429] Loss: 0.4152 | LR: 0.005
[938/1429] Loss: 0.9953 | LR: 0.005
[939/1429] Loss: 0.8852 | LR: 0.005
[940/1429] Loss: 0.3742 | LR: 0.005
[941/1429] Loss: 0.7714 | LR: 0.005
[942/1429] Loss: 0.9159 | LR: 0.005
[943/1429] Loss: 0.9807 | LR: 0.005
[944/1429] Loss: 0.8617 | LR: 0.005
[945/1429] Loss: 0.4163 | LR: 0.005
[946/1429] Loss: 0.628 | LR: 0.005
[947/1429] Loss: 0.0652 | LR: 0.005
[948/1429] Loss: 0.1649 | LR: 0.005
[949/1429] Loss: 0.3261 | LR: 0.005
[950/1429] Loss: 0.1487 | LR: 0.005
[951/1429] Loss: 0.5085 | LR: 0.005
[952/1429] Loss: 0.7047 | LR: 0.005
[953/1429] Loss: 0.3382 | LR: 0.005
[954/1429] Loss: 0.983 | LR: 0.005
[955/1429] Loss: 0.7528 | LR: 0.005
[956/1429] Loss: 0.761 | LR: 0.005
[957/1429] Loss: 0.7927 | LR: 0.005
[958/1429] Loss: 0.1718 | LR: 0.005
[959/1429] Loss: 0.8035 | LR: 0.005
[960/1429] Loss: 0.1253 | LR: 0.005
[961/1429] Loss: 0.4067 | LR: 0.005
[962/1429] Loss: 0.9975 | LR: 0.005
[963/1429] Loss: 0.6162 | LR: 0.005
[964/1429] Loss: 0.6921 | LR: 0.005
[965/1429] Loss: 0.1827 | LR: 0.005
[966/1429] Loss: 0.1665 | LR: 0.005
[967/1429] Loss: 0.1486 | LR: 0.005
[968/1429] Loss: 0.6231 | LR: 0.005
[969/1429] Loss: 0.5895 | LR: 0.005
[970/1429] Loss: 0.9982 | LR: 0.005
[971/1429] Loss: 0.5058 | LR: 0.005
[972/1429] Loss: 0.6743 | LR: 0.005
[973/1429] Loss: 0.7333 | LR: 0.005
[974/1429] Loss: 0.1758 | LR: 0.005
[975/1429] Loss: 0.0729 | LR: 0.005
[976/1429] Loss: 0.3716 | LR: 0.005
[977/1429] Loss: 0.1792 | LR: 0.005
[978/1429] Loss: 0.1067 | LR: 0.005
[979/1429] Loss: 0.6759 | LR: 0.005
[980/1429] Loss: 0.9056 | LR: 0.005
[981/1429] Loss: 0.7046 | LR: 0.005
[982/1429] Loss: 0.9281 | LR: 0.005
[983/1429] Loss: 0.7 | LR: 0.005
[984/1429] Loss: 0.7851 | LR: 0.005
[985/1429] Loss: 0.6538 | LR: 0.005
[986/1429] Loss: 0.6318 | LR: 0.005
[987/1429] Loss: 0.7051 | LR: 0.005
[988/1429] Loss: 0.5193 | LR: 0.005
[989/1429] Loss: 0.8998 | LR: 0.005
[990/1429] Loss: 0.8555 | LR: 0.005
[991/1429] Loss: 0.2686 | LR: 0.005
[992/1429] Loss: 0.5085 | LR: 0.005
[993/1429] Loss: 0.816 | LR: 0.005
[994/1429] Loss: 0.4009 | LR: 0.005
[995/1429] Loss: 0.6881 | LR: 0.005
[996/1429] Loss: 0.4045 | LR: 0.005
[997/1429] Loss: 0.0562 | LR: 0.005
[998/1429] Loss: 0.7906 | LR: 0.005
[999/1429] Loss: 0.4354 | LR: 0.005
[1000/1429] Loss: 0.5887 | LR: 0.005
[1001/1429] Loss: 0.5468 | LR: 0.005
[1002/1429] Loss: 0.7982 | LR: 0.005
[1003/1429] Loss: 0.7717 | LR: 0.005
[1004/1429] Loss: 0.7318 | LR: 0.005
[1005/1429] Loss: 0.4101 | LR: 0.005
[1006/1429] Loss: 0.3382 | LR: 0.005
[1007/1429] Loss: 0.6994 | LR: 0.005
[1008/1429] Loss: 0.0737 | LR: 0.005
[1009/1429] Loss: 0.0235 | LR: 0.005
[1010/1429] Loss: 0.7021 | LR: 0.005
[1011/1429] Loss: 0.6091 | LR: 0.005
[1012/1429] Loss: 0.4195 | LR: 0.005
[1013/1429] Loss: 0.5703 | LR: 0.005
[1014/1429] Loss: 0.187 | LR: 0.005
[1015/1429] Loss: 0.3388 | LR: 0.005
[1016/1429] Loss: 0.4155 | LR: 0.005
[1017/1429] Loss: 0.6446 | LR: 0.005
[1018/1429] Loss: 0.0384 | LR: 0.005
[1019/1429] Loss: 0.2878 | LR: 0.005
[1020/1429] Loss: 0.1422 | LR: 0.005
[1021/1429] Loss: 0.9467 | LR: 0.005
[1022/1429] Loss: 0.1738 | LR: 0.005
[1023/1429] Loss: 0.9122 | LR: 0.005
[1024/1429] Loss: 0.2678 | LR: 0.005
[1025/1429] Loss: 0.0907 | LR: 0.005
[1026/1429] Loss: 0.5375 | LR: 0.005
[1027/1429] Loss: 0.5399 | LR: 0.005
[1028/1429] Loss: 0.8194 | LR: 0.005
[1029/1429] Loss: 0.323 | LR: 0.005
[1030/1429] Loss: 0.6568 | LR: 0.005
[1031/1429] Loss: 0.1325 | LR: 0.005
[1032/1429] Loss: 0.5188 | LR: 0.005
[1033/1429] Loss: 0.3241 | LR: 0.005
[1034/1429] Loss: 0.1813 | LR: 0.005
[1035/1429] Loss: 0.7143 | LR: 0.005
[1036/1429] Loss: 0.7968 | LR: 0.005
[1037/1429] Loss: 0.613 | LR: 0.005
[1038/1429] Loss: 0.1169 | LR: 0.005
[1039/1429] Loss: 0.563 | LR: 0.005
[1040/1429] Loss: 0.0383 | LR: 0.005
[1041/1429] Loss: 0.4517 | LR: 0.005
[1042/1429] Loss: 0.2863 | LR: 0.005
[1043/1429] Loss: 0.4414 | LR: 0.005
[1044/1429] Loss: 0.3791 | LR: 0.005
[1045/1429] Loss: 0.7993 | LR: 0.005
[1046/1429] Loss: 0.531 | LR: 0.005
[1047/1429] Loss: 0.9555 | LR: 0.005
[1048/1429] Loss: 0.4485 | LR: 0.005
[1049/1429] Loss: 0.4644 | LR: 0.005
[1050/1429] Loss: 0.7535 | LR: 0.005
[1051/1429] Loss: 0.4119 | LR: 0.005
[1052/1429] Loss: 0.6574 | LR: 0.005
[1053/1429] Loss: 0.4798 | LR: 0.005
[1054/1429] Loss: 0.7627 | LR: 0.005
[1055/1429] Loss: 0.0712 | LR: 0.005
[1056/1429] Loss: 0.7004 | LR: 0.005
[1057/1429] Loss: 0.4277 | LR: 0.005
[1058/1429] Loss: 0.2315 | LR: 0.005
[1059/1429] Loss: 0.7626 | LR: 0.005
[1060/1429] Loss: 0.0236 | LR: 0.005
[1061/1429] Loss: 0.546 | LR: 0.005
[1062/1429] Loss: 0.6108 | LR: 0.005
[1063/1429] Loss: 0.6188 | LR: 0.005
[1064/1429] Loss: 0.645 | LR: 0.005
[1065/1429] Loss: 0.2386 | LR: 0.005
[1066/1429] Loss: 0.473 | LR: 0.005
[1067/1429] Loss: 0.3237 | LR: 0.005
[1068/1429] Loss: 0.0522 | LR: 0.005
[1069/1429] Loss: 0.8146 | LR: 0.005
[1070/1429] Loss: 0.2214 | LR: 0.005
[1071/1429] Loss: 0.9822 | LR: 0.005
[1072/1429] Loss: 0.5332 | LR: 0.005
[1073/1429] Loss: 0.3278 | LR: 0.005
[1074/1429] Loss: 0.7837 | LR: 0.005
[1075/1429] Loss: 0.2676 | LR: 0.005
[1076/1429] Loss: 0.191 | LR: 0.005
[1077/1429] Loss: 0.538 | LR: 0.005
[1078/1429] Loss: 0.6599 | LR: 0.005
[1079/1429] Loss: 0.6793 | LR: 0.005
[1080/1429] Loss: 0.5368 | LR: 0.005
[1081/1429] Loss: 0.0413 | LR: 0.005
[1082/1429] Loss: 0.417 | LR: 0.005
[1083/1429] Loss: 0.4015 | LR: 0.005
[1084/1429] Loss: 0.3056 | LR: 0.005
[1085/1429] Loss: 0.7618 | LR: 0.005
[1086/1429] Loss: 0.1606 | LR: 0.005
[1087/1429] Loss: 0.106 | LR: 0.005
[1088/1429] Loss: 0.6375 | LR: 0.005
[1089/1429] Loss: 0.9855 | LR: 0.005
[1090/1429] Loss: 0.3051 | LR: 0.005
[1091/1429] Loss: 0.2512 | LR: 0.005
[1092/1429] Loss: 0.8957 | LR: 0.005
[1093/1429] Loss: 0.1821 | LR: 0.005
[1094/1429] Loss: 0.7003 | LR: 0.005
[1095/1429] Loss: 0.9991 | LR: 0.005
[1096/1429] Loss: 0.975 | LR: 0.005
[1097/1429] Loss: 0.8927 | LR: 0.005
[1098/1429] Loss: 0.5337 | LR: 0.005
[1099/1429] Loss: 0.3403 | LR: 0.005
[1100/1429] Loss: 0.2136 | LR: 0.005
[1101/1429] Loss: 0.0945 | LR: 0.005
[1102/1429] Loss: 0.5282 | LR: 0.005
[1103/1429] Loss: 0.7017 | LR: 0.005
[1104/1429] Loss: 0.114 | LR: 0.005
[1105/1429] Loss: 0.5459 | LR: 0.005
[1106/1429] Loss: 0.5734 | LR: 0.005
[1107/1429] Loss: 0.0805 | LR: 0.005
[1108/1429] Loss: 0.7742 | LR: 0.005
[1109/1429] Loss: 0.977 | LR: 0.005
[1110/1429] Loss: 0.5081 | LR: 0.005
[1111/1429] Loss: 0.9101 | LR: 0.005
[1112/1429] Loss: 0.5913 | LR: 0.005
[1113/1429] Loss: 0.8416 | LR: 0.005
[1114/1429] Loss: 0.9718 | LR: 0.005
[1115/1429] Loss: 0.077 | LR: 0.005
[1116/1429] Loss: 0.1702 | LR: 0.005
[1117/1429] Loss: 0.3416 | LR: 0.005
[1118/1429] Loss: 0.9362 | LR: 0.005
[1119/1429] Loss: 0.7459 | LR: 0.005
[1120/1429] Loss: 0.8108 | LR: 0.005
[1121/1429] Loss: 0.2392 | LR: 0.005
[1122/1429] Loss: 0.7295 | LR: 0.005
[1123/1429] Loss: 0.8777 | LR: 0.005
[1124/1429] Loss: 0.5155 | LR: 0.005
[1125/1429] Loss: 0.6059 | LR: 0.005
[1126/1429] Loss: 0.5715 | LR: 0.005
[1127/1429] Loss: 0.3532 | LR: 0.005
[1128/1429] Loss: 0.3169 | LR: 0.005
[1129/1429] Loss: 0.5189 | LR: 0.005
[1130/1429] Loss: 0.853 | LR: 0.005
[1131/1429] Loss: 0.9504 | LR: 0.005
[1132/1429] Loss: 0.639 | LR: 0.005
[1133/1429] Loss: 0.8154 | LR: 0.005
[1134/1429] Loss: 0.9571 | LR: 0.005
[1135/1429] Loss: 0.6284 | LR: 0.005
[1136/1429] Loss: 0.5872 | LR: 0.005
[1137/1429] Loss: 0.7939 | LR: 0.005
[1138/1429] Loss: 0.8462 | LR: 0.005
[1139/1429] Loss: 0.5083 | LR: 0.005
[1140/1429] Loss: 0.7354 | LR: 0.005
[1141/1429] Loss: 0.7233 | LR: 0.005
[1142/1429] Loss: 0.6857 | LR: 0.005
[1143/1429] Loss: 0.1796 | LR: 0.005
[1144/1429] Loss: 0.2448 | LR: 0.005
[1145/1429] Loss: 0.4924 | LR: 0.005
[1146/1429] Loss: 0.6267 | LR: 0.005
[1147/1429] Loss: 0.6395 | LR: 0.005
[1148/1429] Loss: 0.6588 | LR: 0.005
[1149/1429] Loss: 0.7457 | LR: 0.005
[1150/1429] Loss: 0.303 | LR: 0.005
[1151/1429] Loss: 0.0487 | LR: 0.005
[1152/1429] Loss: 0.2347 | LR: 0.005
[1153/1429] Loss: 0.5043 | LR: 0.005
[1154/1429] Loss: 0.6457 | LR: 0.005
[1155/1429] Loss: 0.3013 | LR: 0.005
[1156/1429] Loss: 0.6533 | LR: 0.005
[1157/1429] Loss: 0.2227 | LR: 0.005
[1158/1429] Loss: 0.3295 | LR: 0.005
[1159/1429] Loss: 0.9426 | LR: 0.005
[1160/1429] Loss: 0.5115 | LR: 0.005
[1161/1429] Loss: 0.9527 | LR: 0.005
[1162/1429] Loss: 0.3728 | LR: 0.005
[1163/1429] Loss: 0.2362 | LR: 0.005
[1164/1429] Loss: 0.8478 | LR: 0.005
[1165/1429] Loss: 0.5227 | LR: 0.005
[1166/1429] Loss: 0.4156 | LR: 0.005
[1167/1429] Loss: 0.8412 | LR: 0.005
[1168/1429] Loss: 0.7646 | LR: 0.005
[1169/1429] Loss: 0.3813 | LR: 0.005
[1170/1429] Loss: 0.1005 | LR: 0.005
[1171/1429] Loss: 0.6831 | LR: 0.005
[1172/1429] Loss: 0.5396 | LR: 0.005
[1173/1429] Loss: 0.4548 | LR: 0.005
[1174/1429] Loss: 0.4509 | LR: 0.005
[1175/1429] Loss: 0.0386 | LR: 0.005
[1176/1429] Loss: 0.3601 | LR: 0.005
[1177/1429] Loss: 0.1329 | LR: 0.005
[1178/1429] Loss: 0.9693 | LR: 0.005
[1179/1429] Loss: 0.2761 | LR: 0.005
[1180/1429] Loss: 0.5219 | LR: 0.005
[1181/1429] Loss: 0.3103 | LR: 0.005
[1182/1429] Loss: 0.7356 | LR: 0.005
[1183/1429] Loss: 0.9032 | LR: 0.005
[1184/1429] Loss: 0.5946 | LR: 0.005
[1185/1429] Loss: 0.2129 | LR: 0.005
[1186/1429] Loss: 0.9513 | LR: 0.005
[1187/1429] Loss: 0.4922 | LR: 0.005
[1188/1429] Loss: 0.1518 | LR: 0.005
[1189/1429] Loss: 0.245 | LR: 0.005
[1190/1429] Loss: 0.7376 | LR: 0.005
[1191/1429] Loss: 0.8968 | LR: 0.005
[1192/1429] Loss: 0.7208 | LR: 0.005
[1193/1429] Loss: 0.3183 | LR: 0.005
[1194/1429] Loss: 0.9878 | LR: 0.005
[1195/1429] Loss: 0.1514 | LR: 0.005
[1196/1429] Loss: 0.2572 | LR: 0.005
[1197/1429] Loss: 0.0996 | LR: 0.005
[1198/1429] Loss: 0.6363 | LR: 0.005
[1199/1429] Loss: 0.8883 | LR: 0.005
[1200/1429] Loss: 0.4206 | LR: 0.005
[1201/1429] Loss: 0.9835 | LR: 0.005
[1202/1429] Loss: 0.2842 | LR: 0.005
[1203/1429] Loss: 0.7342 | LR: 0.005
[1204/1429] Loss: 0.824 | LR: 0.005
[1205/1429] Loss: 0.7828 | LR: 0.005
[1206/1429] Loss: 0.2564 | LR: 0.005
[1207/1429] Loss: 0.6776 | LR: 0.005
[1208/1429] Loss: 0.2949 | LR: 0.005
[1209/1429] Loss: 0.5596 | LR: 0.005
[1210/1429] Loss: 0.8568 | LR: 0.005
[1211/1429] Loss: 0.1293 | LR: 0.005
[1212/1429] Loss: 0.9114 | LR: 0.005
[1213/1429] Loss: 0.3556 | LR: 0.005
[1214/1429] Loss: 0.472 | LR: 0.005
[1215/1429] Loss: 0.6475 | LR: 0.005
[1216/1429] Loss: 0.3274 | LR: 0.005
[1217/1429] Loss: 0.589 | LR: 0.005
[1218/1429] Loss: 0.047 | LR: 0.005
[1219/1429] Loss: 0.0198 | LR: 0.005
[1220/1429] Loss: 0.0627 | LR: 0.005
[1221/1429] Loss: 0.5773 | LR: 0.005
[1222/1429] Loss: 0.6411 | LR: 0.005
[1223/1429] Loss: 0.6868 | LR: 0.005
[1224/1429] Loss: 0.3096 | LR: 0.005
[1225/1429] Loss: 0.3643 | LR: 0.005
[1226/1429] Loss: 0.7304 | LR: 0.005
[1227/1429] Loss: 0.2875 | LR: 0.005
[1228/1429] Loss: 0.284 | LR: 0.005
[1229/1429] Loss: 0.0151 | LR: 0.005
[1230/1429] Loss: 0.8267 | LR: 0.005
[1231/1429] Loss: 0.7042 | LR: 0.005
[1232/1429] Loss: 0.9246 | LR: 0.005
[1233/1429] Loss: 0.2316 | LR: 0.005
[1234/1429] Loss: 0.2482 | LR: 0.005
[1235/1429] Loss: 0.0751 | LR: 0.005
[1236/1429] Loss: 0.0041 | LR: 0.005
[1237/1429] Loss: 0.6745 | LR: 0.005
[1238/1429] Loss: 0.1452 | LR: 0.005
[1239/1429] Loss: 0.1136 | LR: 0.005
[1240/1429] Loss: 0.0926 | LR: 0.005
[1241/1429] Loss: 0.3718 | LR: 0.005
[1242/1429] Loss: 0.149 | LR: 0.005
[1243/1429] Loss: 0.4768 | LR: 0.005
[1244/1429] Loss: 0.5332 | LR: 0.005
[1245/1429] Loss: 0.4003 | LR: 0.005
[1246/1429] Loss: 0.1666 | LR: 0.005
[1247/1429] Loss: 0.7103 | LR: 0.005
[1248/1429] Loss: 0.8369 | LR: 0.005
[1249/1429] Loss: 0.4056 | LR: 0.005
[1250/1429] Loss: 0.8557 | LR: 0.005
[1251/1429] Loss: 0.8484 | LR: 0.005
[1252/1429] Loss: 0.1951 | LR: 0.005
[1253/1429] Loss: 0.2227 | LR: 0.005
[1254/1429] Loss: 0.8528 | LR: 0.005
[1255/1429] Loss: 0.1794 | LR: 0.005
[1256/1429] Loss: 0.1924 | LR: 0.005
[1257/1429] Loss: 0.4143 | LR: 0.005
[1258/1429] Loss: 0.166 | LR: 0.005
[1259/1429] Loss: 0.5259 | LR: 0.005
[1260/1429] Loss: 0.7077 | LR: 0.005
[1261/1429] Loss: 0.808 | LR: 0.005
[1262/1429] Loss: 0.4102 | LR: 0.005
[1263/1429] Loss: 0.6963 | LR: 0.005
[1264/1429] Loss: 0.9834 | LR: 0.005
[1265/1429] Loss: 0.8613 | LR: 0.005
[1266/1429] Loss: 0.9928 | LR: 0.005
[1267/1429] Loss: 0.6631 | LR: 0.005
[1268/1429] Loss: 0.0704 | LR: 0.005
[1269/1429] Loss: 0.0703 | LR: 0.005
[1270/1429] Loss: 0.0523 | LR: 0.005
[1271/1429] Loss: 0.7359 | LR: 0.005
[1272/1429] Loss: 0.0274 | LR: 0.005
[1273/1429] Loss: 0.2705 | LR: 0.005
[1274/1429] Loss: 0.1223 | LR: 0.005
[1275/1429] Loss: 0.351 | LR: 0.005
[1276/1429] Loss: 0.8338 | LR: 0.005
[1277/1429] Loss: 0.7055 | LR: 0.005
[1278/1429] Loss: 0.3801 | LR: 0.005
[1279/1429] Loss: 0.5582 | LR: 0.005
[1280/1429] Loss: 0.5163 | LR: 0.005
[1281/1429] Loss: 0.0619 | LR: 0.005
[1282/1429] Loss: 0.8199 | LR: 0.005
[1283/1429] Loss: 0.8834 | LR: 0.005
[1284/1429] Loss: 0.3418 | LR: 0.005
[1285/1429] Loss: 0.5632 | LR: 0.005
[1286/1429] Loss: 0.7004 | LR: 0.005
[1287/1429] Loss: 0.5194 | LR: 0.005
[1288/1429] Loss: 0.714 | LR: 0.005
[1289/1429] Loss: 0.2973 | LR: 0.005
[1290/1429] Loss: 0.4727 | LR: 0.005
[1291/1429] Loss: 0.9095 | LR: 0.005
[1292/1429] Loss: 0.9636 | LR: 0.005
[1293/1429] Loss: 0.5294 | LR: 0.005
[1294/1429] Loss: 0.0204 | LR: 0.005
[1295/1429] Loss: 0.8188 | LR: 0.005
[1296/1429] Loss: 0.5275 | LR: 0.005
[1297/1429] Loss: 0.7303 | LR: 0.005
[1298/1429] Loss: 0.3756 | LR: 0.005
[1299/1429] Loss: 0.1372 | LR: 0.005
[1300/1429] Loss: 0.7211 | LR: 0.005
[1301/1429] Loss: 0.1437 | LR: 0.005
[1302/1429] Loss: 0.6377 | LR: 0.005
[1303/1429] Loss: 0.4848 | LR: 0.005
[1304/1429] Loss: 0.0752 | LR: 0.005
[1305/1429] Loss: 0.8197 | LR: 0.005
[1306/1429] Loss: 0.6962 | LR: 0.005
[1307/1429] Loss: 0.1312 | LR: 0.005
[1308/1429] Loss: 0.6059 | LR: 0.005
[1309/1429] Loss: 0.9387 | LR: 0.005
[1310/1429] Loss: 0.555 | LR: 0.005
[1311/1429] Loss: 0.535 | LR: 0.005
[1312/1429] Loss: 0.7422 | LR: 0.005
[1313/1429] Loss: 0.4342 | LR: 0.005
[1314/1429] Loss: 0.5729 | LR: 0.005
[1315/1429] Loss: 0.5096 | LR: 0.005
[1316/1429] Loss: 0.7667 | LR: 0.005
[1317/1429] Loss: 0.067 | LR: 0.005
[1318/1429] Loss: 0.6004 | LR: 0.005
[1319/1429] Loss: 0.1484 | LR: 0.005
[1320/1429] Loss: 0.6279 | LR: 0.005
[1321/1429] Loss: 0.7801 | LR: 0.005
[1322/1429] Loss: 0.8841 | LR: 0.005
[1323/1429] Loss: 0.6681 | LR: 0.005
[1324/1429] Loss: 0.404 | LR: 0.005
[1325/1429] Loss: 0.6035 | LR: 0.005
[1326/1429] Loss: 0.7294 | LR: 0.005
[1327/1429] Loss: 0.5047 | LR: 0.005
[1328/1429] Loss: 0.039 | LR: 0.005
[1329/1429] Loss: 0.8949 | LR: 0.005
[1330/1429] Loss: 0.3462 | LR: 0.005
[1331/1429] Loss: 0.0991 | LR: 0.005
[1332/1429] Loss: 0.9431 | LR: 0.005
[1333/1429] Loss: 0.736 | LR: 0.005
[1334/1429] Loss: 0.2734 | LR: 0.005
[1335/1429] Loss: 0.8594 | LR: 0.005
[1336/1429] Loss: 0.1228 | LR: 0.005
[1337/1429] Loss: 0.0499 | LR: 0.005
[1338/1429] Loss: 0.7123 | LR: 0.005
[1339/1429] Loss: 0.5396 | LR: 0.005
[1340/1429] Loss: 0.4727 | LR: 0.005
[1341/1429] Loss: 0.4477 | LR: 0.005
[1342/1429] Loss: 0.242 | LR: 0.005
[1343/1429] Loss: 0.1408 | LR: 0.005
[1344/1429] Loss: 0.5507 | LR: 0.005
[1345/1429] Loss: 0.2429 | LR: 0.005
[1346/1429] Loss: 0.9363 | LR: 0.005
[1347/1429] Loss: 0.81 | LR: 0.005
[1348/1429] Loss: 0.7907 | LR: 0.005
[1349/1429] Loss: 0.2546 | LR: 0.005
[1350/1429] Loss: 0.2845 | LR: 0.005
[1351/1429] Loss: 0.1122 | LR: 0.005
[1352/1429] Loss: 0.7543 | LR: 0.005
[1353/1429] Loss: 0.325 | LR: 0.005
[1354/1429] Loss: 0.2068 | LR: 0.005
[1355/1429] Loss: 0.38 | LR: 0.005
[1356/1429] Loss: 0.5056 | LR: 0.005
[1357/1429] Loss: 0.4477 | LR: 0.005
[1358/1429] Loss: 0.7557 | LR: 0.005
[1359/1429] Loss: 0.2757 | LR: 0.005
[1360/1429] Loss: 0.9707 | LR: 0.005
[1361/1429] Loss: 0.5545 | LR: 0.005
[1362/1429] Loss: 0.5212 | LR: 0.005
[1363/1429] Loss: 0.919 | LR: 0.005
[1364/1429] Loss: 0.8559 | LR: 0.005
[1365/1429] Loss: 0.6865 | LR: 0.005
[1366/1429] Loss: 0.4455 | LR: 0.005
[1367/1429] Loss: 0.6007 | LR: 0.005
[1368/1429] Loss: 0.6394 | LR: 0.005
[1369/1429] Loss: 0.8342 | LR: 0.005
[1370/1429] Loss: 0.5055 | LR: 0.005
[1371/1429] Loss: 0.4306 | LR: 0.005
[1372/1429] Loss: 0.9753 | LR: 0.005
[1373/1429] Loss: 0.0444 | LR: 0.005
[1374/1429] Loss: 0.485 | LR: 0.005
[1375/1429] Loss: 0.2898 | LR: 0.005
[1376/1429] Loss: 0.9796 | LR: 0.005
[1377/1429] Loss: 0.1453 | LR: 0.005
[1378/1429] Loss: 0.6939 | LR: 0.005
[1379/1429] Loss: 0.4088 | LR: 0.005
[1380/1429] Loss: 0.8813 | LR: 0.005
[1381/1429] Loss: 0.811 | LR: 0.005
[1382/1429] Loss: 0.7315 | LR: 0.005
[1383/1429] Loss: 0.4533 | LR: 0.005
[1384/1429] Loss: 0.8855 | LR: 0.005
[1385/1429] Loss: 0.851 | LR: 0.005
[1386/1429] Loss: 0.0816 | LR: 0.005
[1387/1429] Loss: 0.5849 | LR: 0.005
[1388/1429] Loss: 0.3884 | LR: 0.005
[1389/1429] Loss: 0.7023 | LR: 0.005
[1390/1429] Loss: 0.7159 | LR: 0.005
[1391/1429] Loss: 0.359 | LR: 0.005
[1392/1429] Loss: 0.0847 | LR: 0.005
[1393/1429] Loss: 0.6196 | LR: 0.005
[1394/1429] Loss: 0.0582 | LR: 0.005
[1395/1429] Loss: 0.559 | LR: 0.005
[1396/1429] Loss: 0.3619 | LR: 0.005
[1397/1429] Loss: 0.1872 | LR: 0.005
[1398/1429] Loss: 0.4817 | LR: 0.005
[1399/1429] Loss: 0.916 | LR: 0.005
[1400/1429] Loss: 0.0634 | LR: 0.005
[1401/1429] Loss: 0.6492 | LR: 0.005
[1402/1429] Loss: 0.7048 | LR: 0.005
[1403/1429] Loss: 0.8013 | LR: 0.005
[1404/1429] Loss: 0.0839 | LR: 0.005
[1405/1429] Loss: 0.245 | LR: 0.005
[1406/1429] Loss: 0.4363 | LR: 0.005
[1407/1429] Loss: 0.734 | LR: 0.005
[1408/1429] Loss: 0.5863 | LR: 0.005
[1409/1429] Loss: 0.734 | LR: 0.005
[1410/1429] Loss: 0.0726 | LR: 0.005
[1411/1429] Loss: 0.482 | LR: 0.005
[1412/1429] Loss: 0.9438 | LR: 0.005
[1413/1429] Loss: 0.1569 | LR: 0.005
[1414/1429] Loss: 0.2129 | LR: 0.005
[1415/1429] Loss: 0.9101 | LR: 0.005
[1416/1429] Loss: 0.5512 | LR: 0.005
[1417/1429] Loss: 0.1749 | LR: 0.005
[1418/1429] Loss: 0.589 | LR: 0.005
[1419/1429] Loss: 0.2885 | LR: 0.005
[1420/1429] Loss: 0.6977 | LR: 0.005
[1421/1429] Loss: 0.5818 | LR: 0.005
[1422/1429] Loss: 0.7934 | LR: 0.005
[1423/1429] Loss: 0.8304 | LR: 0.005
[1424/1429] Loss: 0.8986 | LR: 0.005
[1425/1429] Loss: 0.8549 | LR: 0.005
[1426/1429] Loss: 0.1559 | LR: 0.005
[1427/1429] Loss: 0.8045 | LR: 0.005
[1428/1429] Loss: 0.3365 | LR: 0.005
[1429/1429] Loss: 0.6028 | LR: 0.005
[1429/1429] Epoch Time: 11m 33s

~~~~

Training complete in 34m 42s
Best Loss: 0.3891
Model checkpoint saved to: outputs/maskrcnn_pascalvoc_final.pth

In [ ]:
model.eval()
with torch.no_grad():
    for images, targets in test_loader:
        images = list(img.to(device) for img in images)
        outputs = model(images)  

        import matplotlib.pyplot as plt
        from matplotlib.patches import Rectangle

        img_np = np.array(images[0].cpu().permute(1,2,0))
        plt.imshow(img_np)
        ax = plt.gca()
        for box in outputs[0]['boxes'].cpu():
            xmin, ymin, xmax, ymax = box.numpy()
            ax.add_patch(Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, fill=False, color='red', linewidth=2))
        plt.title('Predicted Boxes')
        plt.show()
        break  
log_file.close()
